<a href="https://colab.research.google.com/github/Justin-Hwang/EEG-AD-FTD-Detection/blob/main/AD_CN_and_AD_FTD_Multiclass_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 그중에 2025 Lab Research 폴더 안을 확인
!ls "/content/drive/MyDrive/2025_Lab_Research"

import sys
sys.path.append('/content/drive/MyDrive/2025_Lab_Research')

import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Running on", DEVICE)  # → “cuda” 가 뜨면 GPU 정상

'AD vs FTD vs CN Binary Classification'   eeg_holdout-5.db
'Binary Classification LOSO Test.ipynb'   eeg_holdout-6.db
'Colab Files'				  eeg_holdout-7.db
'Data Preparation.gdoc'			  eeg_holdout-8.db
 eeg_dataset.py				  eeg_holdout-9.db
 EEGformer_model_training.ipynb		  eeg_holdout.db
 eegformer_optuna_cv_3.db		  eeg_holdout_fixed_1.db
 eegformer_optuna_cv_4.db		  eeg_optuna_trial_1.db
 eegformer_optuna_cv_5.db		  eeg_optuna_trial_2.db
 eeg_grid_search-10.db			  eeg_optuna_trial_3.db
 eeg_grid_search-11.db			 'EEG Transformer Architecture.gdoc'
 eeg_grid_search-12.db			 'Lab Info'
 eeg_grid_search-13.db			 'Lab Research Final Report'
 eeg_grid_search-14.db			 'Lab Research Paper Review'
 eeg_grid_search-15.db			 'Meeting Note.gdoc'
 eeg_grid_search-16.db			  model-data
 eeg_grid_search-17.db			  model-data.zip
 eeg_grid_search-18.db			  model_optimized_2.py
 eeg_grid_search-2.db			  model_optimized_3.py
 eeg_grid_search-3.db			  model_optimized_4.py
 eeg_grid_search-4.db			  model_opt

In [ ]:
import wandb
wandb.login()  # 첫 실행 시 API 키 입력

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jh8032 (jh8032-new-york-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
!pip install optuna
!pip install wandb
!pip install mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 117.7 MB/s eta 0:00:00


### Balancing the Dataset
- To overcome the dataset imbalance

In [ ]:
import os
import json
import random
from collections import Counter

from eeg_dataset import EEGDataset
from torch.utils.data import DataLoader

# 재현성을 위해 랜덤 시드 고정
random.seed(0)

# ─── 설정 ─────────────────────────────────────────────────────────
DATA_DIR   = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE = 'labels.json'
BATCH_SIZE = 32
NUM_WORKERS = 4  # 환경에 맞게 조정

# ─── 1) labels.json 로드 & train 데이터만 추출 ─────────────────────
with open(os.path.join(DATA_DIR, LABEL_FILE), 'r') as f:
    all_meta = json.load(f)
train_data = [d for d in all_meta if d['type'] == 'train']

# ─── 2) 클래스별 분리 ───────────────────────────────────────────────
train_data_A = [d for d in train_data if d['label'] == 'A']  # AD
train_data_C = [d for d in train_data if d['label'] == 'C']  # CN
train_data_F = [d for d in train_data if d['label'] == 'F']  # FTD

# ─── 3) 원본 분포 출력 ─────────────────────────────────────────────
print("Before Balancing")
print(f"  A (AD): {len(train_data_A)}")
print(f"  C (CN): {len(train_data_C)}")
print(f"  F (FTD): {len(train_data_F)}")

# ─── 4) Manual Class Balancing 계산 ───────────────────────────────
# 제시하신 방식: 각 클래스 쌍의 평균 크기 중 최소값
min_samples = min(
    (len(train_data_A) + len(train_data_C)) / 2,
    (len(train_data_A) + len(train_data_F)) / 2,
    (len(train_data_C) + len(train_data_F)) / 2
)

a_index = int(min(min_samples, len(train_data_A)))
c_index = int(min(min_samples, len(train_data_C)))
f_index = int(min(min_samples, len(train_data_F)))

# ─── 5) 언더샘플링 수행 ───────────────────────────────────────────
balanced_train_data = (
    random.sample(train_data_A, a_index) +
    random.sample(train_data_C, c_index) +
    random.sample(train_data_F, f_index)
)
random.shuffle(balanced_train_data)

# ─── 6) 언더샘플링 후 분포 출력 ───────────────────────────────────
print("\nAfter Balancing")
print(f"  A (AD): {a_index}")
print(f"  C (CN): {c_index}")
print(f"  F (FTD): {f_index}")
print(f"  Total: {len(balanced_train_data)}")

# ─── 7) EEGDataset + DataLoader 예시 ─────────────────────────────
dataset = EEGDataset(DATA_DIR, balanced_train_data)
loader  = DataLoader(dataset,
                     batch_size=BATCH_SIZE,
                     shuffle=True,
                     num_workers=NUM_WORKERS)

# loader를 돌면서 데이터 확인 예시
for X, y in loader:
    print("Batch X shape:", X.shape)  # (batch, channels, timepoints)
    print("Batch y:", y)
    break


Now using CUDA device 0
Enabling CUDA with 39.14 GiB available memory
Before Balancing
  A (AD): 1388
  C (CN): 1102
  F (FTD): 729

After Balancing
  A (AD): 915
  C (CN): 915
  F (FTD): 729
  Total: 2559
Batch X shape: torch.Size([32, 19, 1425])
Batch y: tensor([2, 0, 1, 1, 2, 0, 1, 0, 1, 0, 1, 1, 0, 0, 2, 0, 2, 0, 2, 2, 2, 0, 0, 1,
        2, 1, 0, 0, 1, 2, 1, 1])


In [ ]:
!pip install lightgbm scikit-learn

#### Baseline Results (Machine Learning) AD vs CN
- LightGBM
- SVM
- KNN
- MLP
- Random Forests

In [ ]:
# baseline_models.py  – Classical baselines for FTD vs CN
# -------------------------------------------------------
import os, json, random, time, copy
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from sklearn.pipeline       import Pipeline
from sklearn.preprocessing  import StandardScaler
from sklearn.metrics        import (
    recall_score, f1_score, accuracy_score, confusion_matrix
)

from sklearn.svm            import SVC
from sklearn.neighbors      import KNeighborsClassifier
from sklearn.ensemble       import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from lightgbm               import LGBMClassifier   # pip install lightgbm

from eeg_dataset import EEGDataset                 # ← your existing loader

# ─── Reproducibility ────────────────────────────────────────────
SEED = 0
random.seed(SEED)
np.random.seed(SEED)

# ─── Paths & constants (adjust as needed) ───────────────────────
DATA_DIR   = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE = 'labels.json'
NUM_WORKERS = 4               # DataLoader workers

# ─── Dataset: returns flat NumPy feature vector + label ─────────
class FlatEEGDataset(Dataset):
    """
    Wraps EEGDataset but outputs (1‑D float32 NumPy array, int label).
    Works whether EEGDataset returns a tensor or an ndarray.
    """
    def __init__(self, raw_ds, metas, label_map):
        self.raw_ds = raw_ds
        self.labels = [label_map[d['label']] if isinstance(d['label'], str)
                       else d['label'] for d in metas]

    def __len__(self): return len(self.raw_ds)

    def __getitem__(self, idx):
        x, _ = self.raw_ds[idx]                       # tensor OR ndarray
        if isinstance(x, torch.Tensor):
            x = x.flatten().cpu().numpy()
        else:  # already ndarray
            x = x.flatten()
        return x.astype(np.float32), self.labels[idx]

# ─── Utility functions ──────────────────────────────────────────
def count_labels(meta, cls0='F', cls1='C'):
    n0 = sum(1 for d in meta if d['label'] in (cls0, 0))
    n1 = len(meta) - n0
    return n0, n1

def metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    spec = tn / (tn + fp) if (tn + fp) else 0.0
    return {
        'acc':  accuracy_score(y_true, y_pred),
        'rec':  recall_score(y_true, y_pred, zero_division=0),  # sensitivity
        'spec': spec,
        'f1':   f1_score(y_true, y_pred, zero_division=0)
    }

def report(name, res):
    print(f"{name:>11s}: "
          f"Accuracy={res['acc']:.4f} "
          f"Sensitivity={res['rec']:.4f} "
          f"Specificity={res['spec']:.4f} "
          f"F1={res['f1']:.4f}")

# ─── Load label metadata (same JSON your EEGformer uses) ────────
with open(Path(DATA_DIR) / LABEL_FILE, 'r') as f:
    all_meta = json.load(f)

train_meta       = [d for d in all_meta if d['type'] == 'train']
test_within_meta = [d for d in all_meta if d['type'] == 'test_within']
test_cross_meta  = [d for d in all_meta if d['type'] == 'test_cross']

class0, class1   = 'A', 'C'            # AD vs CN
label_map        = {class0: 0, class1: 1}

# Keep only F & C in evaluation splits
test_within_meta = [d for d in test_within_meta if d['label'] in (class0, class1)]
test_cross_meta  = [d for d in test_cross_meta  if d['label'] in (class0, class1)]

# ─── Balance training split (down‑sample larger class) ──────────
data0 = [d for d in train_meta if d['label'] == class0]
data1 = [d for d in train_meta if d['label'] == class1]
k     = min(len(data0), len(data1))
balanced_meta = random.sample(data0, k) + random.sample(data1, k)
random.shuffle(balanced_meta)

for d in balanced_meta:
    d['label'] = label_map[d['label']]     # convert to int

print(f"\n[Balanced] TRAIN  total={len(balanced_meta)} (AD={k}, CN={k})")

# ─── Convert each split to NumPy (X, y) -------------------------
def make_numpy(metas):
    raw = EEGDataset(DATA_DIR, metas)
    ds  = FlatEEGDataset(raw, metas, label_map)
    loader = DataLoader(ds, batch_size=128, shuffle=False,
                        num_workers=NUM_WORKERS)
    X, y = [], []
    for xb, yb in loader:          # xb is *tensor* batch
        X.append(xb)               # keep on CPU
        y.append(yb)
    return torch.cat(X).numpy(), torch.cat(y).numpy()

X_train, y_train = make_numpy(balanced_meta)
X_tw,    y_tw    = make_numpy(test_within_meta)
X_tc,    y_tc    = make_numpy(test_cross_meta)

print(f"Input feature vector length per sample = {X_train.shape[1]}")

# ─── Define classical models ------------------------------------
models = {
    "LightGBM" : LGBMClassifier(
        n_estimators=100, learning_rate=0.001,
        random_state=SEED, class_weight='balanced'
    ),
    "SVM"      : SVC(
        kernel='rbf', probability=False,
        class_weight='balanced', random_state=SEED, gamma='scale'
    ),
    "KNN"      : KNeighborsClassifier(
        n_neighbors=7, weights='distance'
    ),
    "MLP"      : MLPClassifier(
        hidden_layer_sizes=(256, 128), max_iter=200,
        activation='relu', solver='adam', random_state=SEED
    ),
    "RandomForest": RandomForestClassifier(
        n_estimators=100, max_depth=None,
        random_state=SEED, class_weight='balanced'
    )
}

# ─── Train & evaluate -------------------------------------------
print("\n=== Final Evaluation on Test Sets (Classical Baselines) ===")
for name, clf in models.items():
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('clf',    clf)
    ])

    t0 = time.time()
    pipe.fit(X_train, y_train)
    train_time = time.time() - t0

    y_pred_tw = pipe.predict(X_tw)
    y_pred_tc = pipe.predict(X_tc)

    res_tw = metrics(y_tw, y_pred_tw)
    res_tc = metrics(y_tc, y_pred_tc)

    print(f"\n-- {name} (train time {train_time:.1f}s) --")
    print(f"   test_within  total={len(y_tw)}")
    report("test_within", res_tw)
    print(f"   test_cross   total={len(y_tc)}")
    report("test_cross",  res_tc)

Attempting to create new mne-python configuration file:
/root/.mne/mne-python.json
Now using CUDA device 0
Enabling CUDA with 39.14 GiB available memory

[Balanced] TRAIN  total=2204 (AD=1102, CN=1102)
Input feature vector length per sample = 27075

=== Final Evaluation on Test Sets (Classical Baselines) ===


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 1102, number of negative: 1102
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.845540 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6904125
[LightGBM] [Info] Number of data points in the train set: 2204, number of used features: 27075
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



-- LightGBM (train time 83.5s) --
   test_within  total=272
test_within: Accuracy=0.5110 Sensitivity=0.4762 Specificity=0.5411 F1=0.4743
   test_cross   total=626
 test_cross: Accuracy=0.4776 Sensitivity=0.4853 Specificity=0.4702 F1=0.4768

-- SVM (train time 36.2s) --
   test_within  total=272
test_within: Accuracy=0.4779 Sensitivity=0.4841 Specificity=0.4726 F1=0.4621
   test_cross   total=626
 test_cross: Accuracy=0.4521 Sensitivity=0.4560 Specificity=0.4483 F1=0.4494

-- KNN (train time 0.7s) --
   test_within  total=272
test_within: Accuracy=0.5956 Sensitivity=0.5317 Specificity=0.6507 F1=0.5492
   test_cross   total=626
 test_cross: Accuracy=0.4872 Sensitivity=0.4593 Specificity=0.5141 F1=0.4677

-- MLP (train time 45.5s) --
   test_within  total=272
test_within: Accuracy=0.4963 Sensitivity=0.5000 Specificity=0.4932 F1=0.4791
   test_cross   total=626
 test_cross: Accuracy=0.5000 Sensitivity=0.4919 Specificity=0.5078 F1=0.4911

-- RandomForest (train time 42.7s) --
   test_withi

#### AD vs FTD Baseline Results

In [ ]:
# baseline_models.py  – Classical baselines for AD vs FTD
# -------------------------------------------------------
import os, json, random, time, copy
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from sklearn.pipeline       import Pipeline
from sklearn.preprocessing  import StandardScaler
from sklearn.metrics        import (
    recall_score, f1_score, accuracy_score, confusion_matrix
)

from sklearn.svm            import SVC
from sklearn.neighbors      import KNeighborsClassifier
from sklearn.ensemble       import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from lightgbm               import LGBMClassifier   # pip install lightgbm

from eeg_dataset import EEGDataset                 # ← your existing loader

# ─── Reproducibility ────────────────────────────────────────────
SEED = 0
random.seed(SEED)
np.random.seed(SEED)

# ─── Paths & constants (adjust as needed) ───────────────────────
DATA_DIR   = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE = 'labels.json'
NUM_WORKERS = 4               # DataLoader workers

# ─── Dataset: returns flat NumPy feature vector + label ─────────
class FlatEEGDataset(Dataset):
    """
    Wraps EEGDataset but outputs (1‑D float32 NumPy array, int label).
    Works whether EEGDataset returns a tensor or an ndarray.
    """
    def __init__(self, raw_ds, metas, label_map):
        self.raw_ds = raw_ds
        self.labels = [label_map[d['label']] if isinstance(d['label'], str)
                       else d['label'] for d in metas]

    def __len__(self): return len(self.raw_ds)

    def __getitem__(self, idx):
        x, _ = self.raw_ds[idx]                       # tensor OR ndarray
        if isinstance(x, torch.Tensor):
            x = x.flatten().cpu().numpy()
        else:  # already ndarray
            x = x.flatten()
        return x.astype(np.float32), self.labels[idx]

# ─── Utility functions ──────────────────────────────────────────
def count_labels(meta, cls0='A', cls1='F'):
    n0 = sum(1 for d in meta if d['label'] in (cls0, 0))
    n1 = len(meta) - n0
    return n0, n1

def metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    spec = tn / (tn + fp) if (tn + fp) else 0.0
    return {
        'acc':  accuracy_score(y_true, y_pred),
        'rec':  recall_score(y_true, y_pred, zero_division=0),  # sensitivity
        'spec': spec,
        'f1':   f1_score(y_true, y_pred, zero_division=0)
    }

def report(name, res):
    print(f"{name:>11s}: "
          f"Accuracy={res['acc']:.4f} "
          f"Sensitivity={res['rec']:.4f} "
          f"Specificity={res['spec']:.4f} "
          f"F1={res['f1']:.4f}")

# ─── Load label metadata (same JSON your EEGformer uses) ────────
with open(Path(DATA_DIR) / LABEL_FILE, 'r') as f:
    all_meta = json.load(f)

train_meta       = [d for d in all_meta if d['type'] == 'train']
test_within_meta = [d for d in all_meta if d['type'] == 'test_within']
test_cross_meta  = [d for d in all_meta if d['type'] == 'test_cross']

class0, class1   = 'A', 'F'            # AD vs FTD
label_map        = {class0: 0, class1: 1}

# Keep only A & F in evaluation splits
test_within_meta = [d for d in test_within_meta if d['label'] in (class0, class1)]
test_cross_meta  = [d for d in test_cross_meta  if d['label'] in (class0, class1)]

# ─── Balance training split (down‑sample larger class) ──────────
data0 = [d for d in train_meta if d['label'] == class0]
data1 = [d for d in train_meta if d['label'] == class1]
k     = min(len(data0), len(data1))
balanced_meta = random.sample(data0, k) + random.sample(data1, k)
random.shuffle(balanced_meta)

for d in balanced_meta:
    d['label'] = label_map[d['label']]     # convert to int

print(f"\n[Balanced] TRAIN  total={len(balanced_meta)} (AD={k}, FTD={k})")

# ─── Convert each split to NumPy (X, y) -------------------------
def make_numpy(metas):
    raw = EEGDataset(DATA_DIR, metas)
    ds  = FlatEEGDataset(raw, metas, label_map)
    loader = DataLoader(ds, batch_size=128, shuffle=False,
                        num_workers=NUM_WORKERS)
    X, y = [], []
    for xb, yb in loader:          # xb is *tensor* batch
        X.append(xb)               # keep on CPU
        y.append(yb)
    return torch.cat(X).numpy(), torch.cat(y).numpy()

X_train, y_train = make_numpy(balanced_meta)
X_tw,    y_tw    = make_numpy(test_within_meta)
X_tc,    y_tc    = make_numpy(test_cross_meta)

print(f"Input feature vector length per sample = {X_train.shape[1]}")

# ─── Define classical models ------------------------------------
models = {
    "LightGBM" : LGBMClassifier(
        n_estimators=100, learning_rate=0.001,
        random_state=SEED, class_weight='balanced'
    ),
    "SVM"      : SVC(
        kernel='rbf', probability=False,
        class_weight='balanced', random_state=SEED, gamma='scale'
    ),
    "KNN"      : KNeighborsClassifier(
        n_neighbors=7, weights='distance'
    ),
    "MLP"      : MLPClassifier(
        hidden_layer_sizes=(256, 128), max_iter=200,
        activation='relu', solver='adam', random_state=SEED
    ),
    "RandomForest": RandomForestClassifier(
        n_estimators=100, max_depth=None,
        random_state=SEED, class_weight='balanced'
    )
}

# ─── Train & evaluate -------------------------------------------
print("\n=== Final Evaluation on Test Sets (Classical Baselines) ===")
for name, clf in models.items():
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('clf',    clf)
    ])

    t0 = time.time()
    pipe.fit(X_train, y_train)
    train_time = time.time() - t0

    y_pred_tw = pipe.predict(X_tw)
    y_pred_tc = pipe.predict(X_tc)

    res_tw = metrics(y_tw, y_pred_tw)
    res_tc = metrics(y_tc, y_pred_tc)

    print(f"\n-- {name} (train time {train_time:.1f}s) --")
    print(f"   test_within  total={len(y_tw)}")
    report("test_within", res_tw)
    print(f"   test_cross   total={len(y_tc)}")
    report("test_cross",  res_tc)


[Balanced] TRAIN  total=1458 (AD=729, FTD=729)
Input feature vector length per sample = 27075

=== Final Evaluation on Test Sets (Classical Baselines) ===


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 729, number of negative: 729
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.868109 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6904125
[LightGBM] [Info] Number of data points in the train set: 1458, number of used features: 27075
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



-- LightGBM (train time 71.4s) --
   test_within  total=218
test_within: Accuracy=0.5183 Sensitivity=0.4306 Specificity=0.5616 F1=0.3713
   test_cross   total=566
 test_cross: Accuracy=0.4982 Sensitivity=0.4615 Specificity=0.5266 F1=0.4453

-- SVM (train time 16.8s) --
   test_within  total=218
test_within: Accuracy=0.5505 Sensitivity=0.5972 Specificity=0.5274 F1=0.4674
   test_cross   total=566
 test_cross: Accuracy=0.4770 Sensitivity=0.4777 Specificity=0.4765 F1=0.4436

-- KNN (train time 0.5s) --
   test_within  total=218
test_within: Accuracy=0.5229 Sensitivity=0.5000 Specificity=0.5342 F1=0.4091
   test_cross   total=566
 test_cross: Accuracy=0.4664 Sensitivity=0.4534 Specificity=0.4765 F1=0.4259

-- MLP (train time 31.7s) --
   test_within  total=218
test_within: Accuracy=0.5505 Sensitivity=0.5556 Specificity=0.5479 F1=0.4494
   test_cross   total=566
 test_cross: Accuracy=0.4859 Sensitivity=0.4899 Specificity=0.4828 F1=0.4540

-- RandomForest (train time 23.9s) --
   test_withi

### Binary Classification
- A = AD (= 0)
- C = CN (= 1)
- F = FTD (= 2)


### 1) AD vs CN Experiment

#### Before using Optuna, try 5-fold cross val to see whether the model generalize well

In [ ]:
import os
import json
import random
import time
import numpy as np
import torch
import torch.nn as nn
import wandb
from wandb import Settings
from collections import Counter
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score
from torch.utils.data import DataLoader, Subset, Dataset

from eeg_dataset import EEGDataset
from model_optimized_5 import EEGformer

# ─── Configuration ───────────────────────────────────────────────
DATA_DIR     = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE   = 'labels.json'
DEVICE       = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

LR           = 1e-3
MAX_EPOCHS   = 100
BATCH_SIZE   = 32
NUM_WORKERS  = 4
ES_PATIENCE  = 10    # Early Stopping patience

# ─── Wrapper to override labels inside our experiment ────────────
class BinaryEEGDataset(Dataset):
    def __init__(self, raw_ds, metas):
        """
        raw_ds: instance of EEGDataset
        metas: list of dicts with 'file_name' and mapped integer 'label'
        """
        self.raw_ds = raw_ds
        self.metas  = metas

    def __len__(self):
        return len(self.raw_ds)

    def __getitem__(self, idx):
        x, _ = self.raw_ds[idx]            # discard raw label
        y     = self.metas[idx]['label']   # use our mapped 0/1 label
        return x, y

# ─── Which binary pairs to run ───────────────────────────────────
binary_pairs = [
    ("A", "C"),  # AD vs CN
    #("F", "C"),  # FTD vs CN
    #("A", "F"),  # AD vs FTD
]

random.seed(0)
final_results = {}

# ─── Load all train metadata ─────────────────────────────────────
with open(os.path.join(DATA_DIR, LABEL_FILE), 'r') as f:
    all_meta = json.load(f)
train_meta_all = [d for d in all_meta if d["type"] == "train"]

for class0, class1 in binary_pairs:
    print(f"\n=== Experiment: {class0} vs {class1} ===")

    # 1) filter for the two classes
    data0 = [d for d in train_meta_all if d['label'] == class0]
    data1 = [d for d in train_meta_all if d['label'] == class1]

    # 2) decide undersample size (minority class)
    min_count = min(len(data0), len(data1))

    # 3) random undersample
    samp0 = random.sample(data0, k=min_count)
    samp1 = random.sample(data1, k=min_count)
    balanced_meta = samp0 + samp1
    random.shuffle(balanced_meta)

    # 4) map labels to 0/1
    label_map = {class0: 0, class1: 1}
    for d in balanced_meta:
        d['label'] = label_map[d['label']]

    # 5) print balanced counts
    counts = Counter(d['label'] for d in balanced_meta)
    print("  Balanced counts:", counts)

    # 6) build raw EEGDataset and wrap it
    raw_ds = EEGDataset(DATA_DIR, balanced_meta)
    dataset = BinaryEEGDataset(raw_ds, balanced_meta)
    labels  = np.array([d['label'] for d in balanced_meta])

    # 7) 5-Fold Stratified CV
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    fold_accs, fold_precs, fold_recs, fold_f1s = [], [], [], []

    for fold, (train_idx, val_idx) in enumerate(
            skf.split(np.zeros(len(labels)), labels), start=1):

        print(f"\n--- Fold {fold} ---")
        wandb.init(
            project="eeg-binary-AD-CN-cv_1",
            name=f"{class0}_vs_{class1}_fold{fold}",
            config={
                "lr":           LR,
                "max_epochs":   MAX_EPOCHS,
                "batch_size":   BATCH_SIZE,
                "class_pair":   f"{class0} vs {class1}"
            },
            settings=Settings(init_timeout=120)
        )

        train_ds = Subset(dataset, train_idx)
        val_ds   = Subset(dataset, val_idx)
        train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE,
                                  shuffle=True,  num_workers=NUM_WORKERS)
        val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE,
                                  shuffle=False, num_workers=NUM_WORKERS)

        # 8) init model, optimizer, loss
        input_len = dataset[0][0].shape[-1]
        model     = EEGformer(
            in_channels=19,
            input_length=input_len,
            kernel_size=10,
            num_filters=60,
            num_heads=3,
            num_blocks=1,
            num_segments=5,
            num_classes=2,
        ).to(DEVICE)

        optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
        criterion = nn.CrossEntropyLoss()

        # ─── Early Stopping variables ────────────────────────────
        best_val_loss = float("inf")
        es_count      = 0

        # 9) epoch loop
        for epoch in range(1, MAX_EPOCHS + 1):
            t0 = time.time()

            # — train —
            model.train()
            tloss = tcorrect = ttotal = 0
            for X, y in train_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                optimizer.zero_grad()
                logits = model(X)
                loss   = criterion(logits, y)
                loss.backward()
                optimizer.step()

                tloss    += loss.item()
                preds     = logits.argmax(1)
                tcorrect += (preds == y).sum().item()
                ttotal   += y.size(0)

            train_loss = tloss / len(train_loader)
            train_acc  = tcorrect / ttotal

            # — validate —
            model.eval()
            vloss = vcorrect = vtotal = 0
            val_preds, val_labels = [], []
            with torch.no_grad():
                for X, y in val_loader:
                    X, y = X.to(DEVICE), y.to(DEVICE)
                    logits = model(X)
                    loss   = criterion(logits, y)
                    vloss  += loss.item()

                    preds = logits.argmax(1)
                    vcorrect += (preds == y).sum().item()
                    vtotal   += y.size(0)

                    val_preds.append(preds.cpu().numpy())
                    val_labels.append(y.cpu().numpy())

            val_preds  = np.concatenate(val_preds)
            val_labels = np.concatenate(val_labels)
            val_loss   = vloss / len(val_loader)
            val_acc    = (val_preds == val_labels).mean()
            precision  = precision_score(val_labels, val_preds, zero_division=0)
            recall     = recall_score(val_labels, val_preds, zero_division=0)
            f1         = f1_score(val_labels, val_preds, zero_division=0)
            elapsed    = time.time() - t0

            # — terminal print & wandb log —
            print(
                f"Epoch {epoch:03d} | "
                f"train_loss={train_loss:.4f} acc={train_acc:.4f} | "
                f"val_loss={val_loss:.4f} acc={val_acc:.4f} | "
                f"prec={precision:.4f} rec={recall:.4f} f1={f1:.4f} | "
                f"time={elapsed:.1f}s"
            )
            wandb.log({
                "epoch":               epoch,
                "train_loss":          train_loss,
                "train_accuracy":      train_acc,
                "validation_loss":     val_loss,
                "validation_accuracy": val_acc,
                "precision":           precision,
                "recall":              recall,
                "f1_score":            f1
            }, step=epoch)

            # ─── Early Stopping check ─────────────────────────────
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                es_count = 0
            else:
                es_count += 1
                if es_count >= ES_PATIENCE:
                    print(f"Early stopping at epoch {epoch}")
                    break

        # record final metrics for this fold
        fold_accs.append(val_acc)
        fold_precs.append(precision)
        fold_recs.append(recall)
        fold_f1s.append(f1)

        wandb.finish()
        torch.cuda.empty_cache()

    # 10) compute and print average metrics across folds
    mean_acc  = np.mean(fold_accs)
    mean_prec = np.mean(fold_precs)
    mean_rec  = np.mean(fold_recs)
    mean_f1   = np.mean(fold_f1s)
    final_results[f"{class0} vs {class1}"] = {
        "accuracy":  mean_acc,
        "precision": mean_prec,
        "recall":    mean_rec,
        "f1_score":  mean_f1
    }
    print(
        f"\n>>> Average for {class0} vs {class1}: "
        f"acc={mean_acc:.4f}, "
        f"prec={mean_prec:.4f}, "
        f"rec={mean_rec:.4f}, "
        f"f1={mean_f1:.4f}"
    )

# ─── Final summary ───────────────────────────────────────────────
print("\n=== Final Results ===")
for exp, metrics in final_results.items():
    print(
        f"{exp}: "
        f"acc={metrics['accuracy']:.4f}, "
        f"prec={metrics['precision']:.4f}, "
        f"rec={metrics['recall']:.4f}, "
        f"f1={metrics['f1_score']:.4f}"
    )



=== Experiment: A vs C ===
  Balanced counts: Counter({1: 1102, 0: 1102})

--- Fold 1 ---


Epoch 001 | train_loss=0.7490 acc=0.5043 | val_loss=0.6942 acc=0.4921 | prec=0.4965 rec=0.9729 f1=0.6575 | time=126.3s
Epoch 002 | train_loss=0.7162 acc=0.5003 | val_loss=0.6938 acc=0.5011 | prec=0.5011 rec=1.0000 f1=0.6677 | time=11.3s
Epoch 003 | train_loss=0.7056 acc=0.5060 | val_loss=0.6941 acc=0.5011 | prec=0.5011 rec=1.0000 f1=0.6677 | time=11.5s
Epoch 004 | train_loss=0.6953 acc=0.5264 | val_loss=0.6878 acc=0.6372 | prec=0.8020 rec=0.3665 f1=0.5031 | time=11.5s
Epoch 005 | train_loss=0.6936 acc=0.5434 | val_loss=0.6920 acc=0.5011 | prec=0.5011 rec=1.0000 f1=0.6677 | time=11.2s
Epoch 006 | train_loss=0.6608 acc=0.5814 | val_loss=0.5358 acc=0.7256 | prec=0.6953 rec=0.8054 f1=0.7463 | time=11.4s
Epoch 007 | train_loss=0.5864 acc=0.7056 | val_loss=0.4862 acc=0.7823 | prec=0.8698 rec=0.6652 f1=0.7538 | time=11.5s
Epoch 008 | train_loss=0.5406 acc=0.7351 | val_loss=0.5576 acc=0.6803 | prec=0.6143 rec=0.9729 f1=0.7531 | time=11.5s
Epoch 009 | train_loss=0.5221 acc=0.7538 | val_loss=0.5

epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
f1_score,▄▄▄▁▄▆▆▆▆█▆██▇▇▇▇▇▆▇▇▆▆▆▅▇▇▆▆▆
precision,▁▁▁▆▁▄▇▃▃▆▃▇▆▇▅██▇██▆▄▅▅█▇▅▅▆▇
recall,███▁█▆▄██▇█▆▆▆▆▅▅▅▄▅▆▇▆▆▃▅▆▆▅▄
train_accuracy,▁▁▁▁▂▃▅▅▆▆▆▆▆▇▆▆▇▇▇▇▇▇████████
train_loss,██▇▇▇▇▆▅▅▄▄▄▃▃▃▃▃▂▃▂▂▂▁▁▁▁▁▁▁▁
validation_accuracy,▁▁▁▄▁▆▇▅▅█▄███▇▇█▇▇█▇▅▆▆▆▇▇▆▇▇
validation_loss,▇▇▇▇▇▄▃▅▅▂█▁▂▁▂▂▁▃▃▁▂▅▅▅▇▃▅▇▆▇
epoch,30
f1_score,0.73711
precision,0.85629



--- Fold 2 ---


Epoch 001 | train_loss=0.7349 acc=0.5054 | val_loss=0.6927 acc=0.4989 | prec=0.0000 rec=0.0000 f1=0.0000 | time=11.4s
Epoch 002 | train_loss=0.7110 acc=0.4935 | val_loss=0.6924 acc=0.4989 | prec=0.0000 rec=0.0000 f1=0.0000 | time=11.5s
Epoch 003 | train_loss=0.7167 acc=0.4816 | val_loss=0.6907 acc=0.4989 | prec=0.0000 rec=0.0000 f1=0.0000 | time=11.5s
Epoch 004 | train_loss=0.7017 acc=0.5009 | val_loss=0.6744 acc=0.7392 | prec=0.8313 rec=0.6018 f1=0.6982 | time=11.6s
Epoch 005 | train_loss=0.6734 acc=0.5842 | val_loss=0.5930 acc=0.7528 | prec=0.8333 rec=0.6335 f1=0.7198 | time=11.4s
Epoch 006 | train_loss=0.6084 acc=0.6653 | val_loss=0.4823 acc=0.8186 | prec=0.8579 rec=0.7647 f1=0.8086 | time=11.7s
Epoch 007 | train_loss=0.5546 acc=0.7096 | val_loss=0.4528 acc=0.8254 | prec=0.9286 rec=0.7059 f1=0.8021 | time=11.4s
Epoch 008 | train_loss=0.5505 acc=0.7311 | val_loss=0.4094 acc=0.8322 | prec=0.7928 rec=0.9005 f1=0.8432 | time=11.3s
Epoch 009 | train_loss=0.5213 acc=0.7345 | val_loss=0.42

...

epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
f1_score,▁▁▁▁▁▁▆▃▇▇▆▇▇▇▇▃▇█▆▇▆▁▆▆▆
precision,▁▁▁▁▁▁▅▂▇▆▄▇▄▆▇██▅█▅▇▆▆▅▄
recall,██████▅█▄▅▇▄▇▅▄▁▃▆▃▆▃▁▄▅▇
train_accuracy,▁▁▁▁▁▂▄▅▆▆▆▇▇▇▆▇▇█▇██████
train_loss,██▇▇▇▇▆▅▄▅▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁
validation_accuracy,▁▁▁▁▁▁▇▃██▆█▇██▇███▇▇▆▇▇▆
validation_loss,██████▅▆▂▂▃▁▂▁▁▄▂▁▂▂▄▆▃▄▄
epoch,25
f1_score,0.78613
precision,0.68227



--- Fold 5 ---


Epoch 001 | train_loss=0.7487 acc=0.5079 | val_loss=0.7177 acc=0.5000 | prec=0.0000 rec=0.0000 f1=0.0000 | time=11.4s
Epoch 002 | train_loss=0.7247 acc=0.4989 | val_loss=0.6912 acc=0.6523 | prec=0.6031 rec=0.8909 f1=0.7193 | time=11.5s
Epoch 003 | train_loss=0.7036 acc=0.5181 | val_loss=0.7034 acc=0.5000 | prec=0.5000 rec=1.0000 f1=0.6667 | time=11.4s
Epoch 004 | train_loss=0.7053 acc=0.5215 | val_loss=0.6239 acc=0.7591 | prec=0.8239 rec=0.6591 f1=0.7323 | time=11.4s
Epoch 005 | train_loss=0.6541 acc=0.6066 | val_loss=0.6127 acc=0.7295 | prec=0.8855 rec=0.5273 f1=0.6610 | time=11.5s
Epoch 006 | train_loss=0.6167 acc=0.6621 | val_loss=0.4855 acc=0.8455 | prec=0.8878 rec=0.7909 f1=0.8365 | time=11.4s
Epoch 007 | train_loss=0.5764 acc=0.7035 | val_loss=0.5041 acc=0.7591 | prec=0.6900 rec=0.9409 f1=0.7962 | time=11.6s
Epoch 008 | train_loss=0.5593 acc=0.7132 | val_loss=0.4215 acc=0.8523 | prec=0.8934 rec=0.8000 f1=0.8441 | time=11.3s
Epoch 009 | train_loss=0.5269 acc=0.7432 | val_loss=0.38

epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
f1_score,▁▇▆▇▆█▇██▇█▆█▇███▇██▇█▇██
precision,▁▅▅▇▇▇▆▇▇████▇▇█▇█▇▇█▆▇▇▇
recall,▁▇█▆▅▇█▇▇▅▆▅▆▆▇▆▇▆▆▇▅▇▆▇▆
train_accuracy,▁▁▁▁▃▄▅▅▆▆▆▆▇▇▇▇▇▇▇█▇█▇██
train_loss,██▇▇▆▆▅▅▄▅▄▄▃▃▃▃▂▂▂▂▃▂▂▂▁
validation_accuracy,▁▄▁▆▅█▆██▆█▆█▇█▇█▇▇▇▆▇▇█▇
validation_loss,███▆▆▄▄▃▂▄▃▅▂▄▁▂▁▂▂▃▄▄▃▂▃
epoch,25
f1_score,0.82212
precision,0.87245



>>> Average for A vs C: acc=0.7890, prec=0.7899, rec=0.8122, f1=0.7923

=== Final Results ===
A vs C: acc=0.7890, prec=0.7899, rec=0.8122, f1=0.7923


#### Try Block = 1, Head = 3
#### Use Optuna for hyperparameter tuning

In [ ]:
import os
import json
import random
import time
import numpy as np
import torch
import torch.nn as nn
import wandb
import optuna
from collections import Counter
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score
from torch.utils.data import DataLoader, Subset, Dataset

from eeg_dataset import EEGDataset
from model_optimized_5 import EEGformer

# ─── Configuration ───────────────────────────────────────────────
DATA_DIR    = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE  = 'labels.json'
DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MAX_EPOCHS  = 100
BATCH_SIZE  = 32
NUM_WORKERS = 4
ES_PATIENCE = 15

# ─── Define Binary Dataset Wrapper ───────────────────────────────
class BinaryEEGDataset(Dataset):
    def __init__(self, raw_ds, metas):
        self.raw_ds = raw_ds
        self.metas  = metas
    def __len__(self):
        return len(self.raw_ds)
    def __getitem__(self, idx):
        x, _ = self.raw_ds[idx]
        y     = self.metas[idx]['label']
        return x, y

# ─── Prepare balanced metadata for AD vs CN ───────────────────────
with open(os.path.join(DATA_DIR, LABEL_FILE), 'r') as f:
    all_meta = json.load(f)
train_meta = [d for d in all_meta if d['type']=='train']
class0, class1 = 'A','C'

# filter and undersample
data0 = [d for d in train_meta if d['label']==class0]
data1 = [d for d in train_meta if d['label']==class1]
min_count = min(len(data0), len(data1))
samp0 = random.sample(data0, k=min_count)
samp1 = random.sample(data1, k=min_count)
balanced_meta = samp0 + samp1
random.shuffle(balanced_meta)

# map to 0/1
label_map = {class0:0, class1:1}
for d in balanced_meta:
    d['label'] = label_map[d['label']]

# raw and wrapped dataset
raw_ds  = EEGDataset(DATA_DIR, balanced_meta)
dataset = BinaryEEGDataset(raw_ds, balanced_meta)
labels  = np.array([d['label'] for d in balanced_meta])

# stratified k-fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# ─── Optuna objective: minimize val_loss & maximize val_acc ───────────
def objective(trial):
    # Hyperparameter sampling
    lr        = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    wd        = trial.suggest_float('wd', 1e-6, 1e-1, log=True)
    pct_start = trial.suggest_float('pct_start', 0.1, 0.3)

    print(f"Trial {trial.number}: lr={lr:.2e}, wd={wd:.2e}, pct_start={pct_start:.2f}")

    fold_losses, fold_accs, fold_precs, fold_recs, fold_f1s = [], [], [], [], []

    for fold, (train_idx, val_idx) in enumerate(
            skf.split(np.zeros(len(labels)), labels), start=1):
        print(f"--- Fold {fold} ---")
        # initialize W&B run for each fold-trial
        wandb.init(
            project="eeg-optuna-AD-CN-optuna-search-1",
            name=f"trial{trial.number}_fold{fold}",
            config={"lr": lr, "wd": wd, "pct_start": pct_start},
            reinit=True
        )

        train_loader = DataLoader(Subset(dataset, train_idx), batch_size=BATCH_SIZE,
                                  shuffle=True, num_workers=NUM_WORKERS)
        val_loader   = DataLoader(Subset(dataset, val_idx), batch_size=BATCH_SIZE,
                                  shuffle=False, num_workers=NUM_WORKERS)

        # Model, optimizer, scheduler
        input_len   = dataset[0][0].shape[-1]
        model       = EEGformer(
            in_channels=19,
            input_length=input_len,
            kernel_size=10,
            num_filters=120,
            num_heads=3,
            num_blocks=1,
            num_segments=5,
            num_classes=2
        ).to(DEVICE)
        optimizer   = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
        total_steps = MAX_EPOCHS * len(train_loader)
        scheduler   = torch.optim.lr_scheduler.OneCycleLR(
            optimizer,
            max_lr=lr,
            total_steps=total_steps,
            pct_start=pct_start,
            anneal_strategy='cos',
            cycle_momentum=False
        )
        criterion = nn.CrossEntropyLoss()

        best_loss = float('inf')
        es_count  = 0

        for epoch in range(1, MAX_EPOCHS + 1):
            t0 = time.time()
            # Train
            model.train()
            tloss = tcorrect = ttotal = 0
            for X, y in train_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                optimizer.zero_grad()
                logits = model(X)
                loss   = criterion(logits, y)
                loss.backward()
                optimizer.step()
                scheduler.step()
                # Dynamic Weight Decay
                cur_lr = optimizer.param_groups[0]['lr']
                new_wd = wd * (cur_lr / lr)
                for g in optimizer.param_groups:
                    g['weight_decay'] = new_wd
                tloss    += loss.item()
                preds     = logits.argmax(1)
                tcorrect += (preds == y).sum().item()
                ttotal   += y.size(0)
            train_loss = tloss / len(train_loader)
            train_acc  = tcorrect / ttotal

            # Validate
            model.eval()
            vloss = vcorrect = vtotal = 0
            val_preds, val_labels = [], []
            with torch.no_grad():
                for X, y in val_loader:
                    X, y = X.to(DEVICE), y.to(DEVICE)
                    logits = model(X)
                    loss   = criterion(logits, y)
                    vloss  += loss.item()
                    preds   = logits.argmax(1)
                    vcorrect += (preds == y).sum().item()
                    vtotal   += y.size(0)
                    val_preds.append(preds.cpu().numpy())
                    val_labels.append(y.cpu().numpy())
            val_loss = vloss / len(val_loader)
            val_preds  = np.concatenate(val_preds)
            val_labels = np.concatenate(val_labels)
            val_acc    = (val_preds == val_labels).mean()
            precision  = precision_score(val_labels, val_preds, zero_division=0)
            recall     = recall_score(val_labels, val_preds, zero_division=0)
            f1         = f1_score(val_labels, val_preds, zero_division=0)
            elapsed    = time.time() - t0

            # Terminal output
            print(
                f"Epoch {epoch:03d} | "
                f"train_loss={train_loss:.4f} acc={train_acc:.4f} | "
                f"val_loss={val_loss:.4f} acc={val_acc:.4f} | "
                f"prec={precision:.4f} rec={recall:.4f} f1={f1:.4f} | "
                f"time={elapsed:.1f}s"
            )

            # W&B logging
            wandb.log({
                "train_loss": train_loss,
                "train_accuracy": train_acc,
                "validation_loss": val_loss,
                "validation_accuracy": val_acc,
                "precision": precision,
                "recall": recall,
                "f1_score": f1
            }, step=epoch)

            # Early Stopping by val_loss
            if val_loss < best_loss:
                best_loss = val_loss
                es_count  = 0
            else:
                es_count += 1
                if es_count >= ES_PATIENCE:
                    print(f"Early stopping at epoch {epoch}")
                    break

        # collect metrics
        fold_losses.append(best_loss)
        fold_accs.append(val_acc)
        fold_precs.append(precision)
        fold_recs.append(recall)
        fold_f1s.append(f1)

        wandb.finish()
        torch.cuda.empty_cache()

    # 10) compute and print average metrics across folds
    mean_acc  = np.mean(fold_accs)
    mean_prec = np.mean(fold_precs)
    mean_rec  = np.mean(fold_recs)
    mean_f1   = np.mean(fold_f1s)
    print(
        f"\n>>> Average metrics across folds: "
        f"acc={mean_acc:.4f}, "
        f"prec={mean_prec:.4f}, "
        f"rec={mean_rec:.4f}, "
        f"f1={mean_f1:.4f}"
    )

    return np.mean(fold_losses), np.mean(fold_accs)

# ─── Run Optuna multi-objective study ────────────────────────────
study = optuna.create_study(directions=['minimize','maximize'])
study.optimize(objective, n_trials=10)

print('Best trial values (val_loss, val_acc):', study.best_trial.values)
print('Best trial params:', study.best_trial.params)


[I 2025-05-04 07:24:19,147] A new study created in memory with name: no-name-a5c19af3-5512-4e1e-90ae-041bae786695
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Trial 0: lr=3.95e-05, wd=4.65e-06, pct_start=0.26
--- Fold 1 ---


Epoch 001 | train_loss=0.8009 acc=0.5128 | val_loss=0.7055 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=28.7s
Epoch 002 | train_loss=0.7794 acc=0.5309 | val_loss=0.6995 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=13.1s
Epoch 003 | train_loss=0.8031 acc=0.5088 | val_loss=0.7018 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=13.2s
Epoch 004 | train_loss=0.8076 acc=0.4957 | val_loss=0.7084 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=13.2s
Epoch 005 | train_loss=0.7874 acc=0.4974 | val_loss=0.7075 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=13.1s
Epoch 006 | train_loss=0.7727 acc=0.5247 | val_loss=0.7048 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=13.0s
Epoch 007 | train_loss=0.7619 acc=0.4946 | val_loss=0.6978 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=13.0s
Epoch 008 | train_loss=0.7443 acc=0.5133 | val_loss=0.6945 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=12.9s
Epoch 009 | train_loss=0.7358 acc=0.5150 | val_loss=0.69

f1_score,▁▁▁▁▁▁▁▁▁▁▁▁▃▃▆▆▆▆▆▆▆▇▇▇▇▇█▇██▇█▇██████▇
precision,▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▅▄▅▆▆▆▆▆▆▇▇▇█▇▇█▇▇▇▇█▇▇██
recall,██████████▇▁▅▂▆▆▆▃▄▃▅▅▃▂▄▄▂▁▃▄▄▃▃▃▄▄▄▄▃▂
train_accuracy,▁▂▁▁▂▁▁▁▁▁▂▁▁▂▁▄▅▅▆▆▇▇▇▇▇▇▇██▇███▇██████
train_loss,███▇▇▇▇▇▇▇▇▇▇▇▆▅▄▄▄▄▃▃▃▃▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁
validation_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▂▃▆▇▇▆▆▇▇▇▇▇▇▇▇█████████████
validation_loss,████████████▇▆▅▄▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▂▁▂▁▁▁▁
f1_score,0.81731
precision,0.86735
recall,0.77273
train_accuracy,0.80998


--- Fold 2 ---


...

[I 2025-05-04 08:54:06,623] Trial 0 finished with values: [0.40346492316041677, 0.8153401360544217] and parameters: {'lr': 3.948422459576768e-05, 'wd': 4.653871115001634e-06, 'pct_start': 0.25905355008255493}.



>>> Average metrics across folds: acc=0.8153, prec=0.8316, rec=0.8040, f1=0.8121
Trial 1: lr=3.31e-03, wd=6.27e-05, pct_start=0.28
--- Fold 1 ---


Epoch 001 | train_loss=0.7570 acc=0.4906 | val_loss=0.6932 acc=0.5011 | prec=0.0000 rec=0.0000 f1=0.0000 | time=12.9s
Epoch 002 | train_loss=0.7577 acc=0.4929 | val_loss=0.6942 acc=0.5011 | prec=0.0000 rec=0.0000 f1=0.0000 | time=12.9s
Epoch 003 | train_loss=0.7538 acc=0.4810 | val_loss=0.6935 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=12.9s


[W 2025-05-04 08:54:50,056] Trial 1 failed with parameters: {'lr': 0.0033052112326828124, 'wd': 6.27199998744308e-05, 'pct_start': 0.2849821528541324} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "<ipython-input-7-836a626fa4aa>", line 140, in objective
    tloss    += loss.item()
                ^^^^^^^^^^^
KeyboardInterrupt
[W 2025-05-04 08:54:50,058] Trial 1 failed with value None.


KeyboardInterrupt: 

#### 5 Fold Cross Validation (Results)
1] Trial 0: lr=3.95e-05, wd=4.65e-06, pct_start=0.26
* Epoch 084 | train_loss=0.4005 acc=0.8100 | val_loss=0.3896 acc=0.8277 | prec=0.8673 rec=0.7727 f1=0.8173 | time=12.8s
Early stopping at epoch 84

#### Try single fold instead of 5 fold to speed up the searching hyperparameters


In [ ]:
import os
import json
import random
import time
import numpy as np
import torch
import torch.nn as nn
import wandb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from torch.utils.data import DataLoader, Subset, Dataset

from eeg_dataset import EEGDataset
from model_optimized_5 import EEGformer

# ─── Configuration ───────────────────────────────────────────────
DATA_DIR    = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE  = 'labels.json'
DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MAX_EPOCHS  = 100
BATCH_SIZE  = 32
NUM_WORKERS = 4
ES_PATIENCE = 15

# ─── Dataset wrapper ─────────────────────────────────────────────
class BinaryEEGDataset(Dataset):
    def __init__(self, raw_ds, metas):
        self.raw_ds = raw_ds
        self.metas  = metas
    def __len__(self):
        return len(self.raw_ds)
    def __getitem__(self, idx):
        x, _ = self.raw_ds[idx]
        y     = self.metas[idx]['label']
        return x, y

# ─── Prepare data ────────────────────────────────────────────────
with open(os.path.join(DATA_DIR, LABEL_FILE), 'r') as f:
    all_meta = json.load(f)
train_meta = [d for d in all_meta if d['type']=='train']
class0, class1 = 'A','C'

# Undersample for balance
data0 = [d for d in train_meta if d['label']==class0]
data1 = [d for d in train_meta if d['label']==class1]
min_count = min(len(data0), len(data1))
samp0 = random.sample(data0, k=min_count)
samp1 = random.sample(data1, k=min_count)
balanced_meta = samp0 + samp1
random.shuffle(balanced_meta)

# Map labels to 0/1
label_map = {class0:0, class1:1}
for d in balanced_meta:
    d['label'] = label_map[d['label']]

# Build dataset
raw_ds  = EEGDataset(DATA_DIR, balanced_meta)
dataset = BinaryEEGDataset(raw_ds, balanced_meta)
labels  = np.array([d['label'] for d in balanced_meta])

# Single train/validation split
indices = np.arange(len(dataset))
train_idx, val_idx = train_test_split(
    indices, test_size=0.2, stratify=labels, random_state=0
)

# ─── Optuna objective ────────────────────────────────────────────
def objective(trial):
    # Sample hyperparameters
    lr        = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    wd        = trial.suggest_float('wd', 1e-6, 1e-1, log=True)
    pct_start = trial.suggest_float('pct_start', 0.1, 0.3)

    print(f"Trial {trial.number}: lr={lr:.2e}, wd={wd:.2e}, pct_start={pct_start:.2f}")

    # W&B init for this trial
    wandb.init(
        project='eeg-optuna-binary-AD-CN-single-fold',
        name=f'trial{trial.number}',
        config={'lr':lr, 'wd':wd, 'pct_start':pct_start},
        reinit=True
    )

    # DataLoaders
    train_loader = DataLoader(
        Subset(dataset, train_idx),
        batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS
    )
    val_loader   = DataLoader(
        Subset(dataset, val_idx),
        batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
    )

    # Model, optimizer, scheduler, criterion
    input_len   = dataset[0][0].shape[-1]
    model       = EEGformer(
        in_channels=19,
        input_length=input_len,
        kernel_size=10,
        num_filters=120,
        num_heads=3,
        num_blocks=1,
        num_segments=5,
        num_classes=2
    ).to(DEVICE)
    optimizer   = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    total_steps = MAX_EPOCHS * len(train_loader)
    scheduler   = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=lr,
        total_steps=total_steps,
        pct_start=pct_start,
        anneal_strategy='cos',
        cycle_momentum=False
    )
    criterion = nn.CrossEntropyLoss()

    best_loss = float('inf')
    es_count  = 0  # reset counter

    # Epoch loop
    for epoch in range(1, MAX_EPOCHS+1):
        t0 = time.time()
        # Train
        model.train()
        tloss = tcorrect = ttotal = 0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()
            scheduler.step()
            # Dynamic WD
            cur_lr = optimizer.param_groups[0]['lr']
            new_wd = wd * (cur_lr/lr)
            for g in optimizer.param_groups:
                g['weight_decay'] = new_wd

            tloss    += loss.item()
            preds     = logits.argmax(1)
            tcorrect += (preds==y).sum().item()
            ttotal   += y.size(0)
        train_loss = tloss/len(train_loader)
        train_acc  = tcorrect/ttotal

        # Validate
        model.eval()
        vloss = vcorrect = vtotal = 0
        val_preds, val_labels = [], []
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                loss   = criterion(logits, y)
                vloss += loss.item()
                preds = logits.argmax(1)
                vcorrect += (preds==y).sum().item()
                vtotal   += y.size(0)
                val_preds.append(preds.cpu().numpy())
                val_labels.append(y.cpu().numpy())
        val_loss = vloss/len(val_loader)
        val_preds  = np.concatenate(val_preds)
        val_labels = np.concatenate(val_labels)
        val_acc    = (val_preds==val_labels).mean()
        precision  = precision_score(val_labels, val_preds, zero_division=0)
        recall     = recall_score(val_labels, val_preds, zero_division=0)
        f1         = f1_score(val_labels, val_preds, zero_division=0)
        elapsed    = time.time()-t0

        # Terminal output & W&B logging
        print(f"Epoch {epoch:03d} | train_loss={train_loss:.4f} acc={train_acc:.4f} | "
              f"val_loss={val_loss:.4f} acc={val_acc:.4f} | "
              f"prec={precision:.4f} rec={recall:.4f} f1={f1:.4f} | time={elapsed:.1f}s")
        wandb.log({
            'train_loss': train_loss,
            'train_accuracy': train_acc,
            'validation_loss': val_loss,
            'validation_accuracy': val_acc,
            'precision': precision,
            'recall': recall,
            'f1_score': f1
        }, step=epoch)

        # Early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            es_count  = 0  # reset on improvement
        else:
            es_count += 1
            if es_count >= ES_PATIENCE:
                print(f"Early stopping at epoch {epoch}")
                break

    wandb.finish()
    return val_loss, val_acc

# ─── Run Optuna study ─────────────────────────────────────────────
study = optuna.create_study(directions=['minimize','maximize'])
study.optimize(objective, n_trials=10)

# ─── Print Pareto-optimal trials ──────────────────────────────────
print("Pareto-optimal trials (val_loss, val_acc) and their params:")
for t in study.best_trials:
    print(
        f" Trial #{t.number}: values={t.values}\n"
        f"              params={t.params}"
    )

Now using CUDA device 0
Enabling CUDA with 39.14 GiB available memory


[I 2025-05-04 09:07:13,940] A new study created in memory with name: no-name-f1673775-135c-4ddf-b68a-74cb6ae01bb6


Trial 0: lr=4.87e-03, wd=1.46e-06, pct_start=0.28


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Epoch 001 | train_loss=0.7341 acc=0.5071 | val_loss=0.6935 acc=0.5011 | prec=0.0000 rec=0.0000 f1=0.0000 | time=14.4s
Epoch 002 | train_loss=0.7315 acc=0.5116 | val_loss=0.6953 acc=0.5011 | prec=0.0000 rec=0.0000 f1=0.0000 | time=12.8s
Epoch 003 | train_loss=0.7191 acc=0.4742 | val_loss=0.6936 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=12.9s
Epoch 004 | train_loss=0.7146 acc=0.4901 | val_loss=0.6984 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=13.0s
Epoch 005 | train_loss=0.7151 acc=0.4963 | val_loss=0.6927 acc=0.5011 | prec=0.0000 rec=0.0000 f1=0.0000 | time=12.8s
Epoch 006 | train_loss=0.7249 acc=0.5082 | val_loss=0.6920 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=12.9s
Epoch 007 | train_loss=0.7085 acc=0.5082 | val_loss=0.6900 acc=0.6077 | prec=0.5662 rec=0.9136 f1=0.6991 | time=12.9s
Epoch 008 | train_loss=0.6904 acc=0.5349 | val_loss=0.6723 acc=0.7528 | prec=0.7846 rec=0.6955 f1=0.7373 | time=12.9s
Epoch 009 | train_loss=0.7036 acc=0.5139 | val_loss=0.69

...

Trial 9: lr=5.22e-04, wd=1.99e-04, pct_start=0.12


Epoch 001 | train_loss=0.7615 acc=0.5054 | val_loss=0.6940 acc=0.5011 | prec=0.0000 rec=0.0000 f1=0.0000 | time=13.0s
Epoch 002 | train_loss=0.7512 acc=0.4935 | val_loss=0.6955 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=12.9s
Epoch 003 | train_loss=0.7357 acc=0.4974 | val_loss=0.6934 acc=0.5011 | prec=0.0000 rec=0.0000 f1=0.0000 | time=13.0s
Epoch 004 | train_loss=0.7431 acc=0.4861 | val_loss=0.6943 acc=0.5011 | prec=0.0000 rec=0.0000 f1=0.0000 | time=12.9s
Epoch 005 | train_loss=0.7250 acc=0.5003 | val_loss=0.6928 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=13.0s
Epoch 006 | train_loss=0.7212 acc=0.4935 | val_loss=0.6932 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=12.8s
Epoch 007 | train_loss=0.7103 acc=0.5060 | val_loss=0.6939 acc=0.5011 | prec=0.0000 rec=0.0000 f1=0.0000 | time=12.8s
Epoch 008 | train_loss=0.7069 acc=0.5082 | val_loss=0.6930 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=13.0s
Epoch 009 | train_loss=0.7302 acc=0.5213 | val_loss=0.69

f1_score,▁▆▁▁▆▁▆▆▆▆▁▁▆▆▄▆▇██████▇▇██████▇▇▇██▇▇▇█
precision,▁▅▁▁▅▁▅▅▅▅▁▁▅▅██▆▇▇▇▇▇█▆▆▇▇▇█▇▇▆▆▆▇█▆▆▆▇
recall,▁█▁▁█▁████▁▁██▃▄█▇█▇█▇▇███▇▇▇▇▇███▇▇███▇
train_accuracy,▁▁▁▁▁▁▁▂▁▁▂▁▁▁▂▃▅▆▆▆▆▇▇▇▇▇██▇▇██████▇███
train_loss,████▇▇▇██▇▇▇▇▇▇▇▆▅▅▅▄▄▃▃▃▃▂▂▂▂▁▂▁▂▂▁▂▁▂▁
validation_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▅▅▇▇▇▇██▅▆▇████▇▆▅▆▇█▄▆▆▇
validation_loss,███████████████▆▆▄▅▃▃▂▁▄▃▂▁▁▁▁▂▆█▆▄▂▇▆▆▄
f1_score,0.83027
precision,0.75465
recall,0.92273
train_accuracy,0.87067


[I 2025-05-04 11:12:22,166] Trial 9 finished with values: [0.5010083009089742, 0.8117913832199547] and parameters: {'lr': 0.0005219080039578175, 'wd': 0.00019917379546773365, 'pct_start': 0.1229693696038243}.


RuntimeError: A single best trial cannot be retrieved from a multi-objective study. Consider using Study.best_trials to retrieve a list containing the best trials.

#### Block = 1, Head = 3 (Best Trial)
1] Trial 7: lr=7.13e-05, wd=1.30e-05, pct_start=0.23
- Epoch 088 | train_loss=0.2949 acc=0.8525 | val_loss=0.3569 acc=0.8458 | prec=0.8220 rec=0.8818 f1=0.8509 | time=12.9s
Early stopping at epoch 88

2] Trial 8: lr=3.44e-05, wd=3.04e-06, pct_start=0.14
- Epoch 089 | train_loss=0.4120 acc=0.8100 | val_loss=0.4381 acc=0.8186 | prec=0.7734 rec=0.9000 f1=0.8319 | time=13.0s
Early stopping at epoch 89

### Increase the model complexity
#### Try Block = 2, Head = 2 experiment
- Training Loss -> 0 should be confirmed
- Observe Validation Loss & Validation Accuracy & F-1 Score

In [ ]:
import os
import json
import random
import time
import numpy as np
import torch
import torch.nn as nn
import wandb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from torch.utils.data import DataLoader, Subset, Dataset

from eeg_dataset import EEGDataset
from model_optimized_5 import EEGformer

# ─── Configuration ───────────────────────────────────────────────
DATA_DIR    = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE  = 'labels.json'
DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MAX_EPOCHS  = 100
BATCH_SIZE  = 32
NUM_WORKERS = 4
ES_PATIENCE = 15

# ─── Dataset wrapper ─────────────────────────────────────────────
class BinaryEEGDataset(Dataset):
    def __init__(self, raw_ds, metas):
        self.raw_ds = raw_ds
        self.metas  = metas
    def __len__(self):
        return len(self.raw_ds)
    def __getitem__(self, idx):
        x, _ = self.raw_ds[idx]
        y     = self.metas[idx]['label']
        return x, y

# ─── Prepare data ────────────────────────────────────────────────
with open(os.path.join(DATA_DIR, LABEL_FILE), 'r') as f:
    all_meta = json.load(f)
train_meta = [d for d in all_meta if d['type']=='train']
class0, class1 = 'A','C'

# Undersample for balance
data0 = [d for d in train_meta if d['label']==class0]
data1 = [d for d in train_meta if d['label']==class1]
min_count = min(len(data0), len(data1))
samp0 = random.sample(data0, k=min_count)
samp1 = random.sample(data1, k=min_count)
balanced_meta = samp0 + samp1
random.shuffle(balanced_meta)

# Map labels to 0/1
label_map = {class0:0, class1:1}
for d in balanced_meta:
    d['label'] = label_map[d['label']]

# Build dataset
raw_ds  = EEGDataset(DATA_DIR, balanced_meta)
dataset = BinaryEEGDataset(raw_ds, balanced_meta)
labels  = np.array([d['label'] for d in balanced_meta])

# Single train/validation split
indices = np.arange(len(dataset))
train_idx, val_idx = train_test_split(
    indices, test_size=0.2, stratify=labels, random_state=0
)

# ─── Optuna objective ────────────────────────────────────────────
def objective(trial):
    # Sample hyperparameters
    lr        = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    wd        = trial.suggest_float('wd', 1e-6, 1e-1, log=True)
    pct_start = trial.suggest_float('pct_start', 0.1, 0.3)

    print(f"Trial {trial.number}: lr={lr:.2e}, wd={wd:.2e}, pct_start={pct_start:.2f}")

    # W&B init for this trial
    wandb.init(
        project='eeg-optuna-binary-AD-CN-single-fold-2',
        name=f'trial{trial.number}',
        config={'lr':lr, 'wd':wd, 'pct_start':pct_start},
        reinit=True
    )

    # DataLoaders
    train_loader = DataLoader(
        Subset(dataset, train_idx),
        batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS
    )
    val_loader   = DataLoader(
        Subset(dataset, val_idx),
        batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
    )

    # Model, optimizer, scheduler, criterion
    input_len   = dataset[0][0].shape[-1]
    model       = EEGformer(
        in_channels=19,
        input_length=input_len,
        kernel_size=10,
        num_filters=120,
        num_heads=2,
        num_blocks=2,
        num_segments=5,
        num_classes=2
    ).to(DEVICE)
    optimizer   = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    total_steps = MAX_EPOCHS * len(train_loader)
    scheduler   = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=lr,
        total_steps=total_steps,
        pct_start=pct_start,
        anneal_strategy='cos',
        cycle_momentum=False
    )
    criterion = nn.CrossEntropyLoss()

    best_loss = float('inf')
    es_count  = 0  # reset counter

    # Epoch loop
    for epoch in range(1, MAX_EPOCHS+1):
        t0 = time.time()
        # Train
        model.train()
        tloss = tcorrect = ttotal = 0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()
            scheduler.step()
            # Dynamic WD
            cur_lr = optimizer.param_groups[0]['lr']
            new_wd = wd * (cur_lr/lr)
            for g in optimizer.param_groups:
                g['weight_decay'] = new_wd

            tloss    += loss.item()
            preds     = logits.argmax(1)
            tcorrect += (preds==y).sum().item()
            ttotal   += y.size(0)
        train_loss = tloss/len(train_loader)
        train_acc  = tcorrect/ttotal

        # Validate
        model.eval()
        vloss = vcorrect = vtotal = 0
        val_preds, val_labels = [], []
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                loss   = criterion(logits, y)
                vloss += loss.item()
                preds = logits.argmax(1)
                vcorrect += (preds==y).sum().item()
                vtotal   += y.size(0)
                val_preds.append(preds.cpu().numpy())
                val_labels.append(y.cpu().numpy())
        val_loss = vloss/len(val_loader)
        val_preds  = np.concatenate(val_preds)
        val_labels = np.concatenate(val_labels)
        val_acc    = (val_preds==val_labels).mean()
        precision  = precision_score(val_labels, val_preds, zero_division=0)
        recall     = recall_score(val_labels, val_preds, zero_division=0)
        f1         = f1_score(val_labels, val_preds, zero_division=0)
        elapsed    = time.time()-t0

        # Terminal output & W&B logging
        print(f"Epoch {epoch:03d} | train_loss={train_loss:.4f} acc={train_acc:.4f} | "
              f"val_loss={val_loss:.4f} acc={val_acc:.4f} | "
              f"prec={precision:.4f} rec={recall:.4f} f1={f1:.4f} | time={elapsed:.1f}s")
        wandb.log({
            'train_loss': train_loss,
            'train_accuracy': train_acc,
            'validation_loss': val_loss,
            'validation_accuracy': val_acc,
            'precision': precision,
            'recall': recall,
            'f1_score': f1
        }, step=epoch)

        # Early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            es_count  = 0  # reset on improvement
        else:
            es_count += 1
            if es_count >= ES_PATIENCE:
                print(f"Early stopping at epoch {epoch}")
                break

    wandb.finish()
    return val_loss, val_acc

# ─── Run Optuna study ─────────────────────────────────────────────
study = optuna.create_study(directions=['minimize','maximize'])
study.optimize(objective, n_trials=10)

# ─── Print Pareto-optimal trials ──────────────────────────────────
print("Pareto-optimal trials (val_loss, val_acc) and their params:")
for t in study.best_trials:
    print(
        f" Trial #{t.number}: values={t.values}\n"
        f"              params={t.params}"
    )

Attempting to create new mne-python configuration file:
/root/.mne/mne-python.json
Now using CUDA device 0
Enabling CUDA with 39.14 GiB available memory


[I 2025-05-04 18:50:32,727] A new study created in memory with name: no-name-33a7aece-c5fa-4fab-847e-74d125e76683


Trial 0: lr=4.20e-03, wd=8.05e-06, pct_start=0.26


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Epoch 001 | train_loss=0.7283 acc=0.5179 | val_loss=0.6924 acc=0.5102 | prec=0.5046 rec=1.0000 f1=0.6707 | time=142.5s
Epoch 002 | train_loss=0.7269 acc=0.5065 | val_loss=0.7036 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=16.8s
Epoch 003 | train_loss=0.7325 acc=0.5054 | val_loss=0.6935 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=16.8s
Epoch 004 | train_loss=0.7144 acc=0.5167 | val_loss=0.7042 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=16.9s
Epoch 005 | train_loss=0.7172 acc=0.4957 | val_loss=0.6931 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=16.7s
Epoch 006 | train_loss=0.6995 acc=0.5286 | val_loss=0.6931 acc=0.5669 | prec=0.6706 rec=0.2591 f1=0.3738 | time=16.8s
Epoch 007 | train_loss=0.7087 acc=0.4889 | val_loss=0.6931 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=17.0s
Epoch 008 | train_loss=0.7069 acc=0.4986 | val_loss=0.6938 acc=0.5011 | prec=0.0000 rec=0.0000 f1=0.0000 | time=16.7s
Epoch 009 | train_loss=0.6932 acc=0.5009 | val_loss=0.6

...

Trial 9: lr=1.83e-04, wd=5.80e-03, pct_start=0.29


Epoch 001 | train_loss=0.7475 acc=0.4901 | val_loss=0.7048 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=16.7s
Epoch 002 | train_loss=0.7315 acc=0.5128 | val_loss=0.6970 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=16.7s
Epoch 003 | train_loss=0.7135 acc=0.5037 | val_loss=0.6932 acc=0.5170 | prec=0.5202 rec=0.4091 f1=0.4580 | time=16.7s
Epoch 004 | train_loss=0.7262 acc=0.4991 | val_loss=0.6947 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=16.7s
Epoch 005 | train_loss=0.7317 acc=0.4850 | val_loss=0.6942 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=16.8s
Epoch 006 | train_loss=0.7100 acc=0.5162 | val_loss=0.6931 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=16.9s
Epoch 007 | train_loss=0.7215 acc=0.5111 | val_loss=0.6935 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=16.8s
Epoch 008 | train_loss=0.7349 acc=0.4844 | val_loss=0.7001 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=16.7s
Epoch 009 | train_loss=0.7163 acc=0.5116 | val_loss=0.69

f1_score,▆▆▄▆▆▆▆▁▆▆▄▆▆▆▆▇▇▇▇▇█▇██▇███████▇▇██▇█▇█
precision,▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▃▇█▄▄▇▄▇▇█▅▇▆▇▇▇▆██▇▆▇▇█▅
recall,██▃████▁██▃████▆▅▇▇██▆▇▆▅▇▇▆▇▆▆▆▇▆▅▅▆▆▆▇
train_accuracy,▁▂▁▁▁▂▁▂▁▁▁▁▁▁▂▂▄▅▅▆▆▆▆▇▇▇▆▇▇▇██▇████▇██
train_loss,██▇████████▇██▇▇▆▆▅▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁
validation_accuracy,▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▅▇▇▆▆█▆███▆▇████▇▇▇█▇▇█▇▇
validation_loss,██████████████▇▄▄▄▄▄▃▂▁▁▂▂▃▁▁▁▁▂▂▄▂▃▃▂▄▂
f1_score,0.79583
precision,0.73462
recall,0.86818
train_accuracy,0.86614


[I 2025-05-04 20:40:32,569] Trial 9 finished with values: [0.4529983890908105, 0.7777777777777778] and parameters: {'lr': 0.00018265124173547746, 'wd': 0.0058040494218769565, 'pct_start': 0.28932639472707466}.


Pareto-optimal trials (val_loss, val_acc) and their params:
 Trial #0: values=[0.42488924733230043, 0.7936507936507936]
              params={'lr': 0.004200672431740192, 'wd': 8.054570809194863e-06, 'pct_start': 0.2590797714249703}
 Trial #4: values=[0.4844085787023817, 0.8049886621315193]
              params={'lr': 0.0036466844800439376, 'wd': 0.002362616064213731, 'pct_start': 0.25599762193132525}


## Mid-Results Report (Best Trial)

#### [Block = 1, Head = 3] 5 Fold Cross Validation (Results)
1] Trial 0: lr=3.95e-05, wd=4.65e-06, pct_start=0.26
* Epoch 084 | train_loss=0.4005 acc=0.8100 | val_loss=0.3896 acc=0.8277 | prec=0.8673 rec=0.7727 f1=0.8173 | time=12.8s
Early stopping at epoch 84

#### Block = 1, Head = 3 (Best Trial)
1] Trial 7: lr=7.13e-05, wd=1.30e-05, pct_start=0.23
- Epoch 088 | train_loss=0.2949 acc=0.8525 | val_loss=0.3569 acc=0.8458 | prec=0.8220 rec=0.8818 f1=0.8509 | time=12.9s
Early stopping at epoch 88

2] Trial 8: lr=3.44e-05, wd=3.04e-06, pct_start=0.14
- Epoch 089 | train_loss=0.4120 acc=0.8100 | val_loss=0.4381 acc=0.8186 | prec=0.7734 rec=0.9000 f1=0.8319 | time=13.0s
Early stopping at epoch 89

#### [Block = 2, Head = 2] (Best Trial)

1) Trial 0: lr=4.20e-03, wd=8.05e-06, pct_start=0.26

* Epoch 071 | train_loss=0.3092 acc=0.8383 | val_loss=0.4249 acc=0.7937 | prec=0.7345 rec=0.9182 f1=0.8162 | time=16.9s
* Early stopping at epoch 71

2) Trial 9: lr=1.83e-04, wd=5.80e-03, pct_start=0.29

* Epoch 050 | train_loss=0.2896 acc=0.8661 | val_loss=0.4530 acc=0.7778 | prec=0.7346 rec=0.8682 f1=0.7958 | time=16.8s
* Early stopping at epoch 50

#### Return to Block = 1, Head = 3 and increase the filter size to increase the model complexity
- Since we found out that the increasing the block size does not increase the validation accuracy and decrease the validation loss, we decided to decrease the Block size as original and maintain the head size as 3. This means that increasing the block size does not generalize well.
- Block = 2, Head = 2 performance shows less performance compare to Block = 1, Head = 3
- Increase the num_filters = 120 -> 180
- Filter Ablation Studies = [120, 150, 180]


In [ ]:
import os
import json
import random
import time
import numpy as np
import torch
import torch.nn as nn
import wandb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from torch.utils.data import DataLoader, Subset, Dataset

from eeg_dataset import EEGDataset
from model_optimized_5 import EEGformer

# ─── Configuration ───────────────────────────────────────────────
DATA_DIR    = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE  = 'labels.json'
DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MAX_EPOCHS  = 100
BATCH_SIZE  = 32
NUM_WORKERS = 4
ES_PATIENCE = 15

# ─── Dataset wrapper ─────────────────────────────────────────────
class BinaryEEGDataset(Dataset):
    def __init__(self, raw_ds, metas):
        self.raw_ds = raw_ds
        self.metas  = metas
    def __len__(self):
        return len(self.raw_ds)
    def __getitem__(self, idx):
        x, _ = self.raw_ds[idx]
        y     = self.metas[idx]['label']
        return x, y

# ─── Prepare data ────────────────────────────────────────────────
with open(os.path.join(DATA_DIR, LABEL_FILE), 'r') as f:
    all_meta = json.load(f)
train_meta = [d for d in all_meta if d['type']=='train']
class0, class1 = 'A','C'

# Undersample for balance
data0 = [d for d in train_meta if d['label']==class0]
data1 = [d for d in train_meta if d['label']==class1]
min_count = min(len(data0), len(data1))
samp0 = random.sample(data0, k=min_count)
samp1 = random.sample(data1, k=min_count)
balanced_meta = samp0 + samp1
random.shuffle(balanced_meta)

# Map labels to 0/1
label_map = {class0:0, class1:1}
for d in balanced_meta:
    d['label'] = label_map[d['label']]

# Build dataset
raw_ds  = EEGDataset(DATA_DIR, balanced_meta)
dataset = BinaryEEGDataset(raw_ds, balanced_meta)
labels  = np.array([d['label'] for d in balanced_meta])

# Single train/validation split
indices = np.arange(len(dataset))
train_idx, val_idx = train_test_split(
    indices, test_size=0.2, stratify=labels, random_state=0
)

# ─── Optuna objective ────────────────────────────────────────────
def objective(trial):
    # Sample hyperparameters
    lr        = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    wd        = trial.suggest_float('wd', 1e-6, 1e-1, log=True)
    pct_start = 0.23
    NUM_FILTERS = trial.suggest_categorical('num_filters', [120,150,180])

    print(f"Trial {trial.number}: lr={lr:.2e}, wd={wd:.2e}, filters = {NUM_FILTERS}")

    # W&B init for this trial
    wandb.init(
        project='eeg-optuna-binary-AD-CN-single-fold-4',
        name=f'trial{trial.number}',
        config={'lr':lr, 'wd':wd, 'filters':NUM_FILTERS},
        reinit=True
    )

    # DataLoaders
    train_loader = DataLoader(
        Subset(dataset, train_idx),
        batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS
    )
    val_loader   = DataLoader(
        Subset(dataset, val_idx),
        batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
    )

    # Model, optimizer, scheduler, criterion
    input_len   = dataset[0][0].shape[-1]
    model       = EEGformer(
        in_channels=19,
        input_length=input_len,
        kernel_size=10,
        num_filters=NUM_FILTERS,
        num_heads=3,
        num_blocks=1,
        num_segments=5,
        num_classes=2
    ).to(DEVICE)
    optimizer   = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    total_steps = MAX_EPOCHS * len(train_loader)
    scheduler   = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=lr,
        total_steps=total_steps,
        pct_start=pct_start,
        anneal_strategy='cos',
        cycle_momentum=False
    )
    criterion = nn.CrossEntropyLoss()

    best_loss = float('inf')
    es_count  = 0  # reset counter

    # Epoch loop
    for epoch in range(1, MAX_EPOCHS+1):
        t0 = time.time()
        # Train
        model.train()
        tloss = tcorrect = ttotal = 0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()
            scheduler.step()
            # Dynamic WD
            cur_lr = optimizer.param_groups[0]['lr']
            new_wd = wd * (cur_lr/lr)
            for g in optimizer.param_groups:
                g['weight_decay'] = new_wd

            tloss    += loss.item()
            preds     = logits.argmax(1)
            tcorrect += (preds==y).sum().item()
            ttotal   += y.size(0)
        train_loss = tloss/len(train_loader)
        train_acc  = tcorrect/ttotal

        # Validate
        model.eval()
        vloss = vcorrect = vtotal = 0
        val_preds, val_labels = [], []
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                loss   = criterion(logits, y)
                vloss += loss.item()
                preds = logits.argmax(1)
                vcorrect += (preds==y).sum().item()
                vtotal   += y.size(0)
                val_preds.append(preds.cpu().numpy())
                val_labels.append(y.cpu().numpy())
        val_loss = vloss/len(val_loader)
        val_preds  = np.concatenate(val_preds)
        val_labels = np.concatenate(val_labels)
        val_acc    = (val_preds==val_labels).mean()
        precision  = precision_score(val_labels, val_preds, zero_division=0)
        recall     = recall_score(val_labels, val_preds, zero_division=0)
        f1         = f1_score(val_labels, val_preds, zero_division=0)
        elapsed    = time.time()-t0

        # Terminal output & W&B logging
        print(f"Epoch {epoch:03d} | train_loss={train_loss:.4f} acc={train_acc:.4f} | "
              f"val_loss={val_loss:.4f} acc={val_acc:.4f} | "
              f"prec={precision:.4f} rec={recall:.4f} f1={f1:.4f} | time={elapsed:.1f}s")
        wandb.log({
            'train_loss': train_loss,
            'train_accuracy': train_acc,
            'validation_loss': val_loss,
            'validation_accuracy': val_acc,
            'precision': precision,
            'recall': recall,
            'f1_score': f1
        }, step=epoch)

        # Early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            es_count  = 0  # reset on improvement
        else:
            es_count += 1
            if es_count >= ES_PATIENCE:
                print(f"Early stopping at epoch {epoch}")
                break

    wandb.finish()
    return val_loss, val_acc

# ─── Run Optuna study ─────────────────────────────────────────────
study = optuna.create_study(directions=['minimize','maximize'])
study.optimize(objective, n_trials=15)

# ─── Print Pareto-optimal trials ──────────────────────────────────
print("Pareto-optimal trials (val_loss, val_acc) and their params:")
for t in study.best_trials:
    print(
        f" Trial #{t.number}: values={t.values}\n"
        f"              params={t.params}"
    )

Now using CUDA device 0
Enabling CUDA with 39.14 GiB available memory


[I 2025-05-04 22:56:14,213] A new study created in memory with name: no-name-9393b890-7744-4790-a374-2ad58b1bbbf1


Trial 0: lr=1.99e-04, wd=1.25e-03, filters = 180


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Epoch 001 | train_loss=0.7693 acc=0.5116 | val_loss=0.6975 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=24.6s
Epoch 002 | train_loss=0.7537 acc=0.5026 | val_loss=0.6932 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=23.6s
Epoch 003 | train_loss=0.7501 acc=0.4776 | val_loss=0.6936 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=23.6s
Epoch 004 | train_loss=0.7319 acc=0.5105 | val_loss=0.6935 acc=0.5011 | prec=0.0000 rec=0.0000 f1=0.0000 | time=23.5s
Epoch 005 | train_loss=0.7425 acc=0.4980 | val_loss=0.6942 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=23.5s
Epoch 006 | train_loss=0.7423 acc=0.5150 | val_loss=0.6963 acc=0.5011 | prec=0.0000 rec=0.0000 f1=0.0000 | time=23.3s
Epoch 007 | train_loss=0.7429 acc=0.5077 | val_loss=0.7010 acc=0.5011 | prec=0.0000 rec=0.0000 f1=0.0000 | time=23.6s
Epoch 008 | train_loss=0.7369 acc=0.5031 | val_loss=0.6933 acc=0.5011 | prec=0.0000 rec=0.0000 f1=0.0000 | time=23.6s
Epoch 009 | train_loss=0.7326 acc=0.4923 | val_loss=0.69

...

Trial 14: lr=1.79e-03, wd=7.78e-06, filters = 120


Epoch 001 | train_loss=0.7676 acc=0.4923 | val_loss=0.6946 acc=0.5011 | prec=0.0000 rec=0.0000 f1=0.0000 | time=12.9s
Epoch 002 | train_loss=0.7506 acc=0.4906 | val_loss=0.6937 acc=0.5011 | prec=0.0000 rec=0.0000 f1=0.0000 | time=12.8s
Epoch 003 | train_loss=0.7540 acc=0.4889 | val_loss=0.6940 acc=0.5011 | prec=0.0000 rec=0.0000 f1=0.0000 | time=12.9s
Epoch 004 | train_loss=0.7529 acc=0.4946 | val_loss=0.6928 acc=0.4989 | prec=0.4989 rec=1.0000 f1=0.6657 | time=12.9s
Epoch 005 | train_loss=0.7271 acc=0.5077 | val_loss=0.6924 acc=0.5488 | prec=0.5402 rec=0.6409 f1=0.5863 | time=12.8s
Epoch 006 | train_loss=0.7308 acc=0.5026 | val_loss=0.6933 acc=0.5011 | prec=0.5000 rec=0.0091 f1=0.0179 | time=12.8s
Epoch 007 | train_loss=0.7387 acc=0.4821 | val_loss=0.6912 acc=0.5011 | prec=0.0000 rec=0.0000 f1=0.0000 | time=12.9s
Epoch 008 | train_loss=0.7241 acc=0.4878 | val_loss=0.6929 acc=0.5011 | prec=0.0000 rec=0.0000 f1=0.0000 | time=12.6s
Epoch 009 | train_loss=0.7074 acc=0.5026 | val_loss=0.69

f1_score,▁▁▁▇▆▁▁▁▁▆▇▆▄▇█▇███▃▇▇▁▇█▇▇███▇▇██
precision,▁▁▁▅▅▅▁▁▁█▆███▆▆█▆▇█▅▅▁▇▇█▆▇▇▆▅▆▆▇
recall,▁▁▁█▅▁▁▁▁▄▇▄▂▅█▇▆█▇▂██▁▆▇▅█▆▆▇██▇▇
train_accuracy,▁▁▁▁▁▁▁▁▁▁▂▅▅▆▆▇▇▆▇▇▂▂▃▃▅▆▇▆▇▇▇█▇█
train_loss,████▇▇█▇▇▇▇▅▄▄▄▃▃▃▂▃█▇▇▆▄▃▂▃▃▂▁▁▁▁
validation_accuracy,▁▁▁▁▂▁▁▁▁▅▆▅▃▇▆▆█▆█▂▁▁▁▇█▆▅▇█▇▄▆▆▇
validation_loss,▄▄▄▄▄▄▄▄▄▄▃▃▅▂▂▂▁▂▁█▅▅▄▃▁▄▄▂▂▄▆▅▃▅
f1_score,0.81448
precision,0.81081
recall,0.81818
train_accuracy,0.84118


[I 2025-05-05 01:59:47,121] Trial 14 finished with values: [0.7342137077025005, 0.8140589569160998] and parameters: {'lr': 0.0017901670261317257, 'wd': 7.775103911911879e-06, 'num_filters': 120}.


Pareto-optimal trials (val_loss, val_acc) and their params:
 Trial #8: values=[0.36828610407454626, 0.8503401360544217]
              params={'lr': 0.0001655741045417959, 'wd': 0.004116746760159083, 'num_filters': 180}


### Previous Results

#### [Block = 1, Head = 3] 5 Fold Cross Validation (Results)
1] Trial 0: lr=3.95e-05, wd=4.65e-06, pct_start=0.26
* Epoch 084 | train_loss=0.4005 acc=0.8100 | val_loss=0.3896 acc=0.8277 | prec=0.8673 rec=0.7727 f1=0.8173 | time=12.8s
Early stopping at epoch 84

#### Block = 1, Head = 3 (Best Trial)
1] Trial 7: lr=7.13e-05, wd=1.30e-05, pct_start=0.23
- Epoch 088 | train_loss=0.2949 acc=0.8525 | val_loss=0.3569 acc=0.8458 | prec=0.8220 rec=0.8818 f1=0.8509 | time=12.9s
Early stopping at epoch 88

2] Trial 8: lr=3.44e-05, wd=3.04e-06, pct_start=0.14
- Epoch 089 | train_loss=0.4120 acc=0.8100 | val_loss=0.4381 acc=0.8186 | prec=0.7734 rec=0.9000 f1=0.8319 | time=13.0s
Early stopping at epoch 89


#### Filter Change Result
1] Trial 0: lr=1.99e-04, wd=1.25e-03, filters = 180
* Epoch 060 | train_loss=0.2256 acc=0.8866 | val_loss=0.4457 acc=0.8299 | prec=0.8139 rec=0.8545 f1=0.8337 | time=23.7s
Early stopping at epoch 60

2] Trial 3: lr=7.29e-04, wd=1.88e-06, filters = 120
* Epoch 036 | train_loss=0.2683 acc=0.8633 | val_loss=0.5991 acc=0.7506 | prec=0.6884 rec=0.9136 f1=0.7852 | time=12.8s
Early stopping at epoch 36

3] Trial 4: lr=3.90e-05, wd=1.17e-04, filters = 150
* Epoch 079 | train_loss=0.3613 acc=0.8361 | val_loss=0.3856 acc=0.8390 | prec=0.9162 rec=0.7455 f1=0.8221 | time=18.1s
Early stopping at epoch 79

4] Trial 13: lr=1.33e-03, wd=3.66e-06, filters = 180
* Epoch 061 | train_loss=0.2531 acc=0.8656 | val_loss=0.4849 acc=0.7800 | prec=0.7253 rec=0.9000 f1=0.8032 | time=24.2s
Early stopping at epoch 61

#### Final Optuna to decide how many filters to use
- Filter Size = 120 or 180
- Narrow down the search space
- learning rate = [1e-5, 5e-4]
- weight decay = [1e-6, 5e-4]


In [ ]:
import os
import json
import random
import time
import numpy as np
import torch
import torch.nn as nn
import wandb
import optuna

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix
)
from torch.utils.data import DataLoader, Subset, Dataset

from eeg_dataset import EEGDataset
from model_optimized_5 import EEGformer

# ─── Configuration ───────────────────────────────────────────────
DATA_DIR    = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE  = 'labels.json'
DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MAX_EPOCHS  = 100
BATCH_SIZE  = 32
NUM_WORKERS = 4
ES_PATIENCE = 15

# ─── Dataset wrapper ─────────────────────────────────────────────
class BinaryEEGDataset(Dataset):
    def __init__(self, raw_ds, metas):
        self.raw_ds = raw_ds
        self.metas  = metas
    def __len__(self):
        return len(self.raw_ds)
    def __getitem__(self, idx):
        x, _ = self.raw_ds[idx]
        y     = self.metas[idx]['label']
        return x, y

# ─── Prepare data ────────────────────────────────────────────────
with open(os.path.join(DATA_DIR, LABEL_FILE), 'r') as f:
    all_meta = json.load(f)
train_meta = [d for d in all_meta if d['type']=='train']
class0, class1 = 'A','C'

# Undersample for balance
data0 = [d for d in train_meta if d['label']==class0]
data1 = [d for d in train_meta if d['label']==class1]
min_count = min(len(data0), len(data1))
samp0 = random.sample(data0, k=min_count)
samp1 = random.sample(data1, k=min_count)
balanced_meta = samp0 + samp1
random.shuffle(balanced_meta)

# Map labels to 0/1
label_map = {class0:0, class1:1}
for d in balanced_meta:
    d['label'] = label_map[d['label']]

# Build dataset
raw_ds  = EEGDataset(DATA_DIR, balanced_meta)
dataset = BinaryEEGDataset(raw_ds, balanced_meta)
labels  = np.array([d['label'] for d in balanced_meta])

# Single train/validation split
indices = np.arange(len(dataset))
train_idx, val_idx = train_test_split(
    indices, test_size=0.2, stratify=labels, random_state=0
)

# ─── Optuna objective ────────────────────────────────────────────
def objective(trial):
    # Sample hyperparameters
    lr          = trial.suggest_float('lr', 1e-5, 5e-4, log=True)
    wd          = trial.suggest_float('wd', 1e-6, 5e-4, log=True)
    pct_start   = 0.23  # fixed
    num_filters = trial.suggest_categorical('num_filters', [120, 180])

    print(f"Trial {trial.number}: filters={num_filters}, lr={lr:.2e}, wd={wd:.2e}, pct_start={pct_start:.2f}")

    # W&B init for this trial
    wandb.init(
        project='eeg-optuna-binary-AD-CN-single-fold-5',
        name=f'trial{trial.number}',
        config={'lr': lr, 'wd': wd, 'pct_start': pct_start, 'filters': num_filters},
        reinit=True
    )

    # DataLoaders
    train_loader = DataLoader(
        Subset(dataset, train_idx),
        batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS
    )
    val_loader   = DataLoader(
        Subset(dataset, val_idx),
        batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
    )

    # Model, optimizer, scheduler, criterion
    input_len = dataset[0][0].shape[-1]
    model = EEGformer(
        in_channels=19,
        input_length=input_len,
        kernel_size=10,
        num_filters=num_filters,
        num_heads=3,
        num_blocks=1,
        num_segments=5,
        num_classes=2
    ).to(DEVICE)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    total_steps = MAX_EPOCHS * len(train_loader)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=lr,
        total_steps=total_steps,
        pct_start=pct_start,
        anneal_strategy='cos',
        cycle_momentum=False
    )
    criterion = nn.CrossEntropyLoss()

    best_loss = float('inf')
    es_count  = 0

    # Epoch loop
    for epoch in range(1, MAX_EPOCHS + 1):
        t0 = time.time()

        # — train —
        model.train()
        tloss = tcorrect = ttotal = 0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()
            scheduler.step()

            # Dynamic Weight Decay
            cur_lr = optimizer.param_groups[0]['lr']
            new_wd = wd * (cur_lr / lr)
            for g in optimizer.param_groups:
                g['weight_decay'] = new_wd

            tloss    += loss.item()
            preds     = logits.argmax(1)
            tcorrect += (preds == y).sum().item()
            ttotal   += y.size(0)

        train_loss = tloss / len(train_loader)
        train_acc  = tcorrect / ttotal

        # — validate —
        model.eval()
        vloss = vcorrect = vtotal = 0
        val_preds, val_labels = [], []
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                loss   = criterion(logits, y)
                vloss  += loss.item()

                preds   = logits.argmax(1)
                vcorrect += (preds == y).sum().item()
                vtotal   += y.size(0)

                val_preds.append(preds.cpu().numpy())
                val_labels.append(y.cpu().numpy())

        val_loss   = vloss / len(val_loader)
        val_preds  = np.concatenate(val_preds)
        val_labels = np.concatenate(val_labels)
        val_acc    = (val_preds == val_labels).mean()
        precision  = precision_score(val_labels, val_preds, zero_division=0)
        recall     = recall_score(val_labels, val_preds, zero_division=0)
        f1         = f1_score(val_labels, val_preds, zero_division=0)

        # Compute specificity = TN / (TN + FP)
        tn, fp, fn, tp = confusion_matrix(val_labels, val_preds).ravel()
        specificity     = tn / (tn + fp)

        elapsed = time.time() - t0

        # Terminal output & W&B logging
        print(
            f"Epoch {epoch:03d} | "
            f"train_loss={train_loss:.4f} acc={train_acc:.4f} | "
            f"val_loss={val_loss:.4f} acc={val_acc:.4f} | "
            f"prec={precision:.4f} rec={recall:.4f} spec={specificity:.4f} f1={f1:.4f} | "
            f"time={elapsed:.1f}s"
        )
        wandb.log({
            'train_loss': train_loss,
            'train_accuracy': train_acc,
            'validation_loss': val_loss,
            'validation_accuracy': val_acc,
            'precision': precision,
            'recall': recall,
            'specificity': specificity,
            'f1_score': f1
        }, step=epoch)

        # Early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            es_count  = 0
        else:
            es_count += 1
            if es_count >= ES_PATIENCE:
                print(f"Early stopping at epoch {epoch}")
                break

    wandb.finish()
    return val_loss, val_acc

# ─── Run Optuna study ─────────────────────────────────────────────
study = optuna.create_study(directions=['minimize', 'maximize'])
study.optimize(objective, n_trials=10)

# ─── Print Pare토-optimal trials ──────────────────────────────────
print("Pareto-optimal trials (val_loss, val_acc) and their params:")
for t in study.best_trials:
    print(
        f" Trial #{t.number}: values={t.values}\n"
        f"              params={t.params}"
    )


Now using CUDA device 0
Enabling CUDA with 39.14 GiB available memory


[I 2025-05-05 04:06:00,799] A new study created in memory with name: no-name-8a8e5e07-4a09-409c-93bb-69da1073f0c0


Trial 0: filters=180, lr=8.51e-05, wd=1.86e-06, pct_start=0.23


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Epoch 001 | train_loss=0.7415 acc=0.5258 | val_loss=0.6934 acc=0.4989 | prec=0.4989 rec=1.0000 spec=0.0000 f1=0.6657 | time=24.5s
Epoch 002 | train_loss=0.7438 acc=0.5065 | val_loss=0.6934 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=23.5s
Epoch 003 | train_loss=0.7333 acc=0.5054 | val_loss=0.6931 acc=0.5034 | prec=0.5263 rec=0.0455 spec=0.9593 f1=0.0837 | time=23.7s
Epoch 004 | train_loss=0.7365 acc=0.4827 | val_loss=0.6933 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=23.7s
Epoch 005 | train_loss=0.7298 acc=0.4844 | val_loss=0.6939 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=23.6s
Epoch 006 | train_loss=0.7119 acc=0.5173 | val_loss=0.6931 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=23.6s
Epoch 007 | train_loss=0.7202 acc=0.4929 | val_loss=0.6931 acc=0.4989 | prec=0.4982 rec=0.6227 spec=0.3756 f1=0.5535 | time=23.6s
Epoch 008 | train_loss=0.7342 acc=0.4770 | val_loss=0.6947 acc=0.5011 | prec=0.0000 rec=0.

...

Trial 9: filters=120, lr=9.18e-05, wd=4.24e-06, pct_start=0.23


Epoch 001 | train_loss=0.8049 acc=0.4946 | val_loss=0.7319 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.8s
Epoch 002 | train_loss=0.7874 acc=0.4861 | val_loss=0.7255 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.9s
Epoch 003 | train_loss=0.8022 acc=0.4906 | val_loss=0.7058 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.9s
Epoch 004 | train_loss=0.7706 acc=0.5037 | val_loss=0.7023 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.9s
Epoch 005 | train_loss=0.7472 acc=0.5048 | val_loss=0.6967 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.9s
Epoch 006 | train_loss=0.7471 acc=0.5105 | val_loss=0.7035 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.9s
Epoch 007 | train_loss=0.7463 acc=0.4889 | val_loss=0.7037 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.9s
Epoch 008 | train_loss=0.7386 acc=0.5094 | val_loss=0.7059 acc=0.5011 | prec=0.0000 rec=0.

f1_score,▁▁▁▁▁▁▁▁▁▁▁▁▁▃▇██████████████████▇██████
precision,▁▁▁▁▁▁▁▁▁▁▁▁▁█▇▇▇▇▇▇██▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
recall,▁▁▁▁▁▁▁▁▁▁▁▁▁▂▆▇▇▇█▇██▇█▆▇▇▇▇█▇▇▇▇▇▇▇▇▇▇
specificity,█████████████▆▅▆▅▅▅▆▄▆▁▄▄▃▇▅▅▆▆▅▆▆▅▅▅▅▅▅
train_accuracy,▁▁▁▁▁▁▁▂▁▂▁▂▂▃▄▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇█▇▇█▇███
train_loss,███▇▇▇▇▇▇▇▇▇▇▇▆▅▅▅▅▄▄▃▃▃▂▂▂▃▂▂▂▂▁▁▁▂▁▁▁▁
validation_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▂▆▇▇██▇██▇▇██▇█▇▇██▇▇▇▇▇▇▇▇
validation_loss,███▇▇▇█▇▇▇▇▇▇▇▆▄▄▃▃▃▃▂▂▂▂▁▁▂▁▂▁▂▁▃▃▂▂▂▂▂
f1_score,0.81517
precision,0.85149
recall,0.78182


[I 2025-05-05 07:00:35,693] Trial 9 finished with values: [0.41894412466457914, 0.8231292517006803] and parameters: {'lr': 9.178813016933477e-05, 'wd': 4.241884759163697e-06, 'num_filters': 120}.


Pareto-optimal trials (val_loss, val_acc) and their params:
 Trial #0: values=[0.3548754038555281, 0.8412698412698413]
              params={'lr': 8.506993179590607e-05, 'wd': 1.8649812851561947e-06, 'num_filters': 180}
 Trial #1: values=[0.3679687838469233, 0.8707482993197279]
              params={'lr': 0.0002851525315362799, 'wd': 3.3969696548624846e-06, 'num_filters': 180}


### Overall Results Wrap Up

#### [Block = 1, Head = 3, Filter = 120] 5-Fold Cross Validation (Best Trial Result)
1] Trial 0: lr=3.95e-05, wd=4.65e-06, pct_start=0.26
* Epoch 084 | train_loss=0.4005 acc=0.8100 | val_loss=0.3896 acc=0.8277 | prec=0.8673 rec=0.7727 f1=0.8173 | time=12.8s
Early stopping at epoch 84

#### [Block = 1, Head = 3, Filter = 120] (Best Trial)
1] Trial 7: lr=7.13e-05, wd=1.30e-05, pct_start=0.23
- Epoch 088 | train_loss=0.2949 acc=0.8525 | val_loss=0.3569 acc=0.8458 | prec=0.8220 rec=0.8818 f1=0.8509 | time=12.9s
Early stopping at epoch 88

2] Trial 8: lr=3.44e-05, wd=3.04e-06, pct_start=0.14
- Epoch 089 | train_loss=0.4120 acc=0.8100 | val_loss=0.4381 acc=0.8186 | prec=0.7734 rec=0.9000 f1=0.8319 | time=13.0s
Early stopping at epoch 89


#### Filter Change Result 120 vs 150 vs 180 (Best Trial)
1] Trial 0: lr=1.99e-04, wd=1.25e-03, filters = 180
* Epoch 060 | train_loss=0.2256 acc=0.8866 | val_loss=0.4457 acc=0.8299 | prec=0.8139 rec=0.8545 f1=0.8337 | time=23.7s
Early stopping at epoch 60

2] Trial 3: lr=7.29e-04, wd=1.88e-06, filters = 120
* Epoch 036 | train_loss=0.2683 acc=0.8633 | val_loss=0.5991 acc=0.7506 | prec=0.6884 rec=0.9136 f1=0.7852 | time=12.8s
Early stopping at epoch 36

3] Trial 4: lr=3.90e-05, wd=1.17e-04, filters = 150
* Epoch 079 | train_loss=0.3613 acc=0.8361 | val_loss=0.3856 acc=0.8390 | prec=0.9162 rec=0.7455 f1=0.8221 | time=18.1s
Early stopping at epoch 79

4] Trial 13: lr=1.33e-03, wd=3.66e-06, filters = 180
* Epoch 061 | train_loss=0.2531 acc=0.8656 | val_loss=0.4849 acc=0.7800 | prec=0.7253 rec=0.9000 f1=0.8032 | time=24.2s
Early stopping at epoch 61

#### Filter Comparison 120 vs 180 (Best Trial)
1] Trial 0: filters=180, lr=8.51e-05, wd=1.86e-06, pct_start=0.23
* Epoch 055 | train_loss=0.3038 acc=0.8571 | val_loss=0.3549 acc=0.8413 | prec=0.8606 rec=0.8136 spec=0.8688 f1=0.8364 | time=23.6s
Early stopping at epoch 55

2] Trial 1: filters=180, lr=2.85e-04, wd=3.40e-06, pct_start=0.23
* Epoch 058 | train_loss=0.2221 acc=0.8758 | val_loss=0.3680 acc=0.8707 | prec=0.8688 rec=0.8727 spec=0.8688 f1=0.8707 | time=23.7s
Early stopping at epoch 58

3]Trial 2: filters=120, lr=4.35e-05, wd=8.53e-06, pct_start=0.23
* Epoch 100 | train_loss=0.4184 acc=0.8191 | val_loss=0.4237 acc=0.8322 | prec=0.8349 rec=0.8273 spec=0.8371 f1=0.8311 | time=12.9s

4] Trial 4: filters=180, lr=7.40e-05, wd=2.30e-06, pct_start=0.23
* Epoch 064 | train_loss=0.2713 acc=0.8673 | val_loss=0.3856 acc=0.8458 | prec=0.8423 rec=0.8500 spec=0.8416 f1=0.8462 | time=23.5s
Early stopping at epoch 64

5] Trial 7: filters=120, lr=1.24e-04, wd=1.13e-05, pct_start=0.23
* Epoch 061 | train_loss=0.2986 acc=0.8667 | val_loss=0.3796 acc=0.8435 | prec=0.9218 rec=0.7500 spec=0.9367 f1=0.8271 | time=12.8s
Early stopping at epoch 61

6] Trial 8: filters=180, lr=1.07e-04, wd=4.01e-04, pct_start=0.23
* Epoch 059 | train_loss=0.2492 acc=0.8746 | val_loss=0.3780 acc=0.8594 | prec=0.8185 rec=0.9227 spec=0.7964 f1=0.8675 | time=23.3s
Early stopping at epoch 59

7] Trial 9: filters=120, lr=9.18e-05, wd=4.24e-06, pct_start=0.23
* Epoch 060 | train_loss=0.3007 acc=0.8503 | val_loss=0.4189 acc=0.8231 | prec=0.8515 rec=0.7818 spec=0.8643 f1=0.8152 | time=12.8s
Early stopping at epoch 60

#### Decrease the dropout rate and alleviate regularization
* Decrease the dropout rate = 0.2 -> 0.1 and update the EEGformer model to improve the train_loss
* Decrease the dropout rate = 0.5 -> 0.3 in CNN Decoder


In [ ]:
import os
import json
import random
import time
import numpy as np
import torch
import torch.nn as nn
import wandb
import optuna

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix
)
from torch.utils.data import DataLoader, Subset, Dataset

from eeg_dataset import EEGDataset
from model_optimized_6 import EEGformer

# ─── Configuration ───────────────────────────────────────────────
DATA_DIR    = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE  = 'labels.json'
DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MAX_EPOCHS  = 200
BATCH_SIZE  = 32
NUM_WORKERS = 4
ES_PATIENCE = 15

# ─── Dataset wrapper ─────────────────────────────────────────────
class BinaryEEGDataset(Dataset):
    def __init__(self, raw_ds, metas):
        self.raw_ds = raw_ds
        self.metas  = metas
    def __len__(self):
        return len(self.raw_ds)
    def __getitem__(self, idx):
        x, _ = self.raw_ds[idx]
        y     = self.metas[idx]['label']
        return x, y

# ─── Prepare data ────────────────────────────────────────────────
with open(os.path.join(DATA_DIR, LABEL_FILE), 'r') as f:
    all_meta = json.load(f)
train_meta = [d for d in all_meta if d['type']=='train']
class0, class1 = 'A','C'

# Undersample for balance
data0 = [d for d in train_meta if d['label']==class0]
data1 = [d for d in train_meta if d['label']==class1]
min_count = min(len(data0), len(data1))
samp0 = random.sample(data0, k=min_count)
samp1 = random.sample(data1, k=min_count)
balanced_meta = samp0 + samp1
random.shuffle(balanced_meta)

# Map labels to 0/1
label_map = {class0:0, class1:1}
for d in balanced_meta:
    d['label'] = label_map[d['label']]

# Build dataset
raw_ds  = EEGDataset(DATA_DIR, balanced_meta)
dataset = BinaryEEGDataset(raw_ds, balanced_meta)
labels  = np.array([d['label'] for d in balanced_meta])

# Single train/validation split
indices = np.arange(len(dataset))
train_idx, val_idx = train_test_split(
    indices, test_size=0.2, stratify=labels, random_state=0
)

# ─── Optuna objective ────────────────────────────────────────────
def objective(trial):
    # Sample hyperparameters
    lr          = trial.suggest_float('lr', 1e-5, 5e-4, log=True)
    wd          = trial.suggest_float('wd', 1e-6, 5e-4, log=True)
    pct_start   = 0.23  # fixed
    num_filters = 120

    print(f"Trial {trial.number}: filters={num_filters}, lr={lr:.2e}, wd={wd:.2e}, pct_start={pct_start:.2f}")

    # W&B init for this trial
    wandb.init(
        project='eeg-optuna-binary-AD-CN-single-fold-6',
        name=f'trial{trial.number}',
        config={'lr': lr, 'wd': wd, 'pct_start': pct_start, 'filters': num_filters},
        reinit=True
    )

    # DataLoaders
    train_loader = DataLoader(
        Subset(dataset, train_idx),
        batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS
    )
    val_loader   = DataLoader(
        Subset(dataset, val_idx),
        batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
    )

    # Model, optimizer, scheduler, criterion
    input_len = dataset[0][0].shape[-1]
    model = EEGformer(
        in_channels=19,
        input_length=input_len,
        kernel_size=10,
        num_filters=num_filters,
        num_heads=3,
        num_blocks=1,
        num_segments=5,
        num_classes=2
    ).to(DEVICE)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    total_steps = MAX_EPOCHS * len(train_loader)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=lr,
        total_steps=total_steps,
        pct_start=pct_start,
        anneal_strategy='cos',
        cycle_momentum=False
    )
    criterion = nn.CrossEntropyLoss()

    best_loss = float('inf')
    es_count  = 0

    # Epoch loop
    for epoch in range(1, MAX_EPOCHS + 1):
        t0 = time.time()

        # — train —
        model.train()
        tloss = tcorrect = ttotal = 0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()
            scheduler.step()

            # Dynamic Weight Decay
            cur_lr = optimizer.param_groups[0]['lr']
            new_wd = wd * (cur_lr / lr)
            for g in optimizer.param_groups:
                g['weight_decay'] = new_wd

            tloss    += loss.item()
            preds     = logits.argmax(1)
            tcorrect += (preds == y).sum().item()
            ttotal   += y.size(0)

        train_loss = tloss / len(train_loader)
        train_acc  = tcorrect / ttotal

        # — validate —
        model.eval()
        vloss = vcorrect = vtotal = 0
        val_preds, val_labels = [], []
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                loss   = criterion(logits, y)
                vloss  += loss.item()

                preds   = logits.argmax(1)
                vcorrect += (preds == y).sum().item()
                vtotal   += y.size(0)

                val_preds.append(preds.cpu().numpy())
                val_labels.append(y.cpu().numpy())

        val_loss   = vloss / len(val_loader)
        val_preds  = np.concatenate(val_preds)
        val_labels = np.concatenate(val_labels)
        val_acc    = (val_preds == val_labels).mean()
        precision  = precision_score(val_labels, val_preds, zero_division=0)
        recall     = recall_score(val_labels, val_preds, zero_division=0)
        f1         = f1_score(val_labels, val_preds, zero_division=0)

        # Compute specificity = TN / (TN + FP)
        tn, fp, fn, tp = confusion_matrix(val_labels, val_preds).ravel()
        specificity     = tn / (tn + fp)

        elapsed = time.time() - t0

        # Terminal output & W&B logging
        print(
            f"Epoch {epoch:03d} | "
            f"train_loss={train_loss:.4f} acc={train_acc:.4f} | "
            f"val_loss={val_loss:.4f} acc={val_acc:.4f} | "
            f"prec={precision:.4f} rec={recall:.4f} spec={specificity:.4f} f1={f1:.4f} | "
            f"time={elapsed:.1f}s"
        )
        wandb.log({
            'train_loss': train_loss,
            'train_accuracy': train_acc,
            'validation_loss': val_loss,
            'validation_accuracy': val_acc,
            'precision': precision,
            'recall': recall,
            'specificity': specificity,
            'f1_score': f1
        }, step=epoch)

        # Early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            es_count  = 0
        else:
            es_count += 1
            if es_count >= ES_PATIENCE:
                print(f"Early stopping at epoch {epoch}")
                break

    wandb.finish()
    return val_loss, val_acc

# ─── Run Optuna study ─────────────────────────────────────────────
study = optuna.create_study(directions=['minimize', 'maximize'])
study.optimize(objective, n_trials=10)

# ─── Print Pare토-optimal trials ──────────────────────────────────
print("Pareto-optimal trials (val_loss, val_acc) and their params:")
for t in study.best_trials:
    print(
        f" Trial #{t.number}: values={t.values}\n"
        f"              params={t.params}"
    )


Now using CUDA device 0
Enabling CUDA with 39.14 GiB available memory


[I 2025-05-06 02:22:00,633] A new study created in memory with name: no-name-5fa48b97-14e3-47ee-868d-0dbd6a0ae8ca


Trial 0: filters=120, lr=3.43e-04, wd=6.98e-06, pct_start=0.23


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Epoch 001 | train_loss=0.7333 acc=0.5122 | val_loss=0.6980 acc=0.4989 | prec=0.4989 rec=1.0000 spec=0.0000 f1=0.6657 | time=186.3s
Epoch 002 | train_loss=0.7380 acc=0.4884 | val_loss=0.6960 acc=0.4989 | prec=0.4989 rec=1.0000 spec=0.0000 f1=0.6657 | time=13.0s
Epoch 003 | train_loss=0.7203 acc=0.5122 | val_loss=0.6935 acc=0.4989 | prec=0.4989 rec=1.0000 spec=0.0000 f1=0.6657 | time=13.0s
Epoch 004 | train_loss=0.7288 acc=0.4838 | val_loss=0.6987 acc=0.4989 | prec=0.4989 rec=1.0000 spec=0.0000 f1=0.6657 | time=13.0s
Epoch 005 | train_loss=0.7228 acc=0.4969 | val_loss=0.6954 acc=0.4989 | prec=0.4989 rec=1.0000 spec=0.0000 f1=0.6657 | time=12.9s
Epoch 006 | train_loss=0.7125 acc=0.5150 | val_loss=0.6901 acc=0.4989 | prec=0.4989 rec=1.0000 spec=0.0000 f1=0.6657 | time=12.8s
Epoch 007 | train_loss=0.7134 acc=0.5150 | val_loss=0.6869 acc=0.4989 | prec=0.4989 rec=1.0000 spec=0.0000 f1=0.6657 | time=12.8s
Epoch 008 | train_loss=0.6899 acc=0.5462 | val_loss=0.6677 acc=0.6984 | prec=0.6548 rec=0

...

Trial 9: filters=120, lr=2.22e-05, wd=1.91e-05, pct_start=0.23


Epoch 001 | train_loss=0.7443 acc=0.5213 | val_loss=0.7111 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.9s
Epoch 002 | train_loss=0.7519 acc=0.4838 | val_loss=0.7098 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.7s
Epoch 003 | train_loss=0.7445 acc=0.4929 | val_loss=0.7014 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.7s
Epoch 004 | train_loss=0.7377 acc=0.4957 | val_loss=0.6990 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.7s
Epoch 005 | train_loss=0.7345 acc=0.4940 | val_loss=0.6987 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.9s
Epoch 006 | train_loss=0.7316 acc=0.5060 | val_loss=0.7009 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.9s
Epoch 007 | train_loss=0.7307 acc=0.5088 | val_loss=0.6992 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.8s
Epoch 008 | train_loss=0.7218 acc=0.5105 | val_loss=0.7000 acc=0.5011 | prec=0.0000 rec=0.

f1_score,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▇▇▇██████████████████████
precision,▁▁▁▁▁▁▁▁▁▁▁▁▁██▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▇▇▆▇▇▇▇
recall,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▂▆▇▆▇▇█▇█▇▇█▇▇█▇█▇▇███▇█
specificity,██████████▅▇█▄▇▆▅▅▃▃▄▃▅▅▅▅▅▆▇▅▄▂▅▆▄▆▄▁▆▃
train_accuracy,▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▄▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇█▇█████
train_loss,██████████▇▇▇▇▆▆▅▅▅▅▄▄▄▃▃▃▃▂▂▃▂▂▂▂▂▁▁▁▁▁
validation_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▃▂▆▇▇▇▇▇▇▇▇▇█████████▇█████
validation_loss,████████████████▇▇▅▄▄▃▃▃▃▃▃▂▃▂▁▁▁▂▂▂▁▁▁▁
f1_score,0.8394
precision,0.79352
recall,0.89091


[I 2025-05-06 03:47:10,402] Trial 9 finished with values: [0.38471304305962156, 0.8299319727891157] and parameters: {'lr': 2.2239463334903503e-05, 'wd': 1.9124782834348135e-05}.


Pareto-optimal trials (val_loss, val_acc) and their params:
 Trial #2: values=[0.4335672802158764, 0.8390022675736961]
              params={'lr': 6.704038535844278e-05, 'wd': 1.1003236848381844e-05}
 Trial #9: values=[0.38471304305962156, 0.8299319727891157]
              params={'lr': 2.2239463334903503e-05, 'wd': 1.9124782834348135e-05}


#### After decreasing the dropout rate and alleviate regularization (Best Result)

1] Trial 2: filters=120, lr=6.70e-05, wd=1.10e-05, pct_start=0.23
* Epoch 062 | train_loss=0.1612 acc=0.9234 | val_loss=0.4336 acc=0.8390 | prec=0.7922 rec=0.9182 spec=0.7602 f1=0.8505 | time=12.9s
Early stopping at epoch 62

2] Trial 9: filters=120, lr=2.22e-05, wd=1.91e-05, pct_start=0.23
* Epoch 084 | train_loss=0.2691 acc=0.9007 | val_loss=0.3847 acc=0.8299 | prec=0.7935 rec=0.8909 spec=0.7692 f1=0.8394 | time=12.8s
Early stopping at epoch 84

#### Decrease the search space and try again
- Change the MAX EPOCH to 150
- Decrease dropout rate from 0.3 to 0.2 in CNNDecoder

In [ ]:
import os
import json
import random
import time
import numpy as np
import torch
import torch.nn as nn
import wandb
import optuna

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix
)
from torch.utils.data import DataLoader, Subset, Dataset

from eeg_dataset import EEGDataset
from model_optimized_7 import EEGformer

# ─── Configuration ───────────────────────────────────────────────
DATA_DIR    = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE  = 'labels.json'
DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MAX_EPOCHS  = 150
BATCH_SIZE  = 32
NUM_WORKERS = 4
ES_PATIENCE = 15

# ─── Dataset wrapper ─────────────────────────────────────────────
class BinaryEEGDataset(Dataset):
    def __init__(self, raw_ds, metas):
        self.raw_ds = raw_ds
        self.metas  = metas
    def __len__(self):
        return len(self.raw_ds)
    def __getitem__(self, idx):
        x, _ = self.raw_ds[idx]
        y     = self.metas[idx]['label']
        return x, y

# ─── Prepare data ────────────────────────────────────────────────
with open(os.path.join(DATA_DIR, LABEL_FILE), 'r') as f:
    all_meta = json.load(f)
train_meta = [d for d in all_meta if d['type']=='train']
class0, class1 = 'A','C'

# Undersample for balance
data0 = [d for d in train_meta if d['label']==class0]
data1 = [d for d in train_meta if d['label']==class1]
min_count = min(len(data0), len(data1))
samp0 = random.sample(data0, k=min_count)
samp1 = random.sample(data1, k=min_count)
balanced_meta = samp0 + samp1
random.shuffle(balanced_meta)

# Map labels to 0/1
label_map = {class0:0, class1:1}
for d in balanced_meta:
    d['label'] = label_map[d['label']]

# Build dataset
raw_ds  = EEGDataset(DATA_DIR, balanced_meta)
dataset = BinaryEEGDataset(raw_ds, balanced_meta)
labels  = np.array([d['label'] for d in balanced_meta])

# Single train/validation split
indices = np.arange(len(dataset))
train_idx, val_idx = train_test_split(
    indices, test_size=0.2, stratify=labels, random_state=0
)

# ─── Optuna objective ────────────────────────────────────────────
def objective(trial):
    # Sample hyperparameters
    lr          = trial.suggest_float('lr', 1e-5, 2e-4, log=True)
    wd          = trial.suggest_float('wd', 1e-6, 1e-4, log=True)
    pct_start   = 0.2  # fixed
    num_filters = 120

    print(f"Trial {trial.number}: filters={num_filters}, lr={lr:.2e}, wd={wd:.2e}, pct_start={pct_start:.2f}")

    # W&B init for this trial
    wandb.init(
        project='eeg-optuna-binary-AD-CN-single-fold-7',
        name=f'trial{trial.number}',
        config={'lr': lr, 'wd': wd, 'pct_start': pct_start, 'filters': num_filters},
        reinit=True
    )

    # DataLoaders
    train_loader = DataLoader(
        Subset(dataset, train_idx),
        batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS
    )
    val_loader   = DataLoader(
        Subset(dataset, val_idx),
        batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
    )

    # Model, optimizer, scheduler, criterion
    input_len = dataset[0][0].shape[-1]
    model = EEGformer(
        in_channels=19,
        input_length=input_len,
        kernel_size=10,
        num_filters=num_filters,
        num_heads=3,
        num_blocks=1,
        num_segments=5,
        num_classes=2
    ).to(DEVICE)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    total_steps = MAX_EPOCHS * len(train_loader)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=lr,
        total_steps=total_steps,
        pct_start=pct_start,
        anneal_strategy='cos',
        cycle_momentum=False
    )
    criterion = nn.CrossEntropyLoss()

    best_loss = float('inf')
    es_count  = 0

    # Epoch loop
    for epoch in range(1, MAX_EPOCHS + 1):
        t0 = time.time()

        # — train —
        model.train()
        tloss = tcorrect = ttotal = 0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()
            scheduler.step()

            # Dynamic Weight Decay
            cur_lr = optimizer.param_groups[0]['lr']
            new_wd = wd * (cur_lr / lr)
            for g in optimizer.param_groups:
                g['weight_decay'] = new_wd

            tloss    += loss.item()
            preds     = logits.argmax(1)
            tcorrect += (preds == y).sum().item()
            ttotal   += y.size(0)

        train_loss = tloss / len(train_loader)
        train_acc  = tcorrect / ttotal

        # — validate —
        model.eval()
        vloss = vcorrect = vtotal = 0
        val_preds, val_labels = [], []
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                loss   = criterion(logits, y)
                vloss  += loss.item()

                preds   = logits.argmax(1)
                vcorrect += (preds == y).sum().item()
                vtotal   += y.size(0)

                val_preds.append(preds.cpu().numpy())
                val_labels.append(y.cpu().numpy())

        val_loss   = vloss / len(val_loader)
        val_preds  = np.concatenate(val_preds)
        val_labels = np.concatenate(val_labels)
        val_acc    = (val_preds == val_labels).mean()
        precision  = precision_score(val_labels, val_preds, zero_division=0)
        recall     = recall_score(val_labels, val_preds, zero_division=0)
        f1         = f1_score(val_labels, val_preds, zero_division=0)

        # Compute specificity = TN / (TN + FP)
        tn, fp, fn, tp = confusion_matrix(val_labels, val_preds).ravel()
        specificity     = tn / (tn + fp)

        elapsed = time.time() - t0

        # Terminal output & W&B logging
        print(
            f"Epoch {epoch:03d} | "
            f"train_loss={train_loss:.4f} acc={train_acc:.4f} | "
            f"val_loss={val_loss:.4f} acc={val_acc:.4f} | "
            f"prec={precision:.4f} rec={recall:.4f} spec={specificity:.4f} f1={f1:.4f} | "
            f"time={elapsed:.1f}s"
        )
        wandb.log({
            'train_loss': train_loss,
            'train_accuracy': train_acc,
            'validation_loss': val_loss,
            'validation_accuracy': val_acc,
            'precision': precision,
            'recall': recall,
            'specificity': specificity,
            'f1_score': f1
        }, step=epoch)

        # Early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            es_count  = 0
        else:
            es_count += 1
            if es_count >= ES_PATIENCE:
                print(f"Early stopping at epoch {epoch}")
                break

    wandb.finish()
    return val_loss, val_acc

# ─── Run Optuna study ─────────────────────────────────────────────
study = optuna.create_study(directions=['minimize', 'maximize'])
study.optimize(objective, n_trials=10)

# ─── Print Pareto-optimal trials ──────────────────────────────────
print("Pareto-optimal trials (val_loss, val_acc) and their params:")
for t in study.best_trials:
    print(
        f" Trial #{t.number}: values={t.values}\n"
        f"              params={t.params}"
    )


Now using CUDA device 0
Enabling CUDA with 39.14 GiB available memory


[I 2025-05-06 05:48:13,942] A new study created in memory with name: no-name-32fbebaf-7b0f-4b2e-ba90-1dd2cf065fb4


Trial 0: filters=120, lr=1.26e-05, wd=1.65e-05, pct_start=0.20


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Epoch 001 | train_loss=0.7557 acc=0.4912 | val_loss=0.7003 acc=0.4989 | prec=0.4989 rec=1.0000 spec=0.0000 f1=0.6657 | time=31.3s
Epoch 002 | train_loss=0.7505 acc=0.4940 | val_loss=0.6980 acc=0.4989 | prec=0.4989 rec=1.0000 spec=0.0000 f1=0.6657 | time=12.9s
Epoch 003 | train_loss=0.7416 acc=0.5122 | val_loss=0.6940 acc=0.4966 | prec=0.4970 rec=0.7545 spec=0.2398 f1=0.5993 | time=12.8s
Epoch 004 | train_loss=0.7259 acc=0.5133 | val_loss=0.6941 acc=0.5102 | prec=0.5062 rec=0.7409 spec=0.2805 f1=0.6015 | time=12.9s
Epoch 005 | train_loss=0.7220 acc=0.5082 | val_loss=0.6945 acc=0.5011 | prec=0.5000 rec=0.9773 spec=0.0271 f1=0.6615 | time=13.0s
Epoch 006 | train_loss=0.7226 acc=0.5065 | val_loss=0.6944 acc=0.4989 | prec=0.4989 rec=1.0000 spec=0.0000 f1=0.6657 | time=12.9s
Epoch 007 | train_loss=0.7196 acc=0.5133 | val_loss=0.6938 acc=0.5034 | prec=0.5011 rec=0.9909 spec=0.0181 f1=0.6656 | time=12.8s
Epoch 008 | train_loss=0.7061 acc=0.5224 | val_loss=0.6936 acc=0.4966 | prec=0.4975 rec=0.

...

Trial 9: filters=120, lr=1.44e-05, wd=4.65e-06, pct_start=0.20


Epoch 001 | train_loss=0.7235 acc=0.4974 | val_loss=0.7062 acc=0.4989 | prec=0.4989 rec=1.0000 spec=0.0000 f1=0.6657 | time=12.9s
Epoch 002 | train_loss=0.7195 acc=0.4991 | val_loss=0.7038 acc=0.4989 | prec=0.4989 rec=1.0000 spec=0.0000 f1=0.6657 | time=12.8s
Epoch 003 | train_loss=0.7166 acc=0.5020 | val_loss=0.7090 acc=0.4989 | prec=0.4989 rec=1.0000 spec=0.0000 f1=0.6657 | time=12.9s
Epoch 004 | train_loss=0.7156 acc=0.4872 | val_loss=0.7044 acc=0.4989 | prec=0.4989 rec=1.0000 spec=0.0000 f1=0.6657 | time=12.9s
Epoch 005 | train_loss=0.7073 acc=0.5235 | val_loss=0.7109 acc=0.4989 | prec=0.4989 rec=1.0000 spec=0.0000 f1=0.6657 | time=12.9s
Epoch 006 | train_loss=0.7104 acc=0.5111 | val_loss=0.6967 acc=0.4989 | prec=0.4989 rec=1.0000 spec=0.0000 f1=0.6657 | time=12.9s
Epoch 007 | train_loss=0.7142 acc=0.5009 | val_loss=0.6960 acc=0.4989 | prec=0.4989 rec=1.0000 spec=0.0000 f1=0.6657 | time=12.9s
Epoch 008 | train_loss=0.7096 acc=0.5167 | val_loss=0.6982 acc=0.4989 | prec=0.4989 rec=1.

f1_score,▆▆▆▆▆▆▆▆▆▆▅▆▆▆▁▅▇▇▇██████▇▆▇▇▇▇█▇▇▇▇▇▇▇▇
precision,▁▁▁▁▁▁▁▁▁▁▁▁▂▁▄▆▇▆▇▆▇▇▆▇▇▆▇▇▇▇▇▇▇▇▇█▇███
recall,█████████████▁▁▃▆▅▆▅▅▅▅▅▅▅▅▅▄▄▄▄▅▅▄▅▄▄▄▄
specificity,▁▁▁▁▁▁▁▃▁▆▁██▇█▇▇█▇█▇▇▇█████████████████
train_accuracy,▁▁▁▁▁▁▁▁▂▁▅▆▆▆▆▇▇▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇█▇▇█████
train_loss,███████████▇▇▆▆▅▄▄▄▄▃▃▃▃▂▂▂▃▂▂▂▂▁▁▂▂▁▁▂▁
validation_accuracy,▁▁▁▁▁▁▁▁▁▂▁▇▆▇▇▇█▇███▇█████▇▇▇▇██▇█▇███▇
validation_loss,███████████▆▆▆▅▅▄▄▃▃▃▂▂▂▂▂▂▂▂▁▂▂▂▁▂▁▂▁▁▂
f1_score,0.74457
precision,0.92568
recall,0.62273


[I 2025-05-06 08:22:04,232] Trial 9 finished with values: [0.44814891261713846, 0.7868480725623582] and parameters: {'lr': 1.4419843522360348e-05, 'wd': 4.6549935543509025e-06}.


Pareto-optimal trials (val_loss, val_acc) and their params:
 Trial #1: values=[0.39670870133808683, 0.8526077097505669]
              params={'lr': 2.4166877561293106e-05, 'wd': 8.262372253115548e-05}
 Trial #7: values=[0.3690581832613264, 0.8390022675736961]
              params={'lr': 2.3089123428359085e-05, 'wd': 7.204776343575214e-06}


Return back to model_optimized_6.py
- Block = 1, Head = 3
- Decrease the search space
- MAX Epoch = 150
- Dropout rate = 0.2 (model_optimized_7.py) in CNNDecoder makes a bit worse in generalization
- Again, increase the dropout rate = 0.2 to 0.3 in CNNDecoder

In [ ]:
import os
import json
import random
import time
import numpy as np
import torch
import torch.nn as nn
import wandb
import optuna

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix
)
from torch.utils.data import DataLoader, Subset, Dataset

from eeg_dataset import EEGDataset
from model_optimized_6 import EEGformer

# ─── Configuration ───────────────────────────────────────────────
DATA_DIR    = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE  = 'labels.json'
DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MAX_EPOCHS  = 150
BATCH_SIZE  = 32
NUM_WORKERS = 4
ES_PATIENCE = 15

# ─── Dataset wrapper ─────────────────────────────────────────────
class BinaryEEGDataset(Dataset):
    def __init__(self, raw_ds, metas):
        self.raw_ds = raw_ds
        self.metas  = metas
    def __len__(self):
        return len(self.raw_ds)
    def __getitem__(self, idx):
        x, _ = self.raw_ds[idx]
        y     = self.metas[idx]['label']
        return x, y

# ─── Prepare data ────────────────────────────────────────────────
with open(os.path.join(DATA_DIR, LABEL_FILE), 'r') as f:
    all_meta = json.load(f)
train_meta = [d for d in all_meta if d['type']=='train']
class0, class1 = 'A','C'

# Undersample for balance
data0 = [d for d in train_meta if d['label']==class0]
data1 = [d for d in train_meta if d['label']==class1]
min_count = min(len(data0), len(data1))
samp0 = random.sample(data0, k=min_count)
samp1 = random.sample(data1, k=min_count)
balanced_meta = samp0 + samp1
random.shuffle(balanced_meta)

# Map labels to 0/1
label_map = {class0:0, class1:1}
for d in balanced_meta:
    d['label'] = label_map[d['label']]

# Build dataset
raw_ds  = EEGDataset(DATA_DIR, balanced_meta)
dataset = BinaryEEGDataset(raw_ds, balanced_meta)
labels  = np.array([d['label'] for d in balanced_meta])

# Single train/validation split
indices = np.arange(len(dataset))
train_idx, val_idx = train_test_split(
    indices, test_size=0.2, stratify=labels, random_state=0
)

# ─── Optuna objective ────────────────────────────────────────────
def objective(trial):
    # Sample hyperparameters
    lr          = trial.suggest_float('lr', 1e-5, 2e-4, log=True)
    wd          = trial.suggest_float('wd', 1e-6, 1e-4, log=True)
    pct_start   = 0.2  # fixed
    num_filters = 120

    print(f"Trial {trial.number}: filters={num_filters}, lr={lr:.2e}, wd={wd:.2e}, pct_start={pct_start:.2f}")

    # W&B init for this trial
    wandb.init(
        project='eeg-optuna-binary-AD-CN-single-fold-8',
        name=f'trial{trial.number}',
        config={'lr': lr, 'wd': wd, 'pct_start': pct_start, 'filters': num_filters},
        reinit=True
    )

    # DataLoaders
    train_loader = DataLoader(
        Subset(dataset, train_idx),
        batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS
    )
    val_loader   = DataLoader(
        Subset(dataset, val_idx),
        batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
    )

    # Model, optimizer, scheduler, criterion
    input_len = dataset[0][0].shape[-1]
    model = EEGformer(
        in_channels=19,
        input_length=input_len,
        kernel_size=10,
        num_filters=num_filters,
        num_heads=3,
        num_blocks=1,
        num_segments=5,
        num_classes=2
    ).to(DEVICE)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    total_steps = MAX_EPOCHS * len(train_loader)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=lr,
        total_steps=total_steps,
        pct_start=pct_start,
        anneal_strategy='cos',
        cycle_momentum=False
    )
    criterion = nn.CrossEntropyLoss()

    best_loss = float('inf')
    es_count  = 0

    # Epoch loop
    for epoch in range(1, MAX_EPOCHS + 1):
        t0 = time.time()

        # — train —
        model.train()
        tloss = tcorrect = ttotal = 0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()
            scheduler.step()

            # Dynamic Weight Decay
            cur_lr = optimizer.param_groups[0]['lr']
            new_wd = wd * (cur_lr / lr)
            for g in optimizer.param_groups:
                g['weight_decay'] = new_wd

            tloss    += loss.item()
            preds     = logits.argmax(1)
            tcorrect += (preds == y).sum().item()
            ttotal   += y.size(0)

        train_loss = tloss / len(train_loader)
        train_acc  = tcorrect / ttotal

        # — validate —
        model.eval()
        vloss = vcorrect = vtotal = 0
        val_preds, val_labels = [], []
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                loss   = criterion(logits, y)
                vloss  += loss.item()

                preds   = logits.argmax(1)
                vcorrect += (preds == y).sum().item()
                vtotal   += y.size(0)

                val_preds.append(preds.cpu().numpy())
                val_labels.append(y.cpu().numpy())

        val_loss   = vloss / len(val_loader)
        val_preds  = np.concatenate(val_preds)
        val_labels = np.concatenate(val_labels)
        val_acc    = (val_preds == val_labels).mean()
        precision  = precision_score(val_labels, val_preds, zero_division=0)
        recall     = recall_score(val_labels, val_preds, zero_division=0)
        f1         = f1_score(val_labels, val_preds, zero_division=0)

        # Compute specificity = TN / (TN + FP)
        tn, fp, fn, tp = confusion_matrix(val_labels, val_preds).ravel()
        specificity     = tn / (tn + fp)

        elapsed = time.time() - t0

        # Terminal output & W&B logging
        print(
            f"Epoch {epoch:03d} | "
            f"train_loss={train_loss:.4f} acc={train_acc:.4f} | "
            f"val_loss={val_loss:.4f} acc={val_acc:.4f} | "
            f"prec={precision:.4f} rec={recall:.4f} spec={specificity:.4f} f1={f1:.4f} | "
            f"time={elapsed:.1f}s"
        )
        wandb.log({
            'train_loss': train_loss,
            'train_accuracy': train_acc,
            'validation_loss': val_loss,
            'validation_accuracy': val_acc,
            'precision': precision,
            'recall': recall,
            'specificity': specificity,
            'f1_score': f1
        }, step=epoch)

        # Early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            es_count  = 0
        else:
            es_count += 1
            if es_count >= ES_PATIENCE:
                print(f"Early stopping at epoch {epoch}")
                break

    wandb.finish()
    return val_loss, val_acc

# ─── Run Optuna study ─────────────────────────────────────────────
study = optuna.create_study(directions=['minimize', 'maximize'])
study.optimize(objective, n_trials=10)

# ─── Print Pare토-optimal trials ──────────────────────────────────
print("Pareto-optimal trials (val_loss, val_acc) and their params:")
for t in study.best_trials:
    print(
        f" Trial #{t.number}: values={t.values}\n"
        f"              params={t.params}"
    )


[I 2025-05-06 08:36:52,068] A new study created in memory with name: no-name-fe340887-6a2b-4636-a3c6-4016ff16809e


Trial 0: filters=120, lr=8.53e-05, wd=2.20e-05, pct_start=0.20


Epoch 001 | train_loss=0.7195 acc=0.4991 | val_loss=0.6977 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=15.3s
Epoch 002 | train_loss=0.7175 acc=0.5026 | val_loss=0.6945 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.9s
Epoch 003 | train_loss=0.7227 acc=0.4912 | val_loss=0.6957 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.9s
Epoch 004 | train_loss=0.7071 acc=0.5122 | val_loss=0.6945 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.8s
Epoch 005 | train_loss=0.7095 acc=0.5133 | val_loss=0.6932 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.9s
Epoch 006 | train_loss=0.7154 acc=0.4997 | val_loss=0.6929 acc=0.5102 | prec=0.5062 rec=0.7409 spec=0.2805 f1=0.6015 | time=12.9s
Epoch 007 | train_loss=0.7127 acc=0.5031 | val_loss=0.6937 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.9s
Epoch 008 | train_loss=0.7070 acc=0.5167 | val_loss=0.6927 acc=0.5329 | prec=0.5814 rec=0.

f1_score,▁▁▁▁▁▁▄▃▆▇▇▇██████████████▇▇███████████▇
precision,▁▁▅▁▅▅▆█▇▇█████▇█▇▇█▆▇█▇█▇▇▇█▇█▇▇█▇▇█▇▇█
recall,▁▁▁▁▁▁▃▃▆▄▇▅▆▆█▇▇██▇▇██▇▇█▆█▆▆▇█▇█▇▇██▇▆
specificity,███▁█▂▆▆▇▇▇▆▇▇▇▆▇█▇▅▇▆▆▇▆▆▇▇▆▇▇▇▇▇▇▇▇▆▆▇
train_accuracy,▁▁▁▁▁▁▁▁▁▂▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇█▇████████████
train_loss,███████▇▇▆▆▆▅▅▅▄▃▃▃▃▃▃▂▂▃▂▃▂▂▂▂▁▂▁▁▁▁▁▁▁
validation_accuracy,▁▁▁▁▁▁▂▂▄▅▇▇▇▇█▇▇▇▇█▇▇▇█▇██▇█▇█████▇████
validation_loss,█████████▇▄▄▄▃▃▃▃▃▂▁▂▁▁▁▁▂▃▁▁▂▁▁▂▂▂▁▂▂▃▄
f1_score,0.79487
precision,0.91176
recall,0.70455


[I 2025-05-06 08:50:06,102] Trial 0 finished with values: [0.48875192765678677, 0.81859410430839] and parameters: {'lr': 8.534653675580783e-05, 'wd': 2.2038899844018073e-05}.


...

Trial 9: filters=120, lr=2.38e-05, wd=8.19e-05, pct_start=0.20


Epoch 001 | train_loss=0.7661 acc=0.4918 | val_loss=0.6968 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=13.0s
Epoch 002 | train_loss=0.7619 acc=0.4861 | val_loss=0.6994 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=13.0s
Epoch 003 | train_loss=0.7373 acc=0.5145 | val_loss=0.6967 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=13.0s
Epoch 004 | train_loss=0.7394 acc=0.4957 | val_loss=0.6966 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.9s
Epoch 005 | train_loss=0.7349 acc=0.5014 | val_loss=0.6949 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=13.0s
Epoch 006 | train_loss=0.7447 acc=0.4844 | val_loss=0.6940 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=13.0s
Epoch 007 | train_loss=0.7252 acc=0.4957 | val_loss=0.6931 acc=0.5125 | prec=0.8571 rec=0.0273 spec=0.9955 f1=0.0529 | time=13.0s
Epoch 008 | train_loss=0.7205 acc=0.5020 | val_loss=0.6931 acc=0.5034 | prec=1.0000 rec=0.

f1_score,▁▁▁▁▁▁▁▁▁▁▂▃▇▇▇██████████▇██████████████
precision,▁▁▁▁▇▁▁█▁█▇▅▆▇▇▆▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▆▇▇▇▆▇
recall,▁▁▁▁▁▁▁▁▂▂█▇▆█▇▇▇▇▇▇▇█▇▇▇▇█▇██▇▇▇█████▇▇
specificity,██████████▅▆▂▅▅▅▄▄▄▅▅▃▅▅▅▅▄▄▁▆▆▃▃▃▄▄▄▄▅▂
train_accuracy,▁▁▁▁▁▁▁▂▂▂▄▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇██████████████
train_loss,█████▇▇▇▇▇▇▆▅▅▅▅▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁
validation_accuracy,▁▁▁▁▁▁▁▁▁▂▂▅▆▆▇▇▇██▇▇███▇█▇▆█▇████▇▇████
validation_loss,█████████▇▆▅▄▄▃▃▃▃▃▃▂▂▂▂▂▂▄▂▃▂▁▁▂▁▁▁▁▁▁▃
f1_score,0.81818
precision,0.72378
recall,0.94091


[I 2025-05-06 11:02:43,320] Trial 9 finished with values: [0.44804493870053974, 0.7913832199546486] and parameters: {'lr': 2.3847632993026627e-05, 'wd': 8.193296299960752e-05}.


Pareto-optimal trials (val_loss, val_acc) and their params:
 Trial #1: values=[0.3497606185930116, 0.8458049886621315]
              params={'lr': 6.436530952928205e-05, 'wd': 1.8466323534899384e-05}
 Trial #5: values=[0.3735910728573799, 0.854875283446712]
              params={'lr': 3.793344453740315e-05, 'wd': 4.5415221838197576e-05}
 Trial #6: values=[0.3590279668569565, 0.8503401360544217]
              params={'lr': 3.5210516472168496e-05, 'wd': 4.774224777773625e-05}


####

1] Trial 0: filters=120, lr=8.53e-05, wd=2.20e-05, pct_start=0.20
Epoch 061 | train_loss=0.1690 acc=0.9325 | val_loss=0.4888 acc=0.8186 | prec=0.9118 rec=0.7045 spec=0.9321 f1=0.7949 | time=13.6s
Early stopping at epoch 61

2] Trial 1: filters=120, lr=6.44e-05, wd=1.85e-05, pct_start=0.20
Epoch 054 | train_loss=0.1890 acc=0.9246 | val_loss=0.3498 acc=0.8458 | prec=0.8167 rec=0.8909 spec=0.8009 f1=0.8522 | time=12.8s
Early stopping at epoch 54

3] Trial 2: filters=120, lr=1.23e-04, wd=7.13e-05, pct_start=0.20
Epoch 041 | train_loss=0.1800 acc=0.9246 | val_loss=0.3866 acc=0.8413 | prec=0.8151 rec=0.8818 spec=0.8009 f1=0.8472 | time=12.9s
Early stopping at epoch 41

4] Trial 3: filters=120, lr=8.43e-05, wd=1.93e-06, pct_start=0.20
Epoch 048 | train_loss=0.1869 acc=0.9331 | val_loss=0.4680 acc=0.8345 | prec=0.8769 rec=0.7773 spec=0.8914 f1=0.8241 | time=12.8s
Early stopping at epoch 48

5] Trial 4: filters=120, lr=9.64e-05, wd=5.00e-06, pct_start=0.20
Epoch 045 | train_loss=0.2278 acc=0.9075 | val_loss=0.6337 acc=0.7619 | prec=0.6837 rec=0.9727 spec=0.5520 f1=0.8030 | time=13.0s
Early stopping at epoch 45

6] Trial 5: filters=120, lr=3.79e-05, wd=4.54e-05, pct_start=0.20
Epoch 076 | train_loss=0.2106 acc=0.9229 | val_loss=0.3736 acc=0.8549 | prec=0.8714 rec=0.8318 spec=0.8778 f1=0.8512 | time=12.9s
Early stopping at epoch 76

7] Trial 6: filters=120, lr=3.52e-05, wd=4.77e-05, pct_start=0.20
Epoch 078 | train_loss=0.2236 acc=0.9047 | val_loss=0.3590 acc=0.8503 | prec=0.8532 rec=0.8455 spec=0.8552 f1=0.8493 | time=12.9s
Early stopping at epoch 78

8] Trial 7: filters=120, lr=1.02e-05, wd=4.00e-05, pct_start=0.20
Epoch 121 | train_loss=0.3604 acc=0.8520 | val_loss=0.4047 acc=0.8299 | prec=0.8222 rec=0.8409 spec=0.8190 f1=0.8315 | time=12.8s
Early stopping at epoch 121

9] Trial 8: filters=120, lr=3.39e-05, wd=3.04e-05, pct_start=0.20
Epoch 084 | train_loss=0.1903 acc=0.9302 | val_loss=0.4149 acc=0.8435 | prec=0.8159 rec=0.8864 spec=0.8009 f1=0.8497 | time=13.0s
Early stopping at epoch 84

10] Trial 9: filters=120, lr=2.38e-05, wd=8.19e-05, pct_start=0.20
Epoch 069 | train_loss=0.2574 acc=0.9070 | val_loss=0.4480 acc=0.7914 | prec=0.7238 rec=0.9409 spec=0.6425 f1=0.8182 | time=12.8s
Early stopping at epoch 69

#### 5-Fold Cross Validation Result with fixed parameter

In [ ]:
import os
import json
import random
import time
import numpy as np
import torch
import torch.nn as nn
import wandb

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from torch.utils.data import DataLoader, Subset, Dataset

from eeg_dataset import EEGDataset
from model_optimized_6 import EEGformer

# ─── Configuration ───────────────────────────────────────────────
DATA_DIR    = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE  = 'labels.json'
DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MAX_EPOCHS  = 150
BATCH_SIZE  = 32
NUM_WORKERS = 4
ES_PATIENCE = 15

# ─── Fixed Hyper-parameters ───────────────────────────────────────
LR          = 5e-5
WD          = 1e-5
PCT_START   = 0.20
NUM_FILTERS = 120

# ─── Dataset wrapper ─────────────────────────────────────────────
class BinaryEEGDataset(Dataset):
    def __init__(self, raw_ds, metas):
        self.raw_ds = raw_ds
        self.metas  = metas
    def __len__(self):
        return len(self.raw_ds)
    def __getitem__(self, idx):
        x, _ = self.raw_ds[idx]
        y     = self.metas[idx]['label']
        return x, y

# ─── Load & balance metadata ─────────────────────────────────────
with open(os.path.join(DATA_DIR, LABEL_FILE), 'r') as f:
    all_meta = json.load(f)
train_meta = [d for d in all_meta if d['type']=='train']
class0, class1 = 'A','C'

data0 = [d for d in train_meta if d['label']==class0]
data1 = [d for d in train_meta if d['label']==class1]
min_count = min(len(data0), len(data1))

balanced_meta = random.sample(data0, min_count) + random.sample(data1, min_count)
random.shuffle(balanced_meta)

# map to 0/1
label_map = {class0:0, class1:1}
for d in balanced_meta:
    d['label'] = label_map[d['label']]

# build dataset & labels array
raw_ds  = EEGDataset(DATA_DIR, balanced_meta)
dataset = BinaryEEGDataset(raw_ds, balanced_meta)
labels  = np.array([d['label'] for d in balanced_meta])

# ─── 5-Fold Cross-Validation ─────────────────────────────────────
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# store per-fold metrics
fold_metrics = {'loss':[], 'acc':[], 'prec':[], 'rec':[], 'spec':[], 'f1':[]}

for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(labels)), labels), start=1):
    print(f"\n=== Fold {fold} ===")
    wandb.init(
        project="eeg-5fold-fixed-hp",
        name=f"fold{fold}",
        config={
            'lr': LR, 'wd': WD,
            'pct_start': PCT_START,
            'filters': NUM_FILTERS,
            'max_epochs': MAX_EPOCHS,
            'batch_size': BATCH_SIZE
        },
        reinit=True
    )

    train_loader = DataLoader(
        Subset(dataset, train_idx),
        batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS
    )
    val_loader = DataLoader(
        Subset(dataset, val_idx),
        batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
    )

    # build model, optimizer, scheduler, criterion
    input_len = dataset[0][0].shape[-1]
    model = EEGformer(
        in_channels=19,
        input_length=input_len,
        kernel_size=10,
        num_filters=NUM_FILTERS,
        num_heads=3,
        num_blocks=1,
        num_segments=5,
        num_classes=2
    ).to(DEVICE)

    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=LR,
        total_steps=MAX_EPOCHS * len(train_loader),
        pct_start=PCT_START,
        anneal_strategy='cos',
        cycle_momentum=False
    )
    criterion = nn.CrossEntropyLoss()

    best_loss = float('inf')
    es_count  = 0

    # ─── Epoch Loop ───────────────────────────────────────────────
    for epoch in range(1, MAX_EPOCHS+1):
        t0 = time.time()

        # — train —
        model.train()
        tloss = tcorrect = ttotal = 0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()
            scheduler.step()
            # dynamic weight decay
            cur_lr = optimizer.param_groups[0]['lr']
            for g in optimizer.param_groups:
                g['weight_decay'] = WD * (cur_lr / LR)

            tloss    += loss.item()
            preds     = logits.argmax(1)
            tcorrect += (preds == y).sum().item()
            ttotal   += y.size(0)

        train_loss = tloss / len(train_loader)
        train_acc  = tcorrect / ttotal

        # — validate —
        model.eval()
        vloss = vcorrect = vtotal = 0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                loss   = criterion(logits, y)
                vloss += loss.item()

                preds    = logits.argmax(1)
                vcorrect += (preds == y).sum().item()
                vtotal   += y.size(0)

                all_preds.append(preds.cpu().numpy())
                all_labels.append(y.cpu().numpy())

        val_loss = vloss / len(val_loader)
        val_preds  = np.concatenate(all_preds)
        val_labels = np.concatenate(all_labels)
        val_acc    = (val_preds == val_labels).mean()
        precision  = precision_score(val_labels, val_preds, zero_division=0)
        recall     = recall_score(val_labels, val_preds, zero_division=0)
        f1         = f1_score(val_labels, val_preds, zero_division=0)
        tn, fp, fn, tp = confusion_matrix(val_labels, val_preds).ravel()
        specificity   = tn / (tn + fp)

        elapsed = time.time() - t0

        # print to terminal
        print(
            f"Epoch {epoch:03d} | "
            f"train_loss={train_loss:.4f} acc={train_acc:.4f} | "
            f"val_loss={val_loss:.4f} acc={val_acc:.4f} | "
            f"prec={precision:.4f} rec={recall:.4f} spec={specificity:.4f} f1={f1:.4f} | "
            f"time={elapsed:.1f}s"
        )

        # log to W&B
        wandb.log({
            'train_loss':     train_loss,
            'train_accuracy': train_acc,
            'val_loss':       val_loss,
            'val_accuracy':   val_acc,
            'precision':      precision,
            'recall':         recall,
            'specificity':    specificity,
            'f1_score':       f1
        }, step=epoch)

        # early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            es_count  = 0
        else:
            es_count += 1
            if es_count >= ES_PATIENCE:
                print(f"Early stopping at epoch {epoch}")
                break

    # collect best fold metrics
    fold_metrics['loss'].append(best_loss)
    fold_metrics['acc'].append(val_acc)
    fold_metrics['prec'].append(precision)
    fold_metrics['rec'].append(recall)
    fold_metrics['spec'].append(specificity)
    fold_metrics['f1'].append(f1)

    wandb.finish()
    torch.cuda.empty_cache()

# ─── Final 5-Fold Summary ────────────────────────────────────────
print("\n=== 5-Fold Cross-Validation Averages ===")
for key, vals in fold_metrics.items():
    print(f"{key:4s} : {np.mean(vals):.4f} ± {np.std(vals):.4f}")


Attempting to create new mne-python configuration file:
/root/.mne/mne-python.json
Now using CUDA device 0
Enabling CUDA with 39.14 GiB available memory

=== Fold 1 ===


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jh8032 (jh8032-new-york-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


...

f1_score,▁▁▁▁▁▁▁▁▁▅▄▅▇▇▇██████████▇██████████████
precision,██▁▁▁▁▁▅▇▇▇▇▇▇▇▇▇▇▇▆▇▇▇▇█▇█▇▇▇▇▇▇▇▇▇▆▇▇▇
recall,▁▁▁▁▁▁▁▁▄▇▅▇▆▇█▇▆█▆▇█▇▇▇▇▆▇█▆▇█▇▇█▇▇▇██▇
specificity,▁█████████▇▇▇▅▆▄▆▇▃▇▆▄▇▆▇▇▅▇▄▇▆▅▆▅▇▆▃▅▆▅
train_accuracy,▁▂▁▁▁▃▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇█▇███▇██████████
train_loss,██████████▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁
val_accuracy,▁▁▁▁▁▁▁▁▁▁▃▄▃▃▆▇▇▇▇██▇█▇█▇▇█████████▇▇█▇
val_loss,████████▇▅▄▃▃▃▄▂▃▂▂▁▂▃▁▂▁▂▁▁▁▁▂▁▂▂▂▃▂▂▂▂
f1_score,0.87473
precision,0.84681
recall,0.90455



=== Fold 5 ===


Epoch 001 | train_loss=0.7344 acc=0.4722 | val_loss=0.6951 acc=0.5000 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=13.0s
Epoch 002 | train_loss=0.7193 acc=0.5096 | val_loss=0.6949 acc=0.5000 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.7s
Epoch 003 | train_loss=0.7254 acc=0.5045 | val_loss=0.6943 acc=0.5000 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.8s
Epoch 004 | train_loss=0.7274 acc=0.4932 | val_loss=0.6936 acc=0.4955 | prec=0.4444 rec=0.0364 spec=0.9545 f1=0.0672 | time=12.8s
Epoch 005 | train_loss=0.7239 acc=0.4949 | val_loss=0.6938 acc=0.5000 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.7s
Epoch 006 | train_loss=0.7112 acc=0.5085 | val_loss=0.6949 acc=0.5000 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.9s
Epoch 007 | train_loss=0.7177 acc=0.5040 | val_loss=0.6948 acc=0.5000 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.9s
Epoch 008 | train_loss=0.7276 acc=0.4887 | val_loss=0.6930 acc=0.5000 | prec=0.0000 rec=0.

f1_score,▁▁▁▂▁▁▁▁▅▇▇▇▆▇▇▇█▇▇▇████▇▇▇█████████████
precision,▁▁▁▄▁▁▁▅▅▅▇██▇█▇███▇▇▇▇█▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▆
recall,▁▁▁▁▁▁▁▄██▅▄▆▆▆▆▆▆▆▆▆▇▆▅▇█▇▇▇▇▇▇▇▇▇▇▇▇▇█
specificity,███████▆▁▁▇█▇▇▇▇▇██▇▇▇▇█▆▄▆▇▇▇▆▇▆▆▇▇▆▆▆▅
train_accuracy,▁▂▂▁▁▂▁▁▂▂▂▂▃▄▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇██████████
train_loss,███████████▇▇▇▆▆▅▅▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁
val_accuracy,▁▁▁▁▁▁▁▁▃▁▄▆▅▇▇▆▇▇▇▇▇▇██▇▇▅▇█▇█▇██▇▇█▇▇█
val_loss,██████████▇▇▇▆▅▄▃▃▃▃▂▂▂▃▂▆▂▁▁▁▂▁▁▂▁▂▂▂▂▃
f1_score,0.81092
precision,0.7099
recall,0.94545



=== 5-Fold Cross-Validation Averages ===
loss : 0.4243 ± 0.1380
acc  : 0.7614 ± 0.1356
prec : 0.7470 ± 0.1551
rec  : 0.6929 ± 0.3386
spec : 0.8302 ± 0.1203
f1   : 0.6768 ± 0.3136


#### Change the LR and WD to 3.5e-5

In [ ]:
import os
import json
import random
import time
import numpy as np
import torch
import torch.nn as nn
import wandb

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from torch.utils.data import DataLoader, Subset, Dataset

from eeg_dataset import EEGDataset
from model_optimized_6 import EEGformer

# ─── Configuration ───────────────────────────────────────────────
DATA_DIR    = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE  = 'labels.json'
DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MAX_EPOCHS  = 150
BATCH_SIZE  = 32
NUM_WORKERS = 4
ES_PATIENCE = 15

# ─── Fixed Hyper-parameters ───────────────────────────────────────
LR          = 3.5e-5
WD          = 3.5e-5
PCT_START   = 0.20
NUM_FILTERS = 120

# ─── Dataset wrapper ─────────────────────────────────────────────
class BinaryEEGDataset(Dataset):
    def __init__(self, raw_ds, metas):
        self.raw_ds = raw_ds
        self.metas  = metas
    def __len__(self):
        return len(self.raw_ds)
    def __getitem__(self, idx):
        x, _ = self.raw_ds[idx]
        y     = self.metas[idx]['label']
        return x, y

# ─── Load & balance metadata ─────────────────────────────────────
with open(os.path.join(DATA_DIR, LABEL_FILE), 'r') as f:
    all_meta = json.load(f)
train_meta = [d for d in all_meta if d['type']=='train']
class0, class1 = 'A','C'

data0 = [d for d in train_meta if d['label']==class0]
data1 = [d for d in train_meta if d['label']==class1]
min_count = min(len(data0), len(data1))

balanced_meta = random.sample(data0, min_count) + random.sample(data1, min_count)
random.shuffle(balanced_meta)

# map to 0/1
label_map = {class0:0, class1:1}
for d in balanced_meta:
    d['label'] = label_map[d['label']]

# build dataset & labels array
raw_ds  = EEGDataset(DATA_DIR, balanced_meta)
dataset = BinaryEEGDataset(raw_ds, balanced_meta)
labels  = np.array([d['label'] for d in balanced_meta])

# ─── 5-Fold Cross-Validation ─────────────────────────────────────
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# store per-fold metrics
fold_metrics = {'loss':[], 'acc':[], 'prec':[], 'rec':[], 'spec':[], 'f1':[]}

for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(labels)), labels), start=1):
    print(f"\n=== Fold {fold} ===")
    wandb.init(
        project="eeg-5fold-fixed-hp-2",
        name=f"fold{fold}",
        config={
            'lr': LR, 'wd': WD,
            'pct_start': PCT_START,
            'filters': NUM_FILTERS,
            'max_epochs': MAX_EPOCHS,
            'batch_size': BATCH_SIZE
        },
        reinit=True
    )

    train_loader = DataLoader(
        Subset(dataset, train_idx),
        batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS
    )
    val_loader = DataLoader(
        Subset(dataset, val_idx),
        batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
    )

    # build model, optimizer, scheduler, criterion
    input_len = dataset[0][0].shape[-1]
    model = EEGformer(
        in_channels=19,
        input_length=input_len,
        kernel_size=10,
        num_filters=NUM_FILTERS,
        num_heads=3,
        num_blocks=1,
        num_segments=5,
        num_classes=2
    ).to(DEVICE)

    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=LR,
        total_steps=MAX_EPOCHS * len(train_loader),
        pct_start=PCT_START,
        anneal_strategy='cos',
        cycle_momentum=False
    )
    criterion = nn.CrossEntropyLoss()

    best_loss = float('inf')
    es_count  = 0

    # ─── Epoch Loop ───────────────────────────────────────────────
    for epoch in range(1, MAX_EPOCHS+1):
        t0 = time.time()

        # — train —
        model.train()
        tloss = tcorrect = ttotal = 0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()
            scheduler.step()
            # dynamic weight decay
            cur_lr = optimizer.param_groups[0]['lr']
            for g in optimizer.param_groups:
                g['weight_decay'] = WD * (cur_lr / LR)

            tloss    += loss.item()
            preds     = logits.argmax(1)
            tcorrect += (preds == y).sum().item()
            ttotal   += y.size(0)

        train_loss = tloss / len(train_loader)
        train_acc  = tcorrect / ttotal

        # — validate —
        model.eval()
        vloss = vcorrect = vtotal = 0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                loss   = criterion(logits, y)
                vloss += loss.item()

                preds    = logits.argmax(1)
                vcorrect += (preds == y).sum().item()
                vtotal   += y.size(0)

                all_preds.append(preds.cpu().numpy())
                all_labels.append(y.cpu().numpy())

        val_loss = vloss / len(val_loader)
        val_preds  = np.concatenate(all_preds)
        val_labels = np.concatenate(all_labels)
        val_acc    = (val_preds == val_labels).mean()
        precision  = precision_score(val_labels, val_preds, zero_division=0)
        recall     = recall_score(val_labels, val_preds, zero_division=0)
        f1         = f1_score(val_labels, val_preds, zero_division=0)
        tn, fp, fn, tp = confusion_matrix(val_labels, val_preds).ravel()
        specificity   = tn / (tn + fp)

        elapsed = time.time() - t0

        # print to terminal
        print(
            f"Epoch {epoch:03d} | "
            f"train_loss={train_loss:.4f} acc={train_acc:.4f} | "
            f"val_loss={val_loss:.4f} acc={val_acc:.4f} | "
            f"prec={precision:.4f} rec={recall:.4f} spec={specificity:.4f} f1={f1:.4f} | "
            f"time={elapsed:.1f}s"
        )

        # log to W&B
        wandb.log({
            'train_loss':     train_loss,
            'train_accuracy': train_acc,
            'val_loss':       val_loss,
            'val_accuracy':   val_acc,
            'precision':      precision,
            'recall':         recall,
            'specificity':    specificity,
            'f1_score':       f1
        }, step=epoch)

        # early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            es_count  = 0
        else:
            es_count += 1
            if es_count >= ES_PATIENCE:
                print(f"Early stopping at epoch {epoch}")
                break

    # collect best fold metrics
    fold_metrics['loss'].append(best_loss)
    fold_metrics['acc'].append(val_acc)
    fold_metrics['prec'].append(precision)
    fold_metrics['rec'].append(recall)
    fold_metrics['spec'].append(specificity)
    fold_metrics['f1'].append(f1)

    wandb.finish()
    torch.cuda.empty_cache()

# ─── Final 5-Fold Summary ────────────────────────────────────────
print("\n=== 5-Fold Cross-Validation Averages ===")
for key, vals in fold_metrics.items():
    print(f"{key:4s} : {np.mean(vals):.4f} ± {np.std(vals):.4f}")



=== Fold 1 ===


Epoch 001 | train_loss=0.7644 acc=0.4952 | val_loss=0.6976 acc=0.4989 | prec=0.4989 rec=1.0000 spec=0.0000 f1=0.6657 | time=72.6s
Epoch 002 | train_loss=0.7554 acc=0.4980 | val_loss=0.6944 acc=0.4807 | prec=0.4830 rec=0.5818 spec=0.3801 f1=0.5278 | time=12.8s
Epoch 003 | train_loss=0.7457 acc=0.4997 | val_loss=0.6950 acc=0.4875 | prec=0.4211 rec=0.0727 spec=0.9005 f1=0.1240 | time=12.9s
Epoch 004 | train_loss=0.7382 acc=0.5020 | val_loss=0.6945 acc=0.4921 | prec=0.4857 rec=0.3091 spec=0.6742 f1=0.3778 | time=12.7s
Epoch 005 | train_loss=0.7232 acc=0.4991 | val_loss=0.6956 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.9s
Epoch 006 | train_loss=0.7274 acc=0.5105 | val_loss=0.6966 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.8s
Epoch 007 | train_loss=0.7163 acc=0.5065 | val_loss=0.6990 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.9s
Epoch 008 | train_loss=0.7173 acc=0.4912 | val_loss=0.7001 acc=0.5011 | prec=0.0000 rec=0.

f1_score,▆▅▂▁▁▁▁▁▂▇▇█████████████████████████████
precision,▅▅▄▁▁▁▁█▅▇▇▆▇▇███▇▇▇█▇▇██████▇█████████▇
recall,█▂▁▁▁█▃▄▅▄▆▇▇▆▇▇▇▇▇▇▇▇▆▇▇▆▆▇▇▆▇▆▇▇▇▇▇▆▇▆
specificity,▁▄▇████▇▇▇▇▅▇▆▇█▆▇▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
train_accuracy,▁▁▁▁▁▁▁▁▁▂▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇█████▇██
train_loss,███████▇▆▆▅▅▅▄▄▄▄▄▄▄▃▃▂▂▃▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁
val_accuracy,▁▁▁▁▁▁▁▂▂▅▆▇▇▇▇█████████▇███████████████
val_loss,███████▇▇▆▅▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁
f1_score,0.81863
precision,0.8883
recall,0.75909



=== Fold 2 ===


Epoch 001 | train_loss=0.7757 acc=0.4793 | val_loss=0.7227 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.9s
Epoch 002 | train_loss=0.7323 acc=0.5082 | val_loss=0.7067 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.9s
Epoch 003 | train_loss=0.7354 acc=0.5020 | val_loss=0.7065 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.8s
Epoch 004 | train_loss=0.7393 acc=0.4765 | val_loss=0.7032 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.8s
Epoch 005 | train_loss=0.7136 acc=0.5048 | val_loss=0.6993 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.7s
Epoch 006 | train_loss=0.7143 acc=0.5060 | val_loss=0.7004 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=13.0s
Epoch 007 | train_loss=0.7160 acc=0.4844 | val_loss=0.6967 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.9s
Epoch 008 | train_loss=0.7206 acc=0.4935 | val_loss=0.6977 acc=0.5011 | prec=0.0000 rec=0.

...

f1_score,▁▁▁▁▁▁▆▁▁▁▃▁▅▇▂▆▇█▇▇████████████████████
precision,▁▁▁▁▅▁▅▁▁▁▅▅█▆▆▆▆▇▇▆▇▇▇▇▇▆▇▆▇▇▇▇▇▇▇▇▆▇▇▆
recall,▁▁▁▁▁▁▇▁▁▁▂▁▄█▁▇▇▇▆▅▆▆▆▆▇▆▇▇▇▇▆▇▆▆▇▇▆▇▆▇
specificity,███████▁████▇▆▃▄▇▇▅▆▆▇▇▆▆▇▆▆▇▇▆▇▆▆▇▆▆▆▆▆
train_accuracy,▁▂▂▁▁▂▁▁▁▁▂▂▂▂▃▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇█▇▇█▇█████
train_loss,████▇█▇▇▇▇▇▇▇▇▇▆▆▆▆▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁
val_accuracy,▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▅▇▇▆▇▇▇▇██▇▇█████▇█▇▇▇▇▇▇
val_loss,█████▇▇▇▇▇▇▇▇▇▇▆▅▄▄▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▂▂
f1_score,0.80088
precision,0.78355
recall,0.819



=== Fold 5 ===


Epoch 001 | train_loss=0.7132 acc=0.4875 | val_loss=0.6936 acc=0.5000 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.9s
Epoch 002 | train_loss=0.7074 acc=0.5142 | val_loss=0.6947 acc=0.5000 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=13.0s
Epoch 003 | train_loss=0.7164 acc=0.4830 | val_loss=0.6939 acc=0.5000 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.9s
Epoch 004 | train_loss=0.7066 acc=0.4938 | val_loss=0.6928 acc=0.5159 | prec=1.0000 rec=0.0318 spec=1.0000 f1=0.0617 | time=13.0s
Epoch 005 | train_loss=0.7019 acc=0.5283 | val_loss=0.6943 acc=0.5000 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.9s
Epoch 006 | train_loss=0.7007 acc=0.5181 | val_loss=0.6956 acc=0.5000 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=13.0s
Epoch 007 | train_loss=0.7118 acc=0.4915 | val_loss=0.6960 acc=0.5000 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.9s
Epoch 008 | train_loss=0.7041 acc=0.5068 | val_loss=0.6931 acc=0.5000 | prec=0.0000 rec=0.

f1_score,▁▁▁▂▁▁▁▁▁▃▁▁▂▂▇██████████▇██████▇███████
precision,▁▁▁█▁▁▁▁▁▁▁▁██▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██▇▇█▇▇
recall,▁▁▁▁▁▁▁▁▁▁▂▁▁▁▆▇▇██▇█▇▇▇█▆█▇██▇▇▇▆▆▇▇▇▇▇
specificity,█████████▆████▃▄▄▄▂▃▅▅▃▅▆▄▅▇▄▆▃▆▁▆▅▇▆▆▆▆
train_accuracy,▁▂▁▁▂▁▁▁▂▁▂▂▂▄▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇█▇███████
train_loss,██████████████▇▅▅▅▅▄▄▃▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁
val_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▂▁▆▇▇██▇█████████████▇██████
val_loss,████████████▅▄▃▃▂▂▂▂▂▂▂▁▂▁▂▂▁▁▁▂▁▂▃▁▂▃▃▂
f1_score,0.80905
precision,0.90449
recall,0.73182



=== 5-Fold Cross-Validation Averages ===
loss : 0.3935 ± 0.0322
acc  : 0.8172 ± 0.0207
prec : 0.8597 ± 0.0527
rec  : 0.7640 ± 0.0291
spec : 0.8702 ± 0.0615
f1   : 0.8073 ± 0.0157


#### Next Step (AD vs CN)
- 1] Fix the Block = 1, Head = 3, and Filter Size according to the best trial
- 2] Try 5 Fold Cross Validation to see how the model generalize well
- 3] Fix the parameter and train using the entire train dataset. Don't forget to save the model. Because we want to see the test performance.
- 4] Testing the performance using test-within and test-cross


#### Next Step (FTD vs CN)
- 1] Optuna Search using the Block = 1, Head = 3, and Filter Size that we found out in AD vs CN result. (setting the same search space as we did on AD vs CN)
- 2] Try 5 Fold Cross Validation to see how the model generalize well
- 3] Fix the parameter and train using the entire train dataset. Save the model.
- 4] Testing the performance using test-within and test-cross

#### Next Step (AD vs FTD)
- 1] Optuna Search same as before
- 2] Try 5 Fold Cross Validation same as before. Using the fixed parameters.
- 3] Fix the parameter and train using the entire train dataset. Save the model.
- 4] Testing the performance using test-within and test-cross

#### AD vs CN Model Performance ('test_within' and 'test_cross')
- Choose the best hyperparameter in 10 trials
- Use the epoch stored in best trial
- Train the model again using the entire train data
- Evaluate the model in 'test_within' and 'test_cross'

In [ ]:
import os
import json
import random
import time
import copy

import numpy as np
import torch
import torch.nn as nn
import wandb
import optuna

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from torch.utils.data import DataLoader, Subset, Dataset

from eeg_dataset import EEGDataset
from model_optimized_6 import EEGformer

# ─── Reproducibility ────────────────────────────────────────────
SEED = 0
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark     = False

# ─── Configuration ───────────────────────────────────────────────
DATA_DIR    = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE  = 'labels.json'
DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MAX_EPOCHS  = 150
BATCH_SIZE  = 32
NUM_WORKERS = 4
ES_PATIENCE = 15

# ─── Fixed Hyper-parameters ───────────────────────────────────────
PCT_START   = 0.2  # fixed

# ─── Dataset wrapper ─────────────────────────────────────────────
class BinaryEEGDataset(Dataset):
    def __init__(self, raw_ds, metas):
        self.raw_ds = raw_ds
        self.metas  = metas
    def __len__(self):
        return len(self.raw_ds)
    def __getitem__(self, idx):
        x, _ = self.raw_ds[idx]
        y     = self.metas[idx]['label']
        return x, y

# ─── Load & count splits ──────────────────────────────────────────
with open(os.path.join(DATA_DIR, LABEL_FILE), 'r') as f:
    all_meta = json.load(f)

train_meta       = [d for d in all_meta if d['type']=='train']
test_within_meta = [d for d in all_meta if d['type']=='test_within']
test_cross_meta  = [d for d in all_meta if d['type']=='test_cross']

class0, class1 = 'A','C'
label_map = {class0:0, class1:1}

# A vs C 만 필터링
test_within_meta = [d for d in test_within_meta if d['label'] in (class0, class1)]
test_cross_meta  = [d for d in test_cross_meta  if d['label'] in (class0, class1)]

def count_labels(meta_list):
    cnt0 = cnt1 = 0
    for d in meta_list:
        lbl = d['label']
        if isinstance(lbl, str):
            if lbl == class0: cnt0 += 1
            elif lbl == class1: cnt1 += 1
        else:
            if lbl == 0: cnt0 += 1
            elif lbl == 1: cnt1 += 1
    return cnt0, cnt1

n_tr0, n_tr1 = count_labels(train_meta)
n_tw0, n_tw1 = count_labels(test_within_meta)
n_tc0, n_tc1 = count_labels(test_cross_meta)

print(f"--> Data counts before balancing:")
print(f"    TRAIN         total={len(train_meta)}  A(D)={n_tr0}, C(N)={n_tr1}")
print(f"    TEST_WITHIN   total={len(test_within_meta)}  A(D)={n_tw0}, C(N)={n_tw1}")
print(f"    TEST_CROSS    total={len(test_cross_meta)}  A(D)={n_tc0}, C(N)={n_tc1}\n")

# ─── Balance train set ───────────────────────────────────────────
data0 = [d for d in train_meta if d['label']==class0]
data1 = [d for d in train_meta if d['label']==class1]
min_count = min(len(data0), len(data1))

balanced_meta = random.sample(data0, min_count) + random.sample(data1, min_count)
balanced_meta = [copy.deepcopy(d) for d in balanced_meta]
random.shuffle(balanced_meta)

for d in balanced_meta:
    d['label'] = label_map[d['label']]

raw_ds_train  = EEGDataset(DATA_DIR, balanced_meta)
dataset_train = BinaryEEGDataset(raw_ds_train, balanced_meta)
labels_train  = np.array([d['label'] for d in balanced_meta])

# ─── Optuna Objective ────────────────────────────────────────────
def objective(trial):
    # 1) sample hyperparameters
    lr = trial.suggest_float('lr', 1e-5, 1e-4, log=True)
    wd = trial.suggest_float('wd', 1e-6, 1e-4, log=True)

    print(f"\n--- Trial {trial.number} hyperparams --> lr={lr:.2e}, wd={wd:.2e}")

    wandb.init(
        project='eeg-optuna-AD-CN-test-within-cross-2',
        name=f"trial{trial.number}",
        config={'lr':lr, 'wd':wd, 'pct_start':PCT_START},
        reinit=True
    )

    # train/validation split
    idx = np.arange(len(dataset_train))
    tr_idx, va_idx = train_test_split(
        idx, test_size=0.2,
        stratify=labels_train, random_state=SEED
    )
    train_loader = DataLoader(
        Subset(dataset_train, tr_idx),
        batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS
    )
    val_loader   = DataLoader(
        Subset(dataset_train, va_idx),
        batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
    )

    # model & optimizer & scheduler & loss
    input_len = dataset_train[0][0].shape[-1]
    model = EEGformer(
        in_channels=19, input_length=input_len,
        kernel_size=10, num_filters=120,
        num_heads=3, num_blocks=1,
        num_segments=5, num_classes=2
    ).to(DEVICE)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=lr, epochs=MAX_EPOCHS,
        steps_per_epoch=len(train_loader), pct_start=PCT_START,
        anneal_strategy='cos', cycle_momentum=False
    )
    criterion = nn.CrossEntropyLoss()

    best_val_loss = float('inf')
    best_val_acc  = 0.0
    es_count      = 0

    for epoch in range(1, MAX_EPOCHS+1):
        t0 = time.time()
        # ── train ──
        model.train()
        train_loss_sum = train_correct = train_total = 0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()
            scheduler.step()
            # dynamic weight_decay
            cur_lr = optimizer.param_groups[0]['lr']
            for g in optimizer.param_groups:
                g['weight_decay'] = wd * (cur_lr / lr)
            train_loss_sum += loss.item()
            preds = logits.argmax(1)
            train_correct += (preds == y).sum().item()
            train_total   += y.size(0)
        train_loss = train_loss_sum / len(train_loader)
        train_acc  = train_correct / train_total

        # ── validate ──
        model.eval()
        vloss = 0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                vloss += criterion(logits, y).item()
                preds = logits.argmax(1)
                all_preds.append(preds.cpu().numpy())
                all_labels.append(y.cpu().numpy())
        val_loss = vloss / len(val_loader)

        preds = np.concatenate(all_preds)
        labs  = np.concatenate(all_labels)
        val_acc = (preds == labs).sum() / labs.size

        # ── metrics ──
        tn, fp, fn, tp = confusion_matrix(labs, preds, labels=[0,1]).ravel()
        spec = tn / (tn+fp) if (tn+fp)>0 else 0.0
        prec = precision_score(labs, preds, zero_division=0)
        rec  = recall_score(labs, preds, zero_division=0)
        f1   = f1_score(labs, preds, zero_division=0)
        elapsed = time.time() - t0

        print(
            f"Epoch {epoch:03d} | "
            f"train_loss={train_loss:.4f} acc={train_acc:.4f} | "
            f"val_loss={val_loss:.4f} acc={val_acc:.4f} | "
            f"prec={prec:.4f} rec={rec:.4f} spec={spec:.4f} f1={f1:.4f} | "
            f"time={elapsed:.1f}s"
        )

        wandb.log({
            'epoch':       epoch,
            'train_loss':  train_loss,
            'train_acc':   train_acc,
            'val_loss':    val_loss,
            'val_acc':     val_acc,
            'specificity': spec,
            'precision':   prec,
            'recall':      rec,
            'f1_score':    f1
        })

        # ── Early Stopping (val_loss 기준) ──
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_val_acc  = val_acc
            es_count      = 0
            trial.set_user_attr('best_epoch', epoch)
        else:
            es_count += 1
            if es_count >= ES_PATIENCE:
                break

    wandb.finish()
    del model, optimizer, train_loader, val_loader, scheduler
    torch.cuda.empty_cache()

    # 다중목적 리턴
    return best_val_loss, best_val_acc

# ─── Run Optuna Study ────────────────────────────────────────────
if __name__ == "__main__":
    study = optuna.create_study(
        directions=['minimize', 'maximize'],
        study_name="eeg_multiobj"
    )
    study.optimize(objective, n_trials=10)

    # Pareto front 중에서 val_acc가 가장 높은 trial 선택
    best = max(study.best_trials, key=lambda t: t.values[1])
    bv_loss, bv_acc = best.values
    best_epoch      = best.user_attrs['best_epoch']
    print(f"\n=== Selected Trial #{best.number} ===")
    print(
        f" val_loss={bv_loss:.4f}, val_acc={bv_acc:.4f}, "
        f"params={best.params}, best_epoch={best_epoch}"
    )

    # ─── Final Retrain on FULL BALANCED TRAIN SET ─────────────────────
    final_loader = DataLoader(
        dataset_train,
        batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS
    )
    input_len = dataset_train[0][0].shape[-1]
    final_model = EEGformer(
        in_channels=19, input_length=input_len,
        kernel_size=10, num_filters=120,
        num_heads=3, num_blocks=1,
        num_segments=5, num_classes=2
    ).to(DEVICE)
    optimizer = torch.optim.AdamW(
        final_model.parameters(),
        lr=best.params['lr'], weight_decay=best.params['wd']
    )
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=best.params['lr'],
        epochs=best_epoch, steps_per_epoch=len(final_loader),
        pct_start=PCT_START, anneal_strategy='cos',
        cycle_momentum=False
    )
    criterion = nn.CrossEntropyLoss()

    for epoch in range(1, best_epoch+1):
        final_model.train()
        for X, y in final_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = final_model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()
            scheduler.step()

    os.makedirs('models', exist_ok=True)
    torch.save(final_model.state_dict(), 'models/eeg_best_AD_CN_1.pth')
    print(f"Saved final model to models/eeg_best_AD_CN_1.pth\n")

    # ─── Evaluation 함수 ────────────────────────────────────────────
    def evaluate(model, metas):
        metas_copy = copy.deepcopy(metas)
        for d in metas_copy:
            if isinstance(d['label'], str):
                d['label'] = label_map[d['label']]

        ds     = BinaryEEGDataset(EEGDataset(DATA_DIR, metas_copy), metas_copy)
        loader = DataLoader(ds, batch_size=BATCH_SIZE,
                            shuffle=False, num_workers=NUM_WORKERS)

        model.eval()
        vloss=vcorrect=vtotal=0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for X, y in loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                vloss    += criterion(logits, y).item()
                preds     = logits.argmax(1)
                vcorrect += (preds==y).sum().item()
                vtotal   += y.size(0)
                all_preds.append(preds.cpu().numpy())
                all_labels.append(y.cpu().numpy())

        preds = np.concatenate(all_preds)
        labs  = np.concatenate(all_labels)
        tn, fp, fn, tp = confusion_matrix(labs, preds, labels=[0,1]).ravel()
        return {
            'loss': vloss/len(loader),
            'acc':  vcorrect/vtotal,
            'sensitivity': recall_score(labs, preds, zero_division=0),
            'specificity': tn/(tn+fp) if (tn+fp)>0 else 0.0,
            'f1':   f1_score(labs, preds, zero_division=0)
        }

    # ─── Final Evaluation on Test Sets ───────────────────────────────
    print("=== Final Evaluation on Test Sets ===")
    for name, metas in [('test_within', test_within_meta), ('test_cross', test_cross_meta)]:
        res = evaluate(final_model, metas)
        n0, n1 = count_labels(metas)
        print(f"-- {name} -- total={len(metas)}  A(D)={n0}, C(N)={n1}")
        print(
            f" Accuracy={res['acc']:.4f} "
            f"Sensitivity={res['sensitivity']:.4f} "
            f"Specificity={res['specificity']:.4f} "
            f"F1={res['f1']:.4f}\n"
        )

Now using CUDA device 0
Enabling CUDA with 39.14 GiB available memory


[I 2025-05-07 00:59:58,286] A new study created in memory with name: eeg_multiobj


--> Data counts before balancing:
    TRAIN         total=3219  A(D)=1388, C(N)=1102
    TEST_WITHIN   total=272  A(D)=146, C(N)=126
    TEST_CROSS    total=626  A(D)=319, C(N)=307


--- Trial 0 hyperparams --> lr=2.75e-05, wd=8.57e-05


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Epoch 001 | train_loss=0.7493 acc=0.5201 | val_loss=0.6940 acc=0.4785 | prec=0.4728 rec=0.3955 spec=0.5611 f1=0.4307 | time=13.7s
Epoch 002 | train_loss=0.7472 acc=0.4969 | val_loss=0.6935 acc=0.4807 | prec=0.4876 rec=0.8045 spec=0.1584 f1=0.6072 | time=13.1s
Epoch 003 | train_loss=0.7550 acc=0.4765 | val_loss=0.6937 acc=0.4490 | prec=0.4633 rec=0.6591 spec=0.2398 f1=0.5441 | time=13.2s
Epoch 004 | train_loss=0.7532 acc=0.4935 | val_loss=0.6939 acc=0.4830 | prec=0.4888 rec=0.7955 spec=0.1719 f1=0.6055 | time=13.1s
Epoch 005 | train_loss=0.7400 acc=0.4963 | val_loss=0.6947 acc=0.4966 | prec=0.4977 rec=0.9955 spec=0.0000 f1=0.6636 | time=13.1s
Epoch 006 | train_loss=0.7454 acc=0.4844 | val_loss=0.6939 acc=0.4989 | prec=0.4989 rec=1.0000 spec=0.0000 f1=0.6657 | time=13.0s
Epoch 007 | train_loss=0.7349 acc=0.4929 | val_loss=0.6943 acc=0.4966 | prec=0.4977 rec=0.9955 spec=0.0000 f1=0.6636 | time=13.0s
Epoch 008 | train_loss=0.7243 acc=0.5111 | val_loss=0.6940 acc=0.4989 | prec=0.4989 rec=0.

...


--- Trial 9 hyperparams --> lr=4.26e-05, wd=6.14e-06


Epoch 001 | train_loss=0.7261 acc=0.5139 | val_loss=0.6958 acc=0.4989 | prec=0.4989 rec=1.0000 spec=0.0000 f1=0.6657 | time=13.0s
Epoch 002 | train_loss=0.7296 acc=0.5077 | val_loss=0.6941 acc=0.4966 | prec=0.4977 rec=0.9909 spec=0.0045 f1=0.6626 | time=13.0s
Epoch 003 | train_loss=0.7331 acc=0.5167 | val_loss=0.6950 acc=0.4989 | prec=0.4989 rec=1.0000 spec=0.0000 f1=0.6657 | time=13.1s
Epoch 004 | train_loss=0.7407 acc=0.4884 | val_loss=0.6938 acc=0.4989 | prec=0.4989 rec=0.9864 spec=0.0136 f1=0.6626 | time=13.0s
Epoch 005 | train_loss=0.7183 acc=0.5162 | val_loss=0.6948 acc=0.4989 | prec=0.4989 rec=1.0000 spec=0.0000 f1=0.6657 | time=12.9s
Epoch 006 | train_loss=0.7221 acc=0.4946 | val_loss=0.6933 acc=0.4921 | prec=0.4954 rec=0.9864 spec=0.0000 f1=0.6596 | time=12.8s
Epoch 007 | train_loss=0.7251 acc=0.5082 | val_loss=0.6931 acc=0.5283 | prec=0.5208 rec=0.6818 spec=0.3756 f1=0.5906 | time=13.0s
Epoch 008 | train_loss=0.7173 acc=0.5150 | val_loss=0.6926 acc=0.5238 | prec=0.5373 rec=0.

epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇██
f1_score,▆▆▆▆▅▁▆▇▇▇█▇███▇█████▇███▇██▇█████▇█▇▇██
precision,▁▁▁▁▁▃▁▅▃▅▇▆▆▇▇▇▇▄▇▇▇▆▇▇▇▇▇█▇█▇▇▇▆█▇▇█▇█
recall,█████▁█▅▃▇▅█▆▆▇▆▇▆▇▇▆▆▇▇▆▇▅▆▅▆▇▇▆▆▇▅▆▆▆▇
specificity,▁▁▁▆██▅▆▇▄▇▇▇▇▇▇▅▇▇▇▇▇▇▇▇▇████▇▇▇▇████▇▇
train_acc,▁▁▁▁▁▂▂▃▄▄▆▆▆▅▆▆▇▇▇▇▇▇▇▇▇████▇██████████
train_loss,██████▇▆▆▅▅▄▄▄▄▃▃▃▃▃▂▃▂▃▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁
val_acc,▁▁▁▁▁▂▁▅▆▆▇▇▇▇▇▇▇▇██▇▇▇▇▇▆▇▆▇▇█▇▇▇▇▇▇▇█▇
val_loss,████████▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▄▁▂▃▂▁▁▁▁▃▂▄▃▂▂
epoch,68
f1_score,0.82135


[I 2025-05-07 03:58:16,100] Trial 9 finished with values: [0.34045001970870153, 0.8526077097505669] and parameters: {'lr': 4.256281022561698e-05, 'wd': 6.137703745483915e-06}.



=== Selected Trial #7 ===
 val_loss=0.3224, val_acc=0.8594, params={'lr': 2.1990910603368945e-05, 'wd': 2.299085583693177e-06}, best_epoch=85
Saved final model to models/eeg_best_AD_CN_1.pth

=== Final Evaluation on Test Sets ===
-- test_within -- total=272  A(D)=146, C(N)=126
 Accuracy=0.8272 Sensitivity=0.9206 Specificity=0.7466 F1=0.8315

-- test_cross -- total=626  A(D)=319, C(N)=307
 Accuracy=0.6534 Sensitivity=0.8306 Specificity=0.4828 F1=0.7015



#### Without Full Train dataset
- Remove the full train (train + val) process
- Instead of using the model at early stopping epoch, we are going to use the best performance model (lowest validation loss) before early stopping


In [ ]:
import os
import json
import random
import time
import copy

import numpy as np
import torch
import torch.nn as nn
import wandb
import optuna

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from torch.utils.data import DataLoader, Subset, Dataset

from eeg_dataset import EEGDataset
from model_optimized_6 import EEGformer

# ─── Reproducibility ────────────────────────────────────────────
SEED = 0
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark     = False

# ─── Configuration ───────────────────────────────────────────────
DATA_DIR    = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE  = 'labels.json'
DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MAX_EPOCHS  = 150
BATCH_SIZE  = 32
NUM_WORKERS = 4
ES_PATIENCE = 15

# ─── Fixed Hyper-parameters ───────────────────────────────────────
PCT_START   = 0.2  # fixed

# ─── Dataset wrapper ─────────────────────────────────────────────
class BinaryEEGDataset(Dataset):
    def __init__(self, raw_ds, metas):
        self.raw_ds = raw_ds
        self.metas  = metas
    def __len__(self):
        return len(self.raw_ds)
    def __getitem__(self, idx):
        x, _ = self.raw_ds[idx]
        y     = self.metas[idx]['label']
        return x, y

# ─── Load & count splits ──────────────────────────────────────────
with open(os.path.join(DATA_DIR, LABEL_FILE), 'r') as f:
    all_meta = json.load(f)

train_meta       = [d for d in all_meta if d['type']=='train']
test_within_meta = [d for d in all_meta if d['type']=='test_within']
test_cross_meta  = [d for d in all_meta if d['type']=='test_cross']

class0, class1 = 'A','C'
label_map = {class0:0, class1:1}

# A vs C Filtering
test_within_meta = [d for d in test_within_meta if d['label'] in (class0, class1)]
test_cross_meta  = [d for d in test_cross_meta  if d['label'] in (class0, class1)]

def count_labels(meta_list):
    cnt0 = cnt1 = 0
    for d in meta_list:
        lbl = d['label']
        if isinstance(lbl, str):
            if lbl == class0: cnt0 += 1
            elif lbl == class1: cnt1 += 1
        else:
            if lbl == 0: cnt0 += 1
            elif lbl == 1: cnt1 += 1
    return cnt0, cnt1

n_tr0, n_tr1 = count_labels(train_meta)
n_tw0, n_tw1 = count_labels(test_within_meta)
n_tc0, n_tc1 = count_labels(test_cross_meta)

print(f"--> Data counts before balancing:")
print(f"    TRAIN         total={len(train_meta)}  A(D)={n_tr0}, C(N)={n_tr1}")
print(f"    TEST_WITHIN   total={len(test_within_meta)}  A(D)={n_tw0}, C(N)={n_tw1}")
print(f"    TEST_CROSS    total={len(test_cross_meta)}  A(D)={n_tc0}, C(N)={n_tc1}\n")

# ─── Balance train set ───────────────────────────────────────────
data0 = [d for d in train_meta if d['label']==class0]
data1 = [d for d in train_meta if d['label']==class1]
min_count = min(len(data0), len(data1))

balanced_meta = random.sample(data0, min_count) + random.sample(data1, min_count)
balanced_meta = [copy.deepcopy(d) for d in balanced_meta]
random.shuffle(balanced_meta)

# ─── After Balancing ───────────────────────────────────────────
bal_AD, bal_CN = count_labels(balanced_meta)
print(f"[BALANCED TRAIN] total={len(balanced_meta)}  AD={bal_AD}, CN={bal_CN}\n")

for d in balanced_meta:
    d['label'] = label_map[d['label']]

raw_ds_train  = EEGDataset(DATA_DIR, balanced_meta)
dataset_train = BinaryEEGDataset(raw_ds_train, balanced_meta)
labels_train  = np.array([d['label'] for d in balanced_meta])

# ─── Optuna Objective ────────────────────────────────────────────
def objective(trial):
    # 1) sample hyperparameters
    lr = trial.suggest_float('lr', 1e-5, 1e-4, log=True)
    wd = trial.suggest_float('wd', 1e-6, 1e-4, log=True)

    print(f"\n--- Trial {trial.number} hyperparams --> lr={lr:.2e}, wd={wd:.2e}")

    wandb.init(
        project='eeg-optuna-AD-CN-test-within-cross-3',
        name=f"trial{trial.number}",
        config={'lr':lr, 'wd':wd, 'pct_start':PCT_START},
        reinit=True
    )

    # train/validation split
    idx = np.arange(len(dataset_train))
    tr_idx, va_idx = train_test_split(
        idx, test_size=0.2,
        stratify=labels_train, random_state=SEED
    )

    tr_AD = np.sum(labels_train[tr_idx] == 0)
    tr_CN = np.sum(labels_train[tr_idx] == 1)
    va_AD = np.sum(labels_train[va_idx] == 0)
    va_CN = np.sum(labels_train[va_idx] == 1)
    print(f"[Trial {trial.number}]  "
          f"TRAIN n={len(tr_idx)} (AD={tr_AD}, CN={tr_CN}) | "
          f"VAL n={len(va_idx)} (AD={va_AD}, CN={va_CN})")

    train_loader = DataLoader(
        Subset(dataset_train, tr_idx),
        batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS
    )
    val_loader   = DataLoader(
        Subset(dataset_train, va_idx),
        batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
    )

    # model & optimizer & scheduler & loss
    input_len = dataset_train[0][0].shape[-1]
    model = EEGformer(
        in_channels=19, input_length=input_len,
        kernel_size=10, num_filters=120,
        num_heads=3, num_blocks=1,
        num_segments=5, num_classes=2
    ).to(DEVICE)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=lr, epochs=MAX_EPOCHS,
        steps_per_epoch=len(train_loader), pct_start=PCT_START,
        anneal_strategy='cos', cycle_momentum=False
    )
    criterion = nn.CrossEntropyLoss()

    best_val_loss = float('inf')
    best_val_acc  = 0.0
    best_state= None
    es_count      = 0

    for epoch in range(1, MAX_EPOCHS+1):
        t0 = time.time()
        # ── train ──
        model.train()
        train_loss_sum = train_correct = train_total = 0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()
            scheduler.step()
            # dynamic weight_decay
            cur_lr = optimizer.param_groups[0]['lr']
            for g in optimizer.param_groups:
                g['weight_decay'] = wd * (cur_lr / lr)
            train_loss_sum += loss.item()
            preds = logits.argmax(1)
            train_correct += (preds == y).sum().item()
            train_total   += y.size(0)
        train_loss = train_loss_sum / len(train_loader)
        train_acc  = train_correct / train_total

        # ── validate ──
        model.eval()
        vloss = 0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                vloss += criterion(logits, y).item()
                preds = logits.argmax(1)
                all_preds.append(preds.cpu().numpy())
                all_labels.append(y.cpu().numpy())
        val_loss = vloss / len(val_loader)

        preds = np.concatenate(all_preds)
        labs  = np.concatenate(all_labels)
        val_acc = (preds == labs).sum() / labs.size

        # ── metrics ──
        tn, fp, fn, tp = confusion_matrix(labs, preds, labels=[0,1]).ravel()
        spec = tn / (tn+fp) if (tn+fp)>0 else 0.0
        prec = precision_score(labs, preds, zero_division=0)
        rec  = recall_score(labs, preds, zero_division=0)
        f1   = f1_score(labs, preds, zero_division=0)
        elapsed = time.time() - t0

        print(
            f"Epoch {epoch:03d} | "
            f"train_loss={train_loss:.4f} acc={train_acc:.4f} | "
            f"val_loss={val_loss:.4f} acc={val_acc:.4f} | "
            f"prec={prec:.4f} rec={rec:.4f} spec={spec:.4f} f1={f1:.4f} | "
            f"time={elapsed:.1f}s"
        )

        wandb.log({
            'epoch':       epoch,
            'train_loss':  train_loss,
            'train_acc':   train_acc,
            'val_loss':    val_loss,
            'val_acc':     val_acc,
            'specificity': spec,
            'precision':   prec,
            'recall':      rec,
            'f1_score':    f1
        })

        # ── Early Stopping (val_loss 기준) ──
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_val_acc  = val_acc
            es_count      = 0
            best_state    = copy.deepcopy(model.state_dict())

            os.makedirs('ckpts', exist_ok=True)
            ckpt_path = f'ckpts/trial{trial.number}_optimal_AD_CN_1.pth'
            torch.save(best_state, ckpt_path)
            trial.set_user_attr('ckpt_path', ckpt_path)
            trial.set_user_attr('best_epoch', epoch)
        else:
            es_count += 1
            if es_count >= ES_PATIENCE:
                break

    if best_state is not None:
        model.load_state_dict(best_state)

    wandb.finish()
    del model, optimizer, train_loader, val_loader, scheduler
    torch.cuda.empty_cache()

    # 다중목적 리턴
    return best_val_loss, best_val_acc

# ─── Run Optuna Study ────────────────────────────────────────────
if __name__ == "__main__":
    study = optuna.create_study(
        directions=["minimize", "maximize"],
        study_name="eeg_multiobj"
    )
    study.optimize(objective, n_trials=10)

    # Pareto front 중 val_acc 가 가장 높은 trial 선택
    best       = max(study.best_trials, key=lambda t: t.values[1])
    bv_loss, bv_acc = best.values
    best_epoch = best.user_attrs["best_epoch"]
    ckpt_path  = best.user_attrs["ckpt_path"]

    print(f"\n=== Selected Trial #{best.number} ===")
    print(
        f" val_loss={bv_loss:.4f}, val_acc={bv_acc:.4f}, "
        f"params={best.params}, best_epoch={best_epoch}, ckpt={ckpt_path}"
    )

    # ─── Search‑model 로드 ───────────────────────────────────────
    input_len = dataset_train[0][0].shape[-1]
    search_model = EEGformer(
        in_channels=19, input_length=input_len,
        kernel_size=10, num_filters=120,
        num_heads=3, num_blocks=1,
        num_segments=5, num_classes=2
    ).to(DEVICE)

    search_model.load_state_dict(torch.load(ckpt_path, map_location=DEVICE))
    search_model.eval()
    criterion = nn.CrossEntropyLoss()

    # ─── Evaluation 함수 ─────────────────────────────────────────
    def evaluate(model, metas, criterion):
        metas_copy = copy.deepcopy(metas)
        for d in metas_copy:
            if isinstance(d["label"], str):
                d["label"] = label_map[d["label"]]

        ds = BinaryEEGDataset(EEGDataset(DATA_DIR, metas_copy), metas_copy)
        loader = DataLoader(
            ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
        )

        model.eval()
        vloss = vcorrect = vtotal = 0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for X, y in loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                vloss += criterion(logits, y).item()
                preds = logits.argmax(1)
                vcorrect += (preds == y).sum().item()
                vtotal += y.size(0)
                all_preds.append(preds.cpu().numpy())
                all_labels.append(y.cpu().numpy())

        preds = np.concatenate(all_preds)
        labs  = np.concatenate(all_labels)
        tn, fp, fn, tp = confusion_matrix(labs, preds, labels=[0, 1]).ravel()
        return {
            "loss":        vloss / len(loader),
            "acc":         vcorrect / vtotal,
            "sensitivity": recall_score(labs, preds, zero_division=0),
            "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0.0,
            "f1":          f1_score(labs, preds, zero_division=0),
        }

    # ─── Final Evaluation on Test Sets ────────────────────────────
    print("=== Final Evaluation on Test Sets ===")
    for name, metas in [
        ("test_within", test_within_meta),
        ("test_cross",  test_cross_meta),
    ]:
        res = evaluate(search_model, metas, criterion)
        n0, n1 = count_labels(metas)
        print(f"-- {name} -- total={len(metas)}  A(D)={n0}, C(N)={n1}")
        print(
            f" Accuracy={res['acc']:.4f} "
            f"Sensitivity={res['sensitivity']:.4f} "
            f"Specificity={res['specificity']:.4f} "
            f"F1={res['f1']:.4f}\n"
        )

Attempting to create new mne-python configuration file:
/root/.mne/mne-python.json
Now using CUDA device 0
Enabling CUDA with 39.14 GiB available memory


[I 2025-05-07 21:22:50,544] A new study created in memory with name: eeg_multiobj


--> Data counts before balancing:
    TRAIN         total=3219  A(D)=1388, C(N)=1102
    TEST_WITHIN   total=272  A(D)=146, C(N)=126
    TEST_CROSS    total=626  A(D)=319, C(N)=307

[BALANCED TRAIN] total=2204  AD=1102, CN=1102


--- Trial 0 hyperparams --> lr=1.95e-05, wd=1.48e-06


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


[Trial 0]  TRAIN n=1763 (AD=881, CN=882) | VAL n=441 (AD=221, CN=220)
Epoch 001 | train_loss=0.7506 acc=0.5173 | val_loss=0.6942 acc=0.4898 | prec=0.4935 rec=0.8682 spec=0.1131 f1=0.6293 | time=171.5s
Epoch 002 | train_loss=0.7479 acc=0.4929 | val_loss=0.6946 acc=0.5147 | prec=0.5069 rec=0.9955 spec=0.0362 f1=0.6718 | time=13.5s
Epoch 003 | train_loss=0.7577 acc=0.4793 | val_loss=0.6945 acc=0.5011 | prec=0.5000 rec=0.9955 spec=0.0090 f1=0.6657 | time=13.6s
Epoch 004 | train_loss=0.7551 acc=0.5009 | val_loss=0.6942 acc=0.4966 | prec=0.4977 rec=0.9955 spec=0.0000 f1=0.6636 | time=13.5s
Epoch 005 | train_loss=0.7405 acc=0.5014 | val_loss=0.6940 acc=0.4989 | prec=0.4989 rec=1.0000 spec=0.0000 f1=0.6657 | time=13.5s
Epoch 006 | train_loss=0.7480 acc=0.4872 | val_loss=0.6936 acc=0.4989 | prec=0.4989 rec=1.0000 spec=0.0000 f1=0.6657 | time=13.5s
Epoch 007 | train_loss=0.7377 acc=0.4952 | val_loss=0.6942 acc=0.4966 | prec=0.4977 rec=0.9955 spec=0.0000 f1=0.6636 | time=13.6s
Epoch 008 | train_l

...


--- Trial 9 hyperparams --> lr=3.22e-05, wd=6.65e-05


[Trial 9]  TRAIN n=1763 (AD=881, CN=882) | VAL n=441 (AD=221, CN=220)
Epoch 001 | train_loss=0.7172 acc=0.5071 | val_loss=0.6949 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.8s
Epoch 002 | train_loss=0.7255 acc=0.4884 | val_loss=0.6936 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.9s
Epoch 003 | train_loss=0.7231 acc=0.4918 | val_loss=0.6937 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.8s
Epoch 004 | train_loss=0.7081 acc=0.5071 | val_loss=0.6928 acc=0.4989 | prec=0.0000 rec=0.0000 spec=0.9955 f1=0.0000 | time=12.7s
Epoch 005 | train_loss=0.7113 acc=0.4827 | val_loss=0.6934 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.8s
Epoch 006 | train_loss=0.7056 acc=0.5116 | val_loss=0.6935 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.8s
Epoch 007 | train_loss=0.6986 acc=0.5082 | val_loss=0.6940 acc=0.5011 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=12.7s
Epoch 008 | train_lo

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇██
f1_score,▁▁▁▁▁▁▃▁▇▇▅▇▇▇██████████████████████████
precision,▁▁▁▁▁▅▅▁▆▁▅▆▆▇▇▇▇██▇▇█▇▇█████▇▇▇█▇██▇█▇█
recall,▁▁▁▁▁▁▂▁▂█▄▇▆▆▇█▇▇▇▇▇▇▇▆▇▇▇▇▇▆▆▆▇▆▇▆▆▇▆▇
specificity,██████▅▇██▁▁▃▆▇▇▆▇▇▆▇▇▆▇▆▇▇▇▇▇▇▇▇▇▆▇▇▆▇▇
train_acc,▁▁▁▁▁▁▂▁▁▁▂▃▄▅▅▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇██████████
train_loss,███████████▇▇▆▆▅▅▅▅▅▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁
val_acc,▁▁▁▁▁▁▂▁▁▂▃▆▇▇▇▇██████▇████▇▇██▇█▇█▇██▇█
val_loss,███████████▆▄▄▄▃▂▃▂▂▂▂▂▃▁▁▂▁▁▁▁▁▁▁▁▁▁▂▂▂
epoch,71
f1_score,0.80995


[I 2025-05-08 00:35:25,210] Trial 9 finished with values: [0.39963088078158243, 0.8095238095238095] and parameters: {'lr': 3.219382221806734e-05, 'wd': 6.646893444723269e-05}.



=== Selected Trial #1 ===
 val_loss=0.3126, val_acc=0.8934, params={'lr': 7.26823423077802e-05, 'wd': 9.325987714179456e-05}, best_epoch=66, ckpt=ckpts/trial1_optimal_AD_CN_1.pth
=== Final Evaluation on Test Sets ===
-- test_within -- total=272  A(D)=146, C(N)=126
 Accuracy=0.8493 Sensitivity=0.8651 Specificity=0.8356 F1=0.8417

-- test_cross -- total=626  A(D)=319, C(N)=307
 Accuracy=0.6901 Sensitivity=0.7231 Specificity=0.6583 F1=0.6959



#### === Final Evaluation on Test Sets (Full Train) ===
- test_within -- total=272  A(D)=146, C(N)=126
 Accuracy=0.8272 Sensitivity=0.9206 Specificity=0.7466 F1=0.8315

- test_cross -- total=626  A(D)=319, C(N)=307
 Accuracy=0.6534 Sensitivity=0.8306 Specificity=0.4828 F1=0.7015

 === Final Evaluation on Test Sets ===
- test_within -- total=272  A(D)=146, C(N)=126
 Loss=0.3395 Acc=0.8676 Sens=0.8571 Spec=0.8767 F1=0.8571

- test_cross -- total=626  A(D)=319, C(N)=307
 Loss=0.6494 Acc=0.6757 Sens=0.6612 Spec=0.6897 F1=0.6667

#### === Final Evaluation on Test Sets (without Full Train + use the best trial instead of early stopping epoch model)===
- test_within -- total=272  A(D)=146, C(N)=126
 Accuracy=0.8493 Sensitivity=0.8651 Specificity=0.8356 F1=0.8417

- test_cross -- total=626  A(D)=319, C(N)=307
 Accuracy=0.6901 Sensitivity=0.7231 Specificity=0.6583 F1=0.6959

In [ ]:
# train_with_val_then_full.py
import os, json, random, time
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, f1_score, confusion_matrix

from eeg_dataset import EEGDataset
from model_optimized_6 import EEGformer

# ─── 1) Config & reproducibility ───────────────────────────────
SEED        = 0
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

DATA_DIR    = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE  = 'labels.json'
DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE   = 32
NUM_WORKERS  = 4
MAX_EPOCHS   = 150     # max for search
ES_PATIENCE  = 15     # early-stop patience
PCT_START    = 0.2

# hyperparameters you’ve tuned
LR           = 7.268e-5
WD           = 9.326e-5

# ─── 2) Dataset wrapper ────────────────────────────────────────
class BinaryEEGDataset(Dataset):
    def __init__(self, raw_ds, metas):
        self.raw_ds = raw_ds
        self.metas  = metas
    def __len__(self):
        return len(self.raw_ds)
    def __getitem__(self, idx):
        x, _ = self.raw_ds[idx]
        y     = self.metas[idx]['label']
        return x, y

# ─── 3) Load & prepare metadata ───────────────────────────────
with open(Path(DATA_DIR)/LABEL_FILE,'r') as f:
    all_meta = json.load(f)

train_meta       = [d for d in all_meta if d['type']=='train']
test_within_meta = [d for d in all_meta if d['type']=='test_within']
test_cross_meta  = [d for d in all_meta if d['type']=='test_cross']

# filter A vs F
class0, class1 = 'A','C'
label_map = {class0:0, class1:1}

train_meta       = [d for d in train_meta       if d['label'] in label_map]
test_within_meta = [d for d in test_within_meta if d['label'] in label_map]
test_cross_meta  = [d for d in test_cross_meta  if d['label'] in label_map]

# balance train by down-sampling
data0 = [d for d in train_meta if d['label']==class0]
data1 = [d for d in train_meta if d['label']==class1]
k     = min(len(data0), len(data1))
balanced_meta = random.sample(data0, k) + random.sample(data1, k)
random.shuffle(balanced_meta)

# convert to ints
for d in balanced_meta:       d['label'] = label_map[d['label']]
for d in test_within_meta:    d['label'] = label_map[d['label']]
for d in test_cross_meta:     d['label'] = label_map[d['label']]

print(f"\n[Balanced TRAIN] total={len(balanced_meta)} (each class={k})")

# build full balanced dataset
full_raw    = EEGDataset(DATA_DIR, balanced_meta)
full_ds     = BinaryEEGDataset(full_raw, balanced_meta)
labels_full = np.array([d['label'] for d in balanced_meta])

# ─── 4) Split train/val for epoch search ──────────────────────
idx            = np.arange(len(full_ds))
tr_idx, va_idx = train_test_split(
    idx, test_size=0.2, stratify=labels_full, random_state=SEED
)
train_loader = DataLoader(
    Subset(full_ds, tr_idx), batch_size=BATCH_SIZE,
    shuffle=True,  num_workers=NUM_WORKERS
)
val_loader   = DataLoader(
    Subset(full_ds, va_idx), batch_size=BATCH_SIZE,
    shuffle=False, num_workers=NUM_WORKERS
)

# ─── 5) Model builder & eval fn ───────────────────────────────
def build_model():
    input_len = full_ds[0][0].shape[-1]
    return EEGformer(
        in_channels=19, input_length=input_len,
        kernel_size=10, num_filters=120,
        num_heads=3, num_blocks=1,
        num_segments=5, num_classes=2
    ).to(DEVICE)

def evaluate(loader, model, criterion):
    model.eval()
    loss_sum = correct = total = 0
    with torch.no_grad():
        for X,y in loader:
            X,y    = X.to(DEVICE), y.to(DEVICE)
            logits = model(X)
            loss_sum += criterion(logits,y).item()*y.size(0)
            preds = logits.argmax(1)
            correct += (preds==y).sum().item()
            total   += y.size(0)
    return loss_sum/total, correct/total

# ─── 6) Phase 1: find best_epoch with early stopping ──────────
best_val_loss = float('inf')
best_epoch    = 0
es_count      = 0

model     = build_model()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=LR, epochs=MAX_EPOCHS,
    steps_per_epoch=len(train_loader),
    pct_start=PCT_START, anneal_strategy='cos',
    cycle_momentum=False
)
criterion = nn.CrossEntropyLoss()

print("\nSearching best epoch using validation set...")
for epoch in range(1, MAX_EPOCHS+1):
    t0 = time.time()
    # ── train ──
    model.train()
    tr_loss_sum = tr_correct = tr_total = 0
    for X,y in train_loader:
        X,y = X.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        logits = model(X)
        loss   = criterion(logits,y)
        loss.backward()
        optimizer.step()
        scheduler.step()

        tr_loss_sum  += loss.item()*y.size(0)
        preds        = logits.argmax(1)
        tr_correct  += (preds==y).sum().item()
        tr_total    += y.size(0)

    train_loss = tr_loss_sum / tr_total
    train_acc  = tr_correct / tr_total

    # ── validate ──
    val_loss, val_acc = evaluate(val_loader, model, criterion)
    elapsed = time.time() - t0

    print(
        f"Epoch {epoch:02d} | "
        f"train_loss={train_loss:.4f} | train_acc={train_acc:.4f} | "
        f"val_loss={val_loss:.4f} | val_acc={val_acc:.4f} | "
        f"time={elapsed:.1f}s"
    )

    # check for improvement
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_epoch    = epoch
        es_count      = 0
    else:
        es_count += 1
        if es_count >= ES_PATIENCE:
            print(f"\nEarly stopping at epoch {epoch} (no improvement for {ES_PATIENCE} epochs)")
            break

print(f"\n>> Best epoch by validation: {best_epoch}")

# ─── 7) Phase 2: retrain on full balanced set ─────────────────
print(f"\nRetraining from scratch on FULL set for {best_epoch} epochs...")
model2     = build_model()
optimizer2 = torch.optim.AdamW(model2.parameters(), lr=LR, weight_decay=WD)
scheduler2 = torch.optim.lr_scheduler.OneCycleLR(
    optimizer2, max_lr=LR,
    epochs=best_epoch,
    steps_per_epoch=len(DataLoader(full_ds, batch_size=BATCH_SIZE)),
    pct_start=PCT_START, anneal_strategy='cos',
    cycle_momentum=False
)
criterion2 = nn.CrossEntropyLoss()

for epoch in range(1, best_epoch+1):
    t0 = time.time()
    tr_loss_sum = tr_correct = tr_total = 0
    model2.train()
    for X,y in DataLoader(full_ds, batch_size=BATCH_SIZE,
                          shuffle=True, num_workers=NUM_WORKERS):
        X,y = X.to(DEVICE), y.to(DEVICE)
        optimizer2.zero_grad()
        logits = model2(X)
        loss   = criterion2(logits,y)
        loss.backward()
        optimizer2.step()
        scheduler2.step()

        tr_loss_sum  += loss.item()*y.size(0)
        preds        = logits.argmax(1)
        tr_correct  += (preds==y).sum().item()
        tr_total    += y.size(0)

    avg_loss = tr_loss_sum / tr_total
    acc      = tr_correct / tr_total
    print(f"Epoch {epoch:02d} | train_loss={avg_loss:.4f} acc={acc:.4f} time={time.time()-t0:.1f}s")

# ─── 8) Final Evaluation on Test Sets ─────────────────────────
def full_evaluate(metas, loader):
    model2.eval()
    loss_sum = correct = total = 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for X,y in loader:
            X,y = X.to(DEVICE), y.to(DEVICE)
            logits = model2(X)
            loss_sum += criterion2(logits,y).item()*y.size(0)
            preds    = logits.argmax(1)
            correct += (preds==y).sum().item()
            total   += y.size(0)
            all_preds.append(preds.cpu().numpy())
            all_labels.append(y.cpu().numpy())
    preds = np.concatenate(all_preds)
    labs  = np.concatenate(all_labels)
    tn,fp,fn,tp = confusion_matrix(labs,preds,labels=[0,1]).ravel()
    return {
        "loss": loss_sum/total,
        "acc": correct/total,
        "sensitivity": recall_score(labs,preds,zero_division=0),
        "specificity": tn/(tn+fp) if (tn+fp)>0 else 0.0,
        "f1": f1_score(labs,preds,zero_division=0)
    }

print("\n=== Final Evaluation on Test Sets ===")
for name, metas in [("test_within", test_within_meta), ("test_cross", test_cross_meta)]:
    loader = DataLoader(BinaryEEGDataset(EEGDataset(DATA_DIR, metas), metas),
                        batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
    res = full_evaluate(metas, loader)
    n0 = sum(1 for d in metas if d['label']==0)
    n1 = sum(1 for d in metas if d['label']==1)
    print(f"-- {name} -- total={len(metas)}  A(D)={n0}, C(N)={n1}")
    print(f" Loss={res['loss']:.4f} Acc={res['acc']:.4f} "
          f"Sens={res['sensitivity']:.4f} Spec={res['specificity']:.4f} "
          f"F1={res['f1']:.4f}\n")


[Balanced TRAIN] total=2204 (each class=1102)

Searching best epoch using validation set...
Epoch 01 | train_loss=0.7453 | train_acc=0.5173 | val_loss=0.6950 | val_acc=0.5011 | time=315.8s
Epoch 02 | train_loss=0.7388 | train_acc=0.5003 | val_loss=0.6943 | val_acc=0.4943 | time=13.1s
Epoch 03 | train_loss=0.7470 | train_acc=0.4810 | val_loss=0.6940 | val_acc=0.4943 | time=13.1s
Epoch 04 | train_loss=0.7419 | train_acc=0.4861 | val_loss=0.6940 | val_acc=0.4989 | time=13.0s
Epoch 05 | train_loss=0.7316 | train_acc=0.5014 | val_loss=0.6934 | val_acc=0.5102 | time=13.1s
Epoch 06 | train_loss=0.7283 | train_acc=0.4844 | val_loss=0.6938 | val_acc=0.5011 | time=12.8s
Epoch 07 | train_loss=0.7295 | train_acc=0.4940 | val_loss=0.6971 | val_acc=0.5011 | time=12.9s
Epoch 08 | train_loss=0.7210 | train_acc=0.5071 | val_loss=0.6945 | val_acc=0.5011 | time=13.1s
Epoch 09 | train_loss=0.7240 | train_acc=0.4991 | val_loss=0.6936 | val_acc=0.5057 | time=13.0s
Epoch 10 | train_loss=0.7255 | train_acc=0

## AD vs FTD (Best Optuna Search)

In [ ]:
import os
import json
import random
import time
import copy

import numpy as np
import torch
import torch.nn as nn
import wandb
import optuna

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from torch.utils.data import DataLoader, Subset, Dataset

from eeg_dataset import EEGDataset
from model_optimized_6 import EEGformer

# ─── Reproducibility ────────────────────────────────────────────
SEED = 0
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark     = False

# ─── Configuration ───────────────────────────────────────────────
DATA_DIR    = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE  = 'labels.json'
DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MAX_EPOCHS  = 150
BATCH_SIZE  = 32
NUM_WORKERS = 4
ES_PATIENCE = 15

# ─── Fixed Hyper-parameters ───────────────────────────────────────
PCT_START   = 0.2  # fixed

# ─── Dataset wrapper ─────────────────────────────────────────────
class BinaryEEGDataset(Dataset):
    def __init__(self, raw_ds, metas):
        self.raw_ds = raw_ds
        self.metas  = metas
    def __len__(self):
        return len(self.raw_ds)
    def __getitem__(self, idx):
        x, _ = self.raw_ds[idx]
        y     = self.metas[idx]['label']
        return x, y

# ─── Load & count splits ──────────────────────────────────────────
with open(os.path.join(DATA_DIR, LABEL_FILE), 'r') as f:
    all_meta = json.load(f)

train_meta       = [d for d in all_meta if d['type']=='train']
test_within_meta = [d for d in all_meta if d['type']=='test_within']
test_cross_meta  = [d for d in all_meta if d['type']=='test_cross']

class0, class1 = 'A','F'
label_map = {class0:0, class1:1}

# AD vs FTD Filtering
train_meta       = [d for d in train_meta if d['label'] in (class0, class1)]
test_within_meta = [d for d in test_within_meta if d['label'] in (class0, class1)]
test_cross_meta  = [d for d in test_cross_meta  if d['label'] in (class0, class1)]

def count_labels(meta_list):
    cnt0 = cnt1 = 0
    for d in meta_list:
        lbl = d['label']
        if isinstance(lbl, str):
            if lbl == class0: cnt0 += 1
            elif lbl == class1: cnt1 += 1
        else:
            if lbl == 0: cnt0 += 1
            elif lbl == 1: cnt1 += 1
    return cnt0, cnt1

n_tr0, n_tr1 = count_labels(train_meta)
n_tw0, n_tw1 = count_labels(test_within_meta)
n_tc0, n_tc1 = count_labels(test_cross_meta)

print(f"--> Data counts before balancing:")
print(f"    TRAIN         total={len(train_meta)}  A(D)={n_tr0}, F(TD)={n_tr1}")
print(f"    TEST_WITHIN   total={len(test_within_meta)}  A(D)={n_tw0}, F(TD)={n_tw1}")
print(f"    TEST_CROSS    total={len(test_cross_meta)}  A(D)={n_tc0}, F(TD)={n_tc1}\n")

# ─── Balance train set ───────────────────────────────────────────
data0 = [d for d in train_meta if d['label']==class0]
data1 = [d for d in train_meta if d['label']==class1]
min_count = min(len(data0), len(data1))

balanced_meta = random.sample(data0, min_count) + random.sample(data1, min_count)
balanced_meta = [copy.deepcopy(d) for d in balanced_meta]
random.shuffle(balanced_meta)

# ─── After Balancing ───────────────────────────────────────────
bal_AD, bal_FTD = count_labels(balanced_meta)
print(f"[BALANCED TRAIN] total={len(balanced_meta)}  AD={bal_AD}, FTD={bal_FTD}\n")

for d in balanced_meta:
    d['label'] = label_map[d['label']]

raw_ds_train  = EEGDataset(DATA_DIR, balanced_meta)
dataset_train = BinaryEEGDataset(raw_ds_train, balanced_meta)
labels_train  = np.array([d['label'] for d in balanced_meta])

# ─── Optuna Objective ────────────────────────────────────────────
def objective(trial):
    # 1) sample hyperparameters
    lr = trial.suggest_float('lr', 5e-5, 5e-4, log=True)
    wd = trial.suggest_float('wd', 5e-5, 5e-4, log=True)


    print(f"\n--- Trial {trial.number} hyperparams --> lr={lr:.2e}, wd={wd:.2e}")

    wandb.init(
        project='eeg-optuna-AD-FTD-test-within-cross-1',
        name=f"trial{trial.number}",
        config={'lr':lr, 'wd':wd, 'pct_start':PCT_START},
        reinit=True
    )

    # train/validation split
    idx = np.arange(len(dataset_train))
    tr_idx, va_idx = train_test_split(
        idx, test_size=0.2,
        stratify=labels_train, random_state=SEED
    )

    tr_AD = np.sum(labels_train[tr_idx] == 0)
    tr_FTD = np.sum(labels_train[tr_idx] == 1)
    va_AD = np.sum(labels_train[va_idx] == 0)
    va_FTD = np.sum(labels_train[va_idx] == 1)
    print(f"[Trial {trial.number}]  "
          f"TRAIN n={len(tr_idx)} AD={tr_AD}, FTD={tr_FTD}) | "
          f"VAL n={len(va_idx)} AD={va_AD}, FTD={va_FTD}")

    train_loader = DataLoader(
        Subset(dataset_train, tr_idx),
        batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS
    )
    val_loader   = DataLoader(
        Subset(dataset_train, va_idx),
        batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
    )

    # model & optimizer & scheduler & loss
    input_len = dataset_train[0][0].shape[-1]
    model = EEGformer(
        in_channels=19, input_length=input_len,
        kernel_size=10, num_filters=120,
        num_heads=3, num_blocks=1,
        num_segments=5, num_classes=2
    ).to(DEVICE)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=lr, epochs=MAX_EPOCHS,
        steps_per_epoch=len(train_loader), pct_start=PCT_START,
        anneal_strategy='cos', cycle_momentum=False
    )
    criterion = nn.CrossEntropyLoss()

    best_val_loss = float('inf')
    best_val_acc  = 0.0
    best_state= None
    es_count      = 0

    for epoch in range(1, MAX_EPOCHS+1):
        t0 = time.time()
        # ── train ──
        model.train()
        train_loss_sum = train_correct = train_total = 0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()
            scheduler.step()
            # dynamic weight_decay
            cur_lr = optimizer.param_groups[0]['lr']
            for g in optimizer.param_groups:
                g['weight_decay'] = wd * (cur_lr / lr)
            train_loss_sum += loss.item()
            preds = logits.argmax(1)
            train_correct += (preds == y).sum().item()
            train_total   += y.size(0)
        train_loss = train_loss_sum / len(train_loader)
        train_acc  = train_correct / train_total

        # ── validate ──
        model.eval()
        vloss = 0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                vloss += criterion(logits, y).item()
                preds = logits.argmax(1)
                all_preds.append(preds.cpu().numpy())
                all_labels.append(y.cpu().numpy())
        val_loss = vloss / len(val_loader)

        preds = np.concatenate(all_preds)
        labs  = np.concatenate(all_labels)
        val_acc = (preds == labs).sum() / labs.size

        # ── metrics ──
        tn, fp, fn, tp = confusion_matrix(labs, preds, labels=[0,1]).ravel()
        spec = tn / (tn+fp) if (tn+fp)>0 else 0.0
        prec = precision_score(labs, preds, zero_division=0)
        rec  = recall_score(labs, preds, zero_division=0)
        f1   = f1_score(labs, preds, zero_division=0)
        elapsed = time.time() - t0

        print(
            f"Epoch {epoch:03d} | "
            f"train_loss={train_loss:.4f} acc={train_acc:.4f} | "
            f"val_loss={val_loss:.4f} acc={val_acc:.4f} | "
            f"prec={prec:.4f} rec={rec:.4f} spec={spec:.4f} f1={f1:.4f} | "
            f"time={elapsed:.1f}s"
        )

        wandb.log({
            'epoch':       epoch,
            'train_loss':  train_loss,
            'train_acc':   train_acc,
            'val_loss':    val_loss,
            'val_acc':     val_acc,
            'specificity': spec,
            'precision':   prec,
            'recall':      rec,
            'f1_score':    f1
        })

        # ── Early Stopping (val_loss 기준) ──
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_val_acc  = val_acc
            es_count      = 0
            best_state    = copy.deepcopy(model.state_dict())

            os.makedirs('ckpts', exist_ok=True)
            ckpt_path = f'ckpts/trial{trial.number}_optimal_AD_FTD_1.pth'
            torch.save(best_state, ckpt_path)
            trial.set_user_attr('ckpt_path', ckpt_path)
            trial.set_user_attr('best_epoch', epoch)
        else:
            es_count += 1
            if es_count >= ES_PATIENCE:
                break

    if best_state is not None:
        model.load_state_dict(best_state)

    wandb.finish()
    del model, optimizer, train_loader, val_loader, scheduler
    torch.cuda.empty_cache()

    # 다중목적 리턴
    return best_val_loss, best_val_acc

# ─── Run Optuna Study ────────────────────────────────────────────
if __name__ == "__main__":
    study = optuna.create_study(
        directions=["minimize", "maximize"],
        study_name="eeg_multiobj_AD_FTD_1"
    )
    study.optimize(objective, n_trials=10)

    # Pareto front 중 val_acc 가 가장 높은 trial 선택
    best       = max(study.best_trials, key=lambda t: t.values[1])
    bv_loss, bv_acc = best.values
    best_epoch = best.user_attrs["best_epoch"]
    ckpt_path  = best.user_attrs["ckpt_path"]

    print(f"\n=== Selected Trial #{best.number} ===")
    print(
        f" val_loss={bv_loss:.4f}, val_acc={bv_acc:.4f}, "
        f"params={best.params}, best_epoch={best_epoch}, ckpt={ckpt_path}"
    )

    # ─── Search‑model 로드 ───────────────────────────────────────
    input_len = dataset_train[0][0].shape[-1]
    search_model = EEGformer(
        in_channels=19, input_length=input_len,
        kernel_size=10, num_filters=120,
        num_heads=3, num_blocks=1,
        num_segments=5, num_classes=2
    ).to(DEVICE)

    search_model.load_state_dict(torch.load(ckpt_path, map_location=DEVICE))
    search_model.eval()
    criterion = nn.CrossEntropyLoss()

    # ─── Evaluation 함수 ─────────────────────────────────────────
    def evaluate(model, metas, criterion):
        metas_copy = copy.deepcopy(metas)
        for d in metas_copy:
            if isinstance(d["label"], str):
                d["label"] = label_map[d["label"]]

        ds = BinaryEEGDataset(EEGDataset(DATA_DIR, metas_copy), metas_copy)
        loader = DataLoader(
            ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
        )

        model.eval()
        vloss = vcorrect = vtotal = 0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for X, y in loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                vloss += criterion(logits, y).item()
                preds = logits.argmax(1)
                vcorrect += (preds == y).sum().item()
                vtotal += y.size(0)
                all_preds.append(preds.cpu().numpy())
                all_labels.append(y.cpu().numpy())

        preds = np.concatenate(all_preds)
        labs  = np.concatenate(all_labels)
        tn, fp, fn, tp = confusion_matrix(labs, preds, labels=[0, 1]).ravel()
        return {
            "loss":        vloss / len(loader),
            "acc":         vcorrect / vtotal,
            "sensitivity": recall_score(labs, preds, zero_division=0),
            "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0.0,
            "f1":          f1_score(labs, preds, zero_division=0),
        }

    # ─── Final Evaluation on Test Sets ────────────────────────────
    print("=== Final Evaluation on Test Sets ===")
    for name, metas in [
        ("test_within", test_within_meta),
        ("test_cross",  test_cross_meta),
    ]:
        res = evaluate(search_model, metas, criterion)
        n0, n1 = count_labels(metas)
        print(f"-- {name} -- total={len(metas)}  A(D)={n0}, F(TD)={n1}")
        print(
            f" Accuracy={res['acc']:.4f} "
            f"Sensitivity={res['sensitivity']:.4f} "
            f"Specificity={res['specificity']:.4f} "
            f"F1={res['f1']:.4f}\n"
        )

[I 2025-05-08 06:55:58,296] A new study created in memory with name: eeg_multiobj_AD_FTD_1


--> Data counts before balancing:
    TRAIN         total=2117  A(D)=1388, F(TD)=729
    TEST_WITHIN   total=218  A(D)=146, F(TD)=72
    TEST_CROSS    total=566  A(D)=319, F(TD)=247

[BALANCED TRAIN] total=1458  AD=729, FTD=729


--- Trial 0 hyperparams --> lr=3.94e-05, wd=2.07e-04


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


[Trial 0]  TRAIN n=1166 AD=583, FTD=583) | VAL n=292 AD=146, FTD=146
Epoch 001 | train_loss=0.7508 acc=0.4880 | val_loss=0.6922 acc=0.5000 | prec=0.5000 rec=1.0000 spec=0.0000 f1=0.6667 | time=11.3s
Epoch 002 | train_loss=0.7425 acc=0.5026 | val_loss=0.6922 acc=0.4932 | prec=0.4965 rec=0.9795 spec=0.0068 f1=0.6590 | time=9.1s
Epoch 003 | train_loss=0.7524 acc=0.4983 | val_loss=0.6923 acc=0.5068 | prec=0.5035 rec=0.9863 spec=0.0274 f1=0.6667 | time=9.2s
Epoch 004 | train_loss=0.7578 acc=0.4931 | val_loss=0.6935 acc=0.5068 | prec=0.5185 rec=0.1918 spec=0.8219 f1=0.2800 | time=9.6s
Epoch 005 | train_loss=0.7265 acc=0.5111 | val_loss=0.6920 acc=0.5000 | prec=0.5000 rec=1.0000 spec=0.0000 f1=0.6667 | time=9.4s
Epoch 006 | train_loss=0.7434 acc=0.4871 | val_loss=0.6921 acc=0.5103 | prec=0.5052 rec=0.9932 spec=0.0274 f1=0.6697 | time=9.2s
Epoch 007 | train_loss=0.7350 acc=0.5103 | val_loss=0.6913 acc=0.5000 | prec=0.5000 rec=1.0000 spec=0.0000 f1=0.6667 | time=9.1s
Epoch 008 | train_loss=0.72

epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇██
f1_score,███▄█████████▆▁█▄█▁█▆▁
precision,▇▇▇▇▇▇▇▇▇▇▇▇▇▇▁▇█▇▁▇▇▁
recall,███▂█████████▄▁█▂█▁█▄▁
specificity,▁▁▁▇▁▁▁▁▁▁▁▁▁▅█▁▇▁█▁▅█
train_acc,▂▄▄▃▅▂▅▇▅▇▅▂▃▅▃▅█▁▃▇▅▇
train_loss,▇▆▇█▃▆▄▃▄▂▄▃▄▂▃▁▁▄▃▁▂▁
val_acc,▂▁▃▃▂▄▂▂▂▃▂▂▂█▂▂▆▂▂▂▆▂
val_loss,▁▁▁▂▁▁▁▃▁▁▁▂▂▂▃▁▂▁▃▂▃█
epoch,22
f1_score,0


...


--- Trial 9 hyperparams --> lr=4.27e-03, wd=3.81e-06


[Trial 9]  TRAIN n=1166 AD=583, FTD=583) | VAL n=292 AD=146, FTD=146
Epoch 001 | train_loss=0.7149 acc=0.4880 | val_loss=0.6934 acc=0.4658 | prec=0.4324 rec=0.2192 spec=0.7123 f1=0.2909 | time=9.1s
Epoch 002 | train_loss=0.7022 acc=0.4949 | val_loss=0.7016 acc=0.5000 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=9.0s
Epoch 003 | train_loss=0.6993 acc=0.5043 | val_loss=0.6962 acc=0.4966 | prec=0.4951 rec=0.3493 spec=0.6438 f1=0.4096 | time=9.1s
Epoch 004 | train_loss=0.6913 acc=0.5206 | val_loss=0.6992 acc=0.5137 | prec=0.5455 rec=0.1644 spec=0.8630 f1=0.2526 | time=9.0s
Epoch 005 | train_loss=0.6766 acc=0.5763 | val_loss=0.6924 acc=0.5445 | prec=0.5684 rec=0.3699 spec=0.7192 f1=0.4481 | time=9.1s
Epoch 006 | train_loss=0.6467 acc=0.6235 | val_loss=0.6862 acc=0.5479 | prec=0.5603 rec=0.4452 spec=0.6507 f1=0.4962 | time=9.2s
Epoch 007 | train_loss=0.6179 acc=0.6655 | val_loss=0.6401 acc=0.5788 | prec=0.5550 rec=0.7945 spec=0.3630 f1=0.6535 | time=9.2s
Epoch 008 | train_loss=0.580

epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
f1_score,▄▁▅▄▅▆█▅▆█▇██▇▃▄▅█▆██▁▆▆███▆▆▃█▇█▅█
precision,▅▁▅▆▆▆▆█▇▅▆▅▆▆██▇▆▆▆▆▇▇▆▆▅▅▆▆▆▅▆▆▆▆
recall,▃▁▃▂▄▄▇▃▄█▅█▆▅▂▃▃█▄▆▇▁▃▄▇██▄▃▂█▅▇▃▇
specificity,▆█▅▇▆▅▃█▇▁▆▂▅▆██▇▂▆▅▃█▇▆▃▁▂▆▇█▂▆▄▇▄
train_acc,▁▁▁▂▂▃▄▄▅▆▅▆▆▆▆▇▆▇▇▇▇▇▇▇█▇██████▇▇▇
train_loss,█████▇▇▆▆▅▅▅▅▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁▃▃▂
val_acc,▁▂▂▃▄▄▅▆▇▄▇▄▇▆▄▅▅▅▅█▆▂▆▅▆▃▄▆▅▃▃▇█▅▇
val_loss,▁▁▁▁▁▁▁▁▁▂▁▁▂▁▂▃▂▃▁▁▂▄▁▄▄▅▆▄▅▄▇▄▁█▅
epoch,35
f1_score,0.69081


[I 2025-05-08 07:41:35,178] Trial 9 finished with values: [0.6331340909004212, 0.6575342465753424] and parameters: {'lr': 0.0042651260745797835, 'wd': 3.8141822461242195e-06}.



=== Selected Trial #6 ===
 val_loss=0.5694, val_acc=0.6610, params={'lr': 0.00015285834289101232, 'wd': 0.00017917460394166492}, best_epoch=29, ckpt=ckpts/trial6_optimal_AD_FTD_1.pth
=== Final Evaluation on Test Sets ===
-- test_within -- total=218  A(D)=146, F(TD)=72
 Accuracy=0.6927 Sensitivity=0.5833 Specificity=0.7466 F1=0.5563

-- test_cross -- total=566  A(D)=319, F(TD)=247
 Accuracy=0.5883 Sensitivity=0.3725 Specificity=0.7555 F1=0.4412



In [ ]:
import os
import json
import random
import time
import copy

import numpy as np
import torch
import torch.nn as nn
import wandb
import optuna

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from torch.utils.data import DataLoader, Subset, Dataset

from eeg_dataset import EEGDataset
from model_optimized_6 import EEGformer

# ─── Reproducibility ────────────────────────────────────────────
SEED = 0
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark     = False

# ─── Configuration ───────────────────────────────────────────────
DATA_DIR    = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE  = 'labels.json'
DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MAX_EPOCHS  = 150
BATCH_SIZE  = 32
NUM_WORKERS = 4
ES_PATIENCE = 15

# ─── Fixed Hyper-parameters ───────────────────────────────────────
PCT_START   = 0.2  # fixed

# ─── Dataset wrapper ─────────────────────────────────────────────
class BinaryEEGDataset(Dataset):
    def __init__(self, raw_ds, metas):
        self.raw_ds = raw_ds
        self.metas  = metas
    def __len__(self):
        return len(self.raw_ds)
    def __getitem__(self, idx):
        x, _ = self.raw_ds[idx]
        y     = self.metas[idx]['label']
        return x, y

# ─── Load & count splits ──────────────────────────────────────────
with open(os.path.join(DATA_DIR, LABEL_FILE), 'r') as f:
    all_meta = json.load(f)

train_meta       = [d for d in all_meta if d['type']=='train']
test_within_meta = [d for d in all_meta if d['type']=='test_within']
test_cross_meta  = [d for d in all_meta if d['type']=='test_cross']

class0, class1 = 'A','F'
label_map = {class0:0, class1:1}

# AD vs FTD Filtering
train_meta       = [d for d in train_meta if d['label'] in (class0, class1)]
test_within_meta = [d for d in test_within_meta if d['label'] in (class0, class1)]
test_cross_meta  = [d for d in test_cross_meta  if d['label'] in (class0, class1)]

def count_labels(meta_list):
    cnt0 = cnt1 = 0
    for d in meta_list:
        lbl = d['label']
        if isinstance(lbl, str):
            if lbl == class0: cnt0 += 1
            elif lbl == class1: cnt1 += 1
        else:
            if lbl == 0: cnt0 += 1
            elif lbl == 1: cnt1 += 1
    return cnt0, cnt1

n_tr0, n_tr1 = count_labels(train_meta)
n_tw0, n_tw1 = count_labels(test_within_meta)
n_tc0, n_tc1 = count_labels(test_cross_meta)

print(f"--> Data counts before balancing:")
print(f"    TRAIN         total={len(train_meta)}  A(D)={n_tr0}, F(TD)={n_tr1}")
print(f"    TEST_WITHIN   total={len(test_within_meta)}  A(D)={n_tw0}, F(TD)={n_tw1}")
print(f"    TEST_CROSS    total={len(test_cross_meta)}  A(D)={n_tc0}, F(TD)={n_tc1}\n")

# ─── Balance train set ───────────────────────────────────────────
data0 = [d for d in train_meta if d['label']==class0]
data1 = [d for d in train_meta if d['label']==class1]
min_count = min(len(data0), len(data1))

balanced_meta = random.sample(data0, min_count) + random.sample(data1, min_count)
balanced_meta = [copy.deepcopy(d) for d in balanced_meta]
random.shuffle(balanced_meta)

# ─── After Balancing ───────────────────────────────────────────
bal_AD, bal_FTD = count_labels(balanced_meta)
print(f"[BALANCED TRAIN] total={len(balanced_meta)}  AD={bal_AD}, FTD={bal_FTD}\n")

for d in balanced_meta:
    d['label'] = label_map[d['label']]

raw_ds_train  = EEGDataset(DATA_DIR, balanced_meta)
dataset_train = BinaryEEGDataset(raw_ds_train, balanced_meta)
labels_train  = np.array([d['label'] for d in balanced_meta])

# ─── Optuna Objective ────────────────────────────────────────────
def objective(trial):
    # 1) sample hyperparameters
    lr = trial.suggest_float('lr', 5e-5, 5e-4, log=True)
    wd = trial.suggest_float('wd', 5e-5, 5e-4, log=True)


    print(f"\n--- Trial {trial.number} hyperparams --> lr={lr:.2e}, wd={wd:.2e}")

    wandb.init(
        project='eeg-optuna-AD-FTD-test-within-cross-2',
        name=f"trial{trial.number}",
        config={'lr':lr, 'wd':wd, 'pct_start':PCT_START},
        reinit=True
    )

    # train/validation split
    idx = np.arange(len(dataset_train))
    tr_idx, va_idx = train_test_split(
        idx, test_size=0.2,
        stratify=labels_train, random_state=SEED
    )

    tr_AD = np.sum(labels_train[tr_idx] == 0)
    tr_FTD = np.sum(labels_train[tr_idx] == 1)
    va_AD = np.sum(labels_train[va_idx] == 0)
    va_FTD = np.sum(labels_train[va_idx] == 1)
    print(f"[Trial {trial.number}]  "
          f"TRAIN n={len(tr_idx)} AD={tr_AD}, FTD={tr_FTD}) | "
          f"VAL n={len(va_idx)} AD={va_AD}, FTD={va_FTD}")

    train_loader = DataLoader(
        Subset(dataset_train, tr_idx),
        batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS
    )
    val_loader   = DataLoader(
        Subset(dataset_train, va_idx),
        batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
    )

    # model & optimizer & scheduler & loss
    input_len = dataset_train[0][0].shape[-1]
    model = EEGformer(
        in_channels=19, input_length=input_len,
        kernel_size=10, num_filters=120,
        num_heads=3, num_blocks=1,
        num_segments=5, num_classes=2
    ).to(DEVICE)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=lr, epochs=MAX_EPOCHS,
        steps_per_epoch=len(train_loader), pct_start=PCT_START,
        anneal_strategy='cos', cycle_momentum=False
    )
    criterion = nn.CrossEntropyLoss()

    best_val_loss = float('inf')
    best_val_acc  = 0.0
    best_state= None
    es_count      = 0

    for epoch in range(1, MAX_EPOCHS+1):
        t0 = time.time()
        # ── train ──
        model.train()
        train_loss_sum = train_correct = train_total = 0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()
            scheduler.step()
            # dynamic weight_decay
            cur_lr = optimizer.param_groups[0]['lr']
            for g in optimizer.param_groups:
                g['weight_decay'] = wd * (cur_lr / lr)
            train_loss_sum += loss.item()
            preds = logits.argmax(1)
            train_correct += (preds == y).sum().item()
            train_total   += y.size(0)
        train_loss = train_loss_sum / len(train_loader)
        train_acc  = train_correct / train_total

        # ── validate ──
        model.eval()
        vloss = 0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                vloss += criterion(logits, y).item()
                preds = logits.argmax(1)
                all_preds.append(preds.cpu().numpy())
                all_labels.append(y.cpu().numpy())
        val_loss = vloss / len(val_loader)

        preds = np.concatenate(all_preds)
        labs  = np.concatenate(all_labels)
        val_acc = (preds == labs).sum() / labs.size

        # ── metrics ──
        tn, fp, fn, tp = confusion_matrix(labs, preds, labels=[0,1]).ravel()
        spec = tn / (tn+fp) if (tn+fp)>0 else 0.0
        prec = precision_score(labs, preds, zero_division=0)
        rec  = recall_score(labs, preds, zero_division=0)
        f1   = f1_score(labs, preds, zero_division=0)
        elapsed = time.time() - t0

        print(
            f"Epoch {epoch:03d} | "
            f"train_loss={train_loss:.4f} acc={train_acc:.4f} | "
            f"val_loss={val_loss:.4f} acc={val_acc:.4f} | "
            f"prec={prec:.4f} rec={rec:.4f} spec={spec:.4f} f1={f1:.4f} | "
            f"time={elapsed:.1f}s"
        )

        wandb.log({
            'epoch':       epoch,
            'train_loss':  train_loss,
            'train_acc':   train_acc,
            'val_loss':    val_loss,
            'val_acc':     val_acc,
            'specificity': spec,
            'precision':   prec,
            'recall':      rec,
            'f1_score':    f1
        })

        # ── Early Stopping (val_loss 기준) ──
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_val_acc  = val_acc
            es_count      = 0
            best_state    = copy.deepcopy(model.state_dict())

            os.makedirs('ckpts', exist_ok=True)
            ckpt_path = f'ckpts/trial{trial.number}_optimal_AD_FTD_2.pth'
            torch.save(best_state, ckpt_path)
            trial.set_user_attr('ckpt_path', ckpt_path)
            trial.set_user_attr('best_epoch', epoch)
        else:
            es_count += 1
            if es_count >= ES_PATIENCE:
                break

    if best_state is not None:
        model.load_state_dict(best_state)

    wandb.finish()
    del model, optimizer, train_loader, val_loader, scheduler
    torch.cuda.empty_cache()

    # 다중목적 리턴
    return best_val_loss, best_val_acc

# ─── Run Optuna Study ────────────────────────────────────────────
if __name__ == "__main__":
    study = optuna.create_study(
        directions=["minimize", "maximize"],
        study_name="eeg_multiobj_AD_FTD_2"
    )
    study.optimize(objective, n_trials=10)

    # Pareto front 중 val_acc 가 가장 높은 trial 선택
    best       = max(study.best_trials, key=lambda t: t.values[1])
    bv_loss, bv_acc = best.values
    best_epoch = best.user_attrs["best_epoch"]
    ckpt_path  = best.user_attrs["ckpt_path"]

    print(f"\n=== Selected Trial #{best.number} ===")
    print(
        f" val_loss={bv_loss:.4f}, val_acc={bv_acc:.4f}, "
        f"params={best.params}, best_epoch={best_epoch}, ckpt={ckpt_path}"
    )

    # ─── Search‑model 로드 ───────────────────────────────────────
    input_len = dataset_train[0][0].shape[-1]
    search_model = EEGformer(
        in_channels=19, input_length=input_len,
        kernel_size=10, num_filters=120,
        num_heads=3, num_blocks=1,
        num_segments=5, num_classes=2
    ).to(DEVICE)

    search_model.load_state_dict(torch.load(ckpt_path, map_location=DEVICE))
    search_model.eval()
    criterion = nn.CrossEntropyLoss()

    # ─── Evaluation 함수 ─────────────────────────────────────────
    def evaluate(model, metas, criterion):
        metas_copy = copy.deepcopy(metas)
        for d in metas_copy:
            if isinstance(d["label"], str):
                d["label"] = label_map[d["label"]]

        ds = BinaryEEGDataset(EEGDataset(DATA_DIR, metas_copy), metas_copy)
        loader = DataLoader(
            ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
        )

        model.eval()
        vloss = vcorrect = vtotal = 0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for X, y in loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                vloss += criterion(logits, y).item()
                preds = logits.argmax(1)
                vcorrect += (preds == y).sum().item()
                vtotal += y.size(0)
                all_preds.append(preds.cpu().numpy())
                all_labels.append(y.cpu().numpy())

        preds = np.concatenate(all_preds)
        labs  = np.concatenate(all_labels)
        tn, fp, fn, tp = confusion_matrix(labs, preds, labels=[0, 1]).ravel()
        return {
            "loss":        vloss / len(loader),
            "acc":         vcorrect / vtotal,
            "sensitivity": recall_score(labs, preds, zero_division=0),
            "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0.0,
            "f1":          f1_score(labs, preds, zero_division=0),
        }

    # ─── Final Evaluation on Test Sets ────────────────────────────
    print("=== Final Evaluation on Test Sets ===")
    for name, metas in [
        ("test_within", test_within_meta),
        ("test_cross",  test_cross_meta),
    ]:
        res = evaluate(search_model, metas, criterion)
        n0, n1 = count_labels(metas)
        print(f"-- {name} -- total={len(metas)}  A(D)={n0}, F(TD)={n1}")
        print(
            f" Accuracy={res['acc']:.4f} "
            f"Sensitivity={res['sensitivity']:.4f} "
            f"Specificity={res['specificity']:.4f} "
            f"F1={res['f1']:.4f}\n"
        )

[I 2025-05-08 08:25:56,052] A new study created in memory with name: eeg_multiobj_AD_FTD_2


--> Data counts before balancing:
    TRAIN         total=2117  A(D)=1388, F(TD)=729
    TEST_WITHIN   total=218  A(D)=146, F(TD)=72
    TEST_CROSS    total=566  A(D)=319, F(TD)=247

[BALANCED TRAIN] total=1458  AD=729, FTD=729


--- Trial 0 hyperparams --> lr=1.28e-04, wd=1.39e-04


[Trial 0]  TRAIN n=1166 AD=583, FTD=583) | VAL n=292 AD=146, FTD=146
Epoch 001 | train_loss=0.7494 acc=0.4906 | val_loss=0.6921 acc=0.4966 | prec=0.4983 rec=0.9932 spec=0.0000 f1=0.6636 | time=9.1s
Epoch 002 | train_loss=0.7342 acc=0.5146 | val_loss=0.6940 acc=0.4966 | prec=0.4615 rec=0.0411 spec=0.9521 f1=0.0755 | time=9.1s
Epoch 003 | train_loss=0.7469 acc=0.4940 | val_loss=0.6940 acc=0.5103 | prec=0.5306 rec=0.1781 spec=0.8425 f1=0.2667 | time=9.1s
Epoch 004 | train_loss=0.7486 acc=0.5069 | val_loss=0.6953 acc=0.5000 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=9.0s
Epoch 005 | train_loss=0.7198 acc=0.5232 | val_loss=0.6986 acc=0.5000 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=9.1s
Epoch 006 | train_loss=0.7352 acc=0.4906 | val_loss=0.6918 acc=0.5000 | prec=0.5000 rec=0.9589 spec=0.0411 f1=0.6573 | time=8.9s
Epoch 007 | train_loss=0.7276 acc=0.4991 | val_loss=0.6924 acc=0.5205 | prec=0.5160 rec=0.6644 spec=0.3767 f1=0.5808 | time=9.0s
Epoch 008 | train_loss=0.718

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1_score,█▂▄▁▁▇██▃██▃▆▆▇▆▅▆▇▇▆▇█▇▇▇▇█▇███████▇▇▇▇
precision,▆▆▇▁▁▆▆▆▆▆▆▇▆▇▆▆▇▆▇▇▇█▇████████████▇██▇▇
recall,█▁▂▁▁▆██▂██▂▅▄▇▄▄▅▅▆▄▄▆▅▅▅▆▆▆▆▆▇▆▆▆▇▅▅▅▆
specificity,▁█▇██▄▁▁▇▁▁▇▄▅▂▅▆▅▄▄▆▆▄▆▆▆▅▆▅▅▅▄▅▅▅▄▆▆▆▅
train_acc,▁▁▁▁▂▁▁▁▁▁▁▁▁▂▂▂▂▃▃▃▄▄▅▅▅▆▆▇▇▇▇▇▇███████
train_loss,█████████████▇▇▇▇▇▇▇▆▆▅▅▅▄▄▄▃▃▃▂▂▂▂▁▁▁▁▁
val_acc,▁▁▂▁▁▂▁▁▁▁▁▂▁▂▂▁▂▂▃▄▅▆▆▆▆▇▇█▇█▇████▇█▇▆▆
val_loss,▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▄▃▄▃▃▃▂▂▃▂▂▁▂▂▃▄▄▃▅▅▅▆██
epoch,48
f1_score,0.65231


[I 2025-05-08 08:33:10,956] Trial 0 finished with values: [0.6145085096359253, 0.660958904109589] and parameters: {'lr': 0.00012798057921434346, 'wd': 0.0001388702054424885}.


...


--- Trial 9 hyperparams --> lr=3.81e-04, wd=3.50e-04


[Trial 9]  TRAIN n=1166 AD=583, FTD=583) | VAL n=292 AD=146, FTD=146
Epoch 001 | train_loss=0.7272 acc=0.5163 | val_loss=0.6940 acc=0.5000 | prec=0.5000 rec=1.0000 spec=0.0000 f1=0.6667 | time=9.0s
Epoch 002 | train_loss=0.7242 acc=0.5017 | val_loss=0.7042 acc=0.5000 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=9.0s
Epoch 003 | train_loss=0.7143 acc=0.5172 | val_loss=0.6920 acc=0.5068 | prec=0.5037 rec=0.9247 spec=0.0890 f1=0.6522 | time=9.0s
Epoch 004 | train_loss=0.7341 acc=0.4683 | val_loss=0.6952 acc=0.5000 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=9.0s
Epoch 005 | train_loss=0.7201 acc=0.5120 | val_loss=0.6968 acc=0.5000 | prec=0.0000 rec=0.0000 spec=1.0000 f1=0.0000 | time=9.1s
Epoch 006 | train_loss=0.7155 acc=0.5017 | val_loss=0.6936 acc=0.4418 | prec=0.4605 rec=0.6781 spec=0.2055 f1=0.5485 | time=9.1s
Epoch 007 | train_loss=0.7257 acc=0.4863 | val_loss=0.6930 acc=0.4897 | prec=0.4933 rec=0.7534 spec=0.2260 f1=0.5962 | time=9.0s
Epoch 008 | train_loss=0.707

epoch,▁▁▂▂▃▃▃▄▄▅▅▆▆▆▇▇██
f1_score,█▁█▁▁▇▇▁█▇▇▇▇▆▇▇▇▆
precision,█▁█▁▁▇█▁███▇█▇██▇▇
recall,█▁▇▁▁▆▆▁█▇▅▆▅▅▅▅▅▅
specificity,▁█▂██▂▃█▁▂▄▃▄▄▄▃▃▄
train_acc,▄▃▄▁▄▃▂▄▃▄▅▆▆▅▆▆▇█
train_loss,▇▇▆█▇▆▇▅▅▆▆▄▄▄▃▃▁▁
val_acc,▆▆▇▆▆▁▅▆▆▆▇▂▇▃█▄▄▄
val_loss,▁▂▁▁▁▁▁▂▂▂▂▂▂▃▂▃▄█
epoch,18
f1_score,0.51104


[I 2025-05-08 09:12:30,098] Trial 9 finished with values: [0.6919521510601043, 0.5068493150684932] and parameters: {'lr': 0.0003811633779098951, 'wd': 0.00035001663370376116}.



=== Selected Trial #5 ===
 val_loss=0.5941, val_acc=0.6849, params={'lr': 0.0002519980361581342, 'wd': 0.0001360355853136191}, best_epoch=22, ckpt=ckpts/trial5_optimal_AD_FTD_2.pth
=== Final Evaluation on Test Sets ===
-- test_within -- total=218  A(D)=146, F(TD)=72
 Accuracy=0.6835 Sensitivity=0.6111 Specificity=0.7192 F1=0.5605

-- test_cross -- total=566  A(D)=319, F(TD)=247
 Accuracy=0.6343 Sensitivity=0.4251 Specificity=0.7962 F1=0.5036



## AD vs FTD (Full Train test)

In [ ]:
# train_with_val_then_full.py
import os, json, random, time
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, f1_score, confusion_matrix

from eeg_dataset import EEGDataset
from model_optimized_6 import EEGformer

# ─── 1) Config & reproducibility ───────────────────────────────
SEED        = 0
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

DATA_DIR    = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE  = 'labels.json'
DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE   = 32
NUM_WORKERS  = 4
MAX_EPOCHS   = 100     # max for search
ES_PATIENCE  = 10     # early-stop patience
PCT_START    = 0.2

# hyperparameters you’ve tuned
LR           = 0.00025
WD           = 0.00013

# ─── 2) Dataset wrapper ────────────────────────────────────────
class BinaryEEGDataset(Dataset):
    def __init__(self, raw_ds, metas):
        self.raw_ds = raw_ds
        self.metas  = metas
    def __len__(self):
        return len(self.raw_ds)
    def __getitem__(self, idx):
        x, _ = self.raw_ds[idx]
        y     = self.metas[idx]['label']
        return x, y

# ─── 3) Load & prepare metadata ───────────────────────────────
with open(Path(DATA_DIR)/LABEL_FILE,'r') as f:
    all_meta = json.load(f)

train_meta       = [d for d in all_meta if d['type']=='train']
test_within_meta = [d for d in all_meta if d['type']=='test_within']
test_cross_meta  = [d for d in all_meta if d['type']=='test_cross']

# filter A vs F
class0, class1 = 'A','F'
label_map = {class0:0, class1:1}

train_meta       = [d for d in train_meta       if d['label'] in label_map]
test_within_meta = [d for d in test_within_meta if d['label'] in label_map]
test_cross_meta  = [d for d in test_cross_meta  if d['label'] in label_map]

# balance train by down-sampling
data0 = [d for d in train_meta if d['label']==class0]
data1 = [d for d in train_meta if d['label']==class1]
k     = min(len(data0), len(data1))
balanced_meta = random.sample(data0, k) + random.sample(data1, k)
random.shuffle(balanced_meta)

# convert to ints
for d in balanced_meta:       d['label'] = label_map[d['label']]
for d in test_within_meta:    d['label'] = label_map[d['label']]
for d in test_cross_meta:     d['label'] = label_map[d['label']]

print(f"\n[Balanced TRAIN] total={len(balanced_meta)} (each class={k})")

# build full balanced dataset
full_raw    = EEGDataset(DATA_DIR, balanced_meta)
full_ds     = BinaryEEGDataset(full_raw, balanced_meta)
labels_full = np.array([d['label'] for d in balanced_meta])

# ─── 4) Split train/val for epoch search ──────────────────────
idx            = np.arange(len(full_ds))
tr_idx, va_idx = train_test_split(
    idx, test_size=0.2, stratify=labels_full, random_state=SEED
)
train_loader = DataLoader(
    Subset(full_ds, tr_idx), batch_size=BATCH_SIZE,
    shuffle=True,  num_workers=NUM_WORKERS
)
val_loader   = DataLoader(
    Subset(full_ds, va_idx), batch_size=BATCH_SIZE,
    shuffle=False, num_workers=NUM_WORKERS
)

# ─── 5) Model builder & eval fn ───────────────────────────────
def build_model():
    input_len = full_ds[0][0].shape[-1]
    return EEGformer(
        in_channels=19, input_length=input_len,
        kernel_size=10, num_filters=120,
        num_heads=3, num_blocks=1,
        num_segments=5, num_classes=2
    ).to(DEVICE)

def evaluate(loader, model, criterion):
    model.eval()
    loss_sum = correct = total = 0
    with torch.no_grad():
        for X,y in loader:
            X,y    = X.to(DEVICE), y.to(DEVICE)
            logits = model(X)
            loss_sum += criterion(logits,y).item()*y.size(0)
            preds = logits.argmax(1)
            correct += (preds==y).sum().item()
            total   += y.size(0)
    return loss_sum/total, correct/total

# ─── 6) Phase 1: find best_epoch with early stopping ──────────
best_val_loss = float('inf')
best_epoch    = 0
es_count      = 0

model     = build_model()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=LR, epochs=MAX_EPOCHS,
    steps_per_epoch=len(train_loader),
    pct_start=PCT_START, anneal_strategy='cos',
    cycle_momentum=False
)
criterion = nn.CrossEntropyLoss()

print("\nSearching best epoch using validation set...")
for epoch in range(1, MAX_EPOCHS+1):
    t0 = time.time()
    # ── train ──
    model.train()
    tr_loss_sum = tr_correct = tr_total = 0
    for X,y in train_loader:
        X,y = X.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        logits = model(X)
        loss   = criterion(logits,y)
        loss.backward()
        optimizer.step()
        scheduler.step()

        tr_loss_sum  += loss.item()*y.size(0)
        preds        = logits.argmax(1)
        tr_correct  += (preds==y).sum().item()
        tr_total    += y.size(0)

    train_loss = tr_loss_sum / tr_total
    train_acc  = tr_correct / tr_total

    # ── validate ──
    val_loss, val_acc = evaluate(val_loader, model, criterion)
    elapsed = time.time() - t0

    print(
        f"Epoch {epoch:02d} | "
        f"train_loss={train_loss:.4f} | train_acc={train_acc:.4f} | "
        f"val_loss={val_loss:.4f} | val_acc={val_acc:.4f} | "
        f"time={elapsed:.1f}s"
    )

    # check for improvement
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_epoch    = epoch
        es_count      = 0
    else:
        es_count += 1
        if es_count >= ES_PATIENCE:
            print(f"\nEarly stopping at epoch {epoch} (no improvement for {ES_PATIENCE} epochs)")
            break

print(f"\n>> Best epoch by validation: {best_epoch}")

# ─── 7) Phase 2: retrain on full balanced set ─────────────────
print(f"\nRetraining from scratch on FULL set for {best_epoch} epochs...")
model2     = build_model()
optimizer2 = torch.optim.AdamW(model2.parameters(), lr=LR, weight_decay=WD)
scheduler2 = torch.optim.lr_scheduler.OneCycleLR(
    optimizer2, max_lr=LR,
    epochs=best_epoch,
    steps_per_epoch=len(DataLoader(full_ds, batch_size=BATCH_SIZE)),
    pct_start=PCT_START, anneal_strategy='cos',
    cycle_momentum=False
)
criterion2 = nn.CrossEntropyLoss()

for epoch in range(1, best_epoch+1):
    t0 = time.time()
    tr_loss_sum = tr_correct = tr_total = 0
    model2.train()
    for X,y in DataLoader(full_ds, batch_size=BATCH_SIZE,
                          shuffle=True, num_workers=NUM_WORKERS):
        X,y = X.to(DEVICE), y.to(DEVICE)
        optimizer2.zero_grad()
        logits = model2(X)
        loss   = criterion2(logits,y)
        loss.backward()
        optimizer2.step()
        scheduler2.step()

        tr_loss_sum  += loss.item()*y.size(0)
        preds        = logits.argmax(1)
        tr_correct  += (preds==y).sum().item()
        tr_total    += y.size(0)

    avg_loss = tr_loss_sum / tr_total
    acc      = tr_correct / tr_total
    print(f"Epoch {epoch:02d} | train_loss={avg_loss:.4f} acc={acc:.4f} time={time.time()-t0:.1f}s")

# ─── 8) Final Evaluation on Test Sets ─────────────────────────
def full_evaluate(metas, loader):
    model2.eval()
    loss_sum = correct = total = 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for X,y in loader:
            X,y = X.to(DEVICE), y.to(DEVICE)
            logits = model2(X)
            loss_sum += criterion2(logits,y).item()*y.size(0)
            preds    = logits.argmax(1)
            correct += (preds==y).sum().item()
            total   += y.size(0)
            all_preds.append(preds.cpu().numpy())
            all_labels.append(y.cpu().numpy())
    preds = np.concatenate(all_preds)
    labs  = np.concatenate(all_labels)
    tn,fp,fn,tp = confusion_matrix(labs,preds,labels=[0,1]).ravel()
    return {
        "loss": loss_sum/total,
        "acc": correct/total,
        "sensitivity": recall_score(labs,preds,zero_division=0),
        "specificity": tn/(tn+fp) if (tn+fp)>0 else 0.0,
        "f1": f1_score(labs,preds,zero_division=0)
    }

print("\n=== Final Evaluation on Test Sets ===")
for name, metas in [("test_within", test_within_meta), ("test_cross", test_cross_meta)]:
    loader = DataLoader(BinaryEEGDataset(EEGDataset(DATA_DIR, metas), metas),
                        batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
    res = full_evaluate(metas, loader)
    n0 = sum(1 for d in metas if d['label']==0)
    n1 = sum(1 for d in metas if d['label']==1)
    print(f"-- {name} -- total={len(metas)}  A(D)={n0}, F(TD)={n1}")
    print(f" Loss={res['loss']:.4f} Acc={res['acc']:.4f} "
          f"Sens={res['sensitivity']:.4f} Spec={res['specificity']:.4f} "
          f"F1={res['f1']:.4f}\n")

Attempting to create new mne-python configuration file:
/root/.mne/mne-python.json
Now using CUDA device 0
Enabling CUDA with 39.14 GiB available memory

[Balanced TRAIN] total=1458 (each class=729)

Searching best epoch using validation set...
Epoch 01 | train_loss=0.7511 | train_acc=0.4889 | val_loss=0.6940 | val_acc=0.4966 | time=326.5s
Epoch 02 | train_loss=0.7302 | train_acc=0.5094 | val_loss=0.6928 | val_acc=0.5274 | time=9.2s
Epoch 03 | train_loss=0.7425 | train_acc=0.5009 | val_loss=0.6932 | val_acc=0.4966 | time=8.9s
Epoch 04 | train_loss=0.7458 | train_acc=0.4923 | val_loss=0.6935 | val_acc=0.5377 | time=9.0s
Epoch 05 | train_loss=0.7196 | train_acc=0.5163 | val_loss=0.6939 | val_acc=0.4897 | time=8.9s
Epoch 06 | train_loss=0.7295 | train_acc=0.4991 | val_loss=0.6935 | val_acc=0.5240 | time=8.9s
Epoch 07 | train_loss=0.7184 | train_acc=0.5051 | val_loss=0.6931 | val_acc=0.5000 | time=9.0s
Epoch 08 | train_loss=0.7190 | train_acc=0.5172 | val_loss=0.7064 | val_acc=0.5000 | tim

## Multiclassification Task (AD vs FTD vs CN)


In [ ]:
import os
import json
import random
import time
import copy

import numpy as np
import torch
import torch.nn as nn
import wandb
import optuna

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import DataLoader, Subset, Dataset

from eeg_dataset import EEGDataset
from model_optimized_6 import EEGformer

# ─── Reproducibility ────────────────────────────────────────────
SEED = 0
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark     = False

# ─── Configuration ───────────────────────────────────────────────
DATA_DIR    = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE  = 'labels.json'
DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MAX_EPOCHS  = 150
BATCH_SIZE  = 32
NUM_WORKERS = 4
ES_PATIENCE = 15
PCT_START   = 0.2  # for OneCycleLR

# ─── Dataset wrapper ─────────────────────────────────────────────
class EEGMultiDataset(Dataset):
    def __init__(self, raw_ds, metas):
        self.raw_ds = raw_ds
        self.metas  = metas
    def __len__(self):
        return len(self.raw_ds)
    def __getitem__(self, idx):
        x, _ = self.raw_ds[idx]
        y     = self.metas[idx]['label']
        return x, y

# ─── Load metadata & split ───────────────────────────────────────
with open(os.path.join(DATA_DIR, LABEL_FILE), 'r') as f:
    all_meta = json.load(f)

train_data       = [d for d in all_meta if d['type']=='train']
test_within_meta = [d for d in all_meta if d['type']=='test_within']
test_cross_meta  = [d for d in all_meta if d['type']=='test_cross']

class_names = ['A','C','F']        # AD, CN, FTD
label_map   = {'A':0,'C':1,'F':2}

# ─── Manual Class Balancing ──────────────────────────────────────
train_data_A = [d for d in train_data if d['label']=='A']
train_data_C = [d for d in train_data if d['label']=='C']
train_data_F = [d for d in train_data if d['label']=='F']

min_samples = min(
    (len(train_data_A)+len(train_data_C)) / 2,
    (len(train_data_A)+len(train_data_F)) / 2,
    (len(train_data_C)+len(train_data_F)) / 2
)

a_index = int(min(min_samples, len(train_data_A)))
c_index = int(min(min_samples, len(train_data_C)))
f_index = int(min(min_samples, len(train_data_F)))

balanced_train_data = (
    random.sample(train_data_A, a_index) +
    random.sample(train_data_C, c_index) +
    random.sample(train_data_F, f_index)
)

print(f'Before Balancing  A={len(train_data_A)}, C={len(train_data_C)}, F={len(train_data_F)}')
print(f'After  Balancing  A={a_index}, C={c_index}, F={f_index}  Total={len(balanced_train_data)}')

for d in balanced_train_data:
    d['label'] = label_map[d['label']]

raw_ds_train  = EEGDataset(DATA_DIR, balanced_train_data)
dataset_train = EEGMultiDataset(raw_ds_train, balanced_train_data)
labels_train  = np.array([d['label'] for d in balanced_train_data])

# ─── Utility: count labels ───────────────────────────────────────
def count_labels(meta_list):
    counts = {0:0, 1:0, 2:0}
    for d in meta_list:
        counts[d['label']] += 1
    return counts

# ─── Optuna objective ────────────────────────────────────────────
def objective(trial):
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    wd = trial.suggest_float('wd', 1e-5, 1e-2, log=True)

    print(f"\n--- Trial {trial.number} hyperparams --> lr={lr:.2e}, wd={wd:.2e}")

    wandb.init(
        project='eeg-multiclass-AD_CN_FTD',
        name=f"trial{trial.number}",
        config={'lr':lr,'wd':wd,'pct_start':PCT_START},
        reinit=True
    )

    idx = np.arange(len(dataset_train))
    tr_idx, va_idx = train_test_split(
        idx, test_size=0.2, stratify=labels_train, random_state=SEED
    )

    tr_counts = count_labels([balanced_train_data[i] for i in tr_idx])
    va_counts = count_labels([balanced_train_data[i] for i in va_idx])
    print(f"[Trial {trial.number}] TRAIN n={len(tr_idx)} A={tr_counts[0]}, C={tr_counts[1]}, F={tr_counts[2]} | "
          f"VAL n={len(va_idx)} A={va_counts[0]}, C={va_counts[1]}, F={va_counts[2]}")

    train_loader = DataLoader(Subset(dataset_train, tr_idx),
                              batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
    val_loader   = DataLoader(Subset(dataset_train, va_idx),
                              batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

    input_len = dataset_train[0][0].shape[-1]
    model = EEGformer(
        in_channels=19, input_length=input_len,
        kernel_size=10, num_filters=120,
        num_heads=3, num_blocks=1,
        num_segments=5, num_classes=3
    ).to(DEVICE)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=lr, epochs=MAX_EPOCHS,
        steps_per_epoch=len(train_loader), pct_start=PCT_START,
        anneal_strategy='cos', cycle_momentum=False
    )
    criterion = nn.CrossEntropyLoss()

    best_val_loss = float('inf')
    best_state    = None
    es_count      = 0

    for epoch in range(1, MAX_EPOCHS+1):
        t0 = time.time()
        model.train()
        train_loss = train_correct = train_total = 0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()
            scheduler.step()

            train_loss   += loss.item()
            preds        = logits.argmax(1)
            train_correct+= (preds==y).sum().item()
            train_total  += y.size(0)

        train_loss /= len(train_loader)
        train_acc   = train_correct / train_total

        model.eval()
        val_loss = 0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                val_loss += criterion(logits, y).item()
                all_preds.append(logits.argmax(1).cpu().numpy())
                all_labels.append(y.cpu().numpy())

        val_loss /= len(val_loader)
        preds = np.concatenate(all_preds)
        labs  = np.concatenate(all_labels)

        val_acc = accuracy_score(labs, preds)
        prec    = precision_score(labs, preds, average='macro', zero_division=0)
        rec     = recall_score(labs, preds, average='macro', zero_division=0)
        f1      = f1_score(labs, preds, average='macro', zero_division=0)
        elapsed = time.time() - t0

        print(f"Epoch {epoch:03d} | train_loss={train_loss:.4f} acc={train_acc:.4f} | "
              f"val_loss={val_loss:.4f} acc={val_acc:.4f} | prec={prec:.4f} rec={rec:.4f} f1={f1:.4f} | "
              f"time={elapsed:.1f}s")

        wandb.log({
            'epoch':      epoch,
            'train_loss': train_loss,
            'train_acc':  train_acc,
            'val_loss':   val_loss,
            'val_acc':    val_acc,
            'precision':  prec,
            'recall':     rec,
            'f1_score':   f1
        })

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_state    = copy.deepcopy(model.state_dict())
            es_count      = 0
            os.makedirs('ckpts', exist_ok=True)
            ckpt_path = f'ckpts/trial{trial.number}_best.pth'
            torch.save(best_state, ckpt_path)
            trial.set_user_attr('ckpt_path', ckpt_path)
        else:
            es_count += 1
            if es_count >= ES_PATIENCE:
                break

    wandb.finish()
    del model, optimizer, scheduler, train_loader, val_loader
    torch.cuda.empty_cache()

    return best_val_loss, val_acc

# ─── Run Optuna & Summarize ─────────────────────────────────────
if __name__ == "__main__":
    study = optuna.create_study(
        directions=["minimize","maximize"],
        study_name="eeg_multiobj_AD_CN_FTD"
    )
    study.optimize(objective, n_trials=10)

    print("\nAll trials: hyperparams → (val_loss, val_acc)")
    for t in study.trials:
        print(f"  Trial {t.number}: lr={t.params['lr']:.2e}, wd={t.params['wd']:.2e} → ({t.values[0]:.4f}, {t.values[1]:.4f})")

    best_trial = max(study.best_trials, key=lambda t: t.values[1])
    bv_loss, bv_acc = best_trial.values
    ckpt_path       = best_trial.user_attrs["ckpt_path"]

    print(f"\n=== Selected Trial #{best_trial.number} ===")
    print(f" val_loss={bv_loss:.4f}, val_acc={bv_acc:.4f}")
    print(f" Best hyperparameters: {best_trial.params}")

    # ─── Load Best Model & Evaluate on Test Sets ────────────────
    input_len = dataset_train[0][0].shape[-1]
    model = EEGformer(
        in_channels=19, input_length=input_len,
        kernel_size=10, num_filters=120,
        num_heads=3, num_blocks=1,
        num_segments=5, num_classes=3
    ).to(DEVICE)
    model.load_state_dict(torch.load(ckpt_path, map_location=DEVICE))
    model.eval()

    def evaluate(model, metas):
        metas_copy = []
        for d in metas:
            new_d = d.copy()
            if isinstance(new_d['label'], str):
                new_d['label'] = label_map[new_d['label']]
            metas_copy.append(new_d)

        ds     = EEGMultiDataset(EEGDataset(DATA_DIR, metas_copy), metas_copy)
        loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

        all_preds, all_labels = [], []
        with torch.no_grad():
            for X, y in loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                all_preds.append(logits.argmax(1).cpu().numpy())
                all_labels.append(y.cpu().numpy())

        preds = np.concatenate(all_preds)
        labs  = np.concatenate(all_labels)
        metrics = {
            "acc":       accuracy_score(labs, preds),
            "precision": precision_score(labs, preds, average='macro', zero_division=0),
            "recall":    recall_score(labs, preds, average='macro', zero_division=0),
            "f1":        f1_score(labs, preds, average='macro', zero_division=0),
        }
        counts = count_labels(metas_copy)
        return metrics, counts

    print("\n=== Final Evaluation on Test Sets ===")
    for name, metas in [("test_within", test_within_meta), ("test_cross", test_cross_meta)]:
        res, counts = evaluate(model, metas)
        print(f"-- {name} -- total={len(metas)}  A={counts[0]}, C={counts[1]}, F={counts[2]}")
        print(f" Acc={res['acc']:.4f}  Prec={res['precision']:.4f}  Rec={res['recall']:.4f}  F1={res['f1']:.4f}\n")

Now using CUDA device 0
Enabling CUDA with 39.14 GiB available memory


[I 2025-05-10 09:30:55,938] A new study created in memory with name: eeg_multiobj_AD_CN_FTD_2


Before Balancing  A=1388, C=1102, F=729
After  Balancing  A=915, C=915, F=729  Total=2559

--- Trial 0 hyperparams --> lr=2.79e-05, wd=1.73e-05


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


[Trial 0] TRAIN n=2047 A=732, C=732, F=583 | VAL n=512 A=183, C=183, F=146
Epoch 001 | train_loss=1.1654 acc=0.3298 | val_loss=1.0988 acc=0.3223 | prec=0.5475 rec=0.3356 f1=0.2636 | time=15.1s
Epoch 002 | train_loss=1.1677 acc=0.3214 | val_loss=1.0977 acc=0.3359 | prec=0.3300 rec=0.3216 f1=0.2778 | time=14.9s
Epoch 003 | train_loss=1.1708 acc=0.3317 | val_loss=1.0979 acc=0.3750 | prec=0.3872 rec=0.3862 f1=0.3714 | time=14.9s
Epoch 004 | train_loss=1.1718 acc=0.3258 | val_loss=1.0981 acc=0.3477 | prec=0.3828 rec=0.3616 f1=0.3080 | time=14.6s
Epoch 005 | train_loss=1.1818 acc=0.3004 | val_loss=1.0997 acc=0.2852 | prec=0.1950 rec=0.3319 f1=0.1575 | time=14.5s
Epoch 006 | train_loss=1.1516 acc=0.3459 | val_loss=1.0978 acc=0.3672 | prec=0.3587 rec=0.3558 f1=0.3473 | time=14.6s
Epoch 007 | train_loss=1.1506 acc=0.3405 | val_loss=1.0984 acc=0.3555 | prec=0.2875 rec=0.3555 f1=0.2830 | time=14.4s
Epoch 008 | train_loss=1.1389 acc=0.3439 | val_loss=1.0980 acc=0.3223 | prec=0.2964 rec=0.3098 f1=0

...

[Trial 9] TRAIN n=2047 A=732, C=732, F=583 | VAL n=512 A=183, C=183, F=146
Epoch 001 | train_loss=1.1702 acc=0.3806 | val_loss=1.1143 acc=0.3574 | prec=0.1191 rec=0.3333 f1=0.1755 | time=14.6s
Epoch 002 | train_loss=1.1751 acc=0.3478 | val_loss=1.1044 acc=0.3574 | prec=0.1191 rec=0.3333 f1=0.1755 | time=14.5s
Epoch 003 | train_loss=1.1442 acc=0.3537 | val_loss=1.1021 acc=0.3574 | prec=0.1191 rec=0.3333 f1=0.1755 | time=14.6s
Epoch 004 | train_loss=1.1389 acc=0.3498 | val_loss=1.1005 acc=0.3574 | prec=0.1191 rec=0.3333 f1=0.1755 | time=14.5s
Epoch 005 | train_loss=1.1258 acc=0.3488 | val_loss=1.0949 acc=0.3574 | prec=0.1191 rec=0.3333 f1=0.1755 | time=14.6s
Epoch 006 | train_loss=1.1381 acc=0.3381 | val_loss=1.0952 acc=0.3574 | prec=0.1191 rec=0.3333 f1=0.1755 | time=14.6s
Epoch 007 | train_loss=1.1158 acc=0.3688 | val_loss=1.0944 acc=0.3555 | prec=0.1192 rec=0.3315 f1=0.1753 | time=14.9s
Epoch 008 | train_loss=1.1306 acc=0.3395 | val_loss=1.0958 acc=0.3340 | prec=0.1989 rec=0.3115 f1=0

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇███
f1_score,▁▁▁▁▁▁▁▂▂▂▃▃▆▆▆▅▆▇▇▇▇▇▇▇▇▇██▇███▇███████
precision,▁▁▁▁▁▃▃▂▃▃▃▄▆▆▆▇█▇█▇███▇████████████████
recall,▁▁▁▁▁▁▂▁▁▂▂▃▃▄▅▅▅▅▆▆▇▆▇▇▇▇▇█████▇███████
train_acc,▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▂▄▄▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇█▇█████
train_loss,█████▇▇█▇▇▇▇▇▆▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁
val_acc,▁▁▁▁▁▂▂▂▂▃▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇█▇██████▇██████
val_loss,█████████████▆▅▄▅▅▃▄▃▃▃▃▂▁▂▂▂▁▂▂▁▁▂▂▂▂▃▂
epoch,82
f1_score,0.71112
precision,0.7369


[I 2025-05-10 12:08:28,889] Trial 9 finished with values: [0.6978908181190491, 0.716796875] and parameters: {'lr': 3.7214819629441235e-05, 'wd': 0.00019609787248213046}.



All trials: hyperparams → (val_loss, val_acc)
  Trial 0: lr=2.79e-05, wd=1.73e-05 → (0.6848, 0.7168)
  Trial 1: lr=6.72e-03, wd=5.21e-04 → (0.7927, 0.6523)
  Trial 2: lr=1.34e-04, wd=4.02e-03 → (0.6832, 0.7324)
  Trial 3: lr=2.90e-05, wd=2.05e-05 → (0.6576, 0.7422)
  Trial 4: lr=2.08e-05, wd=6.91e-05 → (0.6542, 0.7305)
  Trial 5: lr=2.93e-04, wd=2.20e-04 → (0.6777, 0.7148)
  Trial 6: lr=3.23e-03, wd=3.78e-05 → (0.7543, 0.6855)
  Trial 7: lr=9.51e-05, wd=2.16e-04 → (0.6466, 0.7266)
  Trial 8: lr=1.11e-04, wd=2.54e-03 → (0.6277, 0.7539)
  Trial 9: lr=3.72e-05, wd=1.96e-04 → (0.6979, 0.7168)

=== Selected Trial #8 ===
 val_loss=0.6277, val_acc=0.7539
 Best hyperparameters: {'lr': 0.00011111361926742459, 'wd': 0.002544246881680688}

=== Final Evaluation on Test Sets ===


KeyError: 2

#### AD vs CN vs FTD (Best Trial test)

In [ ]:
import os
import json
import random
import time
import copy

import numpy as np
import torch
import torch.nn as nn
import wandb
import optuna

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import DataLoader, Subset, Dataset

from eeg_dataset import EEGDataset
from model_optimized_6 import EEGformer

# ─── Reproducibility ────────────────────────────────────────────
SEED = 0
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark     = False

# ─── Configuration ───────────────────────────────────────────────
DATA_DIR    = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE  = 'labels.json'
DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MAX_EPOCHS  = 150
BATCH_SIZE  = 32
NUM_WORKERS = 4
ES_PATIENCE = 15
PCT_START   = 0.2  # for OneCycleLR

# ─── Fixed best hyperparameters ─────────────────────────────────
BEST_LR = 2.90e-05
BEST_WD = 2.05e-05

# ─── Dataset wrapper ─────────────────────────────────────────────
class EEGMultiDataset(Dataset):
    def __init__(self, raw_ds, metas):
        self.raw_ds = raw_ds
        self.metas  = metas
    def __len__(self):
        return len(self.raw_ds)
    def __getitem__(self, idx):
        x, _ = self.raw_ds[idx]
        y     = self.metas[idx]['label']
        return x, y

# ─── Load metadata & split ───────────────────────────────────────
with open(os.path.join(DATA_DIR, LABEL_FILE), 'r') as f:
    all_meta = json.load(f)

train_data       = [d for d in all_meta if d['type']=='train']
test_within_meta = [d for d in all_meta if d['type']=='test_within']
test_cross_meta  = [d for d in all_meta if d['type']=='test_cross']

label_map = {'A':0,'C':1,'F':2}

# ─── Manual Class Balancing ──────────────────────────────────────
train_data_A = [d for d in train_data if d['label']=='A']
train_data_C = [d for d in train_data if d['label']=='C']
train_data_F = [d for d in train_data if d['label']=='F']

min_samples = min(
    (len(train_data_A)+len(train_data_C)) / 2,
    (len(train_data_A)+len(train_data_F)) / 2,
    (len(train_data_C)+len(train_data_F)) / 2
)

a_index = int(min(min_samples, len(train_data_A)))
c_index = int(min(min_samples, len(train_data_C)))
f_index = int(min(min_samples, len(train_data_F)))

balanced_train_data = (
    random.sample(train_data_A, a_index) +
    random.sample(train_data_C, c_index) +
    random.sample(train_data_F, f_index)
)

print(f'Before Balancing  A={len(train_data_A)}, C={len(train_data_C)}, F={len(train_data_F)}')
print(f'After  Balancing  A={a_index}, C={c_index}, F={f_index}  Total={len(balanced_train_data)}')

for d in balanced_train_data:
    d['label'] = label_map[d['label']]

raw_ds_train  = EEGDataset(DATA_DIR, balanced_train_data)
dataset_train = EEGMultiDataset(raw_ds_train, balanced_train_data)
labels_train  = np.array([d['label'] for d in balanced_train_data])

# ─── Utility: count labels ───────────────────────────────────────
def count_labels(meta_list):
    counts = {0:0, 1:0, 2:0}
    for d in meta_list:
        counts[d['label']] += 1
    return counts

# ─── Optuna objective (fixed hyperparams) ───────────────────────
def objective(trial):
    lr = BEST_LR
    wd = BEST_WD

    print(f"\n--- Trial {trial.number} hyperparams (fixed) --> lr={lr:.2e}, wd={wd:.2e}")

    wandb.init(
        project='eeg-multiclass-AD_CN_FTD_fixed',
        name=f"trial{trial.number}",
        config={'lr':lr,'wd':wd,'pct_start':PCT_START},
        reinit=True
    )

    idx = np.arange(len(dataset_train))
    tr_idx, va_idx = train_test_split(
        idx, test_size=0.2, stratify=labels_train, random_state=SEED
    )

    tr_counts = count_labels([balanced_train_data[i] for i in tr_idx])
    va_counts = count_labels([balanced_train_data[i] for i in va_idx])
    print(f"[Trial {trial.number}] TRAIN n={len(tr_idx)} A={tr_counts[0]}, C={tr_counts[1]}, F={tr_counts[2]} | "
          f"VAL n={len(va_idx)} A={va_counts[0]}, C={va_counts[1]}, F={va_counts[2]}")

    train_loader = DataLoader(Subset(dataset_train, tr_idx),
                              batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
    val_loader   = DataLoader(Subset(dataset_train, va_idx),
                              batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

    input_len = dataset_train[0][0].shape[-1]
    model = EEGformer(
        in_channels=19, input_length=input_len,
        kernel_size=10, num_filters=120,
        num_heads=3, num_blocks=1,
        num_segments=5, num_classes=3
    ).to(DEVICE)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=lr, epochs=MAX_EPOCHS,
        steps_per_epoch=len(train_loader), pct_start=PCT_START,
        anneal_strategy='cos', cycle_momentum=False
    )
    criterion = nn.CrossEntropyLoss()

    best_val_loss = float('inf')
    best_state    = None
    es_count      = 0

    for epoch in range(1, MAX_EPOCHS+1):
        t0 = time.time()
        model.train()
        train_loss = train_correct = train_total = 0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()
            scheduler.step()

            train_loss   += loss.item()
            preds        = logits.argmax(1)
            train_correct+= (preds==y).sum().item()
            train_total  += y.size(0)

        train_loss /= len(train_loader)
        train_acc   = train_correct / train_total

        model.eval()
        val_loss = 0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                val_loss += criterion(logits, y).item()
                all_preds.append(logits.argmax(1).cpu().numpy())
                all_labels.append(y.cpu().numpy())

        val_loss /= len(val_loader)
        preds = np.concatenate(all_preds)
        labs  = np.concatenate(all_labels)

        val_acc = accuracy_score(labs, preds)
        prec    = precision_score(labs, preds, average='macro', zero_division=0)
        rec     = recall_score(labs, preds, average='macro', zero_division=0)
        f1      = f1_score(labs, preds, average='macro', zero_division=0)
        elapsed = time.time() - t0

        print(f"Epoch {epoch:03d} | train_loss={train_loss:.4f} acc={train_acc:.4f} | "
              f"val_loss={val_loss:.4f} acc={val_acc:.4f} | prec={prec:.4f} rec={rec:.4f} f1={f1:.4f} | "
              f"time={elapsed:.1f}s")

        wandb.log({
            'epoch':      epoch,
            'train_loss': train_loss,
            'train_acc':  train_acc,
            'val_loss':   val_loss,
            'val_acc':    val_acc,
            'precision':  prec,
            'recall':     rec,
            'f1_score':   f1
        })

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_state    = copy.deepcopy(model.state_dict())
            es_count      = 0
            os.makedirs('ckpts', exist_ok=True)
            ckpt_path = f'ckpts/trial{trial.number}_best.pth'
            torch.save(best_state, ckpt_path)
            trial.set_user_attr('ckpt_path', ckpt_path)
        else:
            es_count += 1
            if es_count >= ES_PATIENCE:
                break

    wandb.finish()
    del model, optimizer, scheduler, train_loader, val_loader
    torch.cuda.empty_cache()

    # return best checkpoint loss & acc
    return best_val_loss, val_acc

# ─── Run Optuna & Evaluate Best ─────────────────────────────────
if __name__ == "__main__":
    study = optuna.create_study(
        directions=["minimize","maximize"],
        study_name="eeg_multiobj_AD_CN_FTD_fixed"
    )
    study.optimize(objective, n_trials=1)

    # best trial is trial 0
    t = study.trials[0]
    ckpt_path = t.user_attrs["ckpt_path"]
    print(f"\nFixed-hparam run → lr={BEST_LR:.2e}, wd={BEST_WD:.2e} → "
          f"(val_loss={t.values[0]:.4f}, val_acc={t.values[1]:.4f})")

    # load best checkpoint
    input_len = dataset_train[0][0].shape[-1]
    model_val = EEGformer(
        in_channels=19, input_length=input_len,
        kernel_size=10, num_filters=120,
        num_heads=3, num_blocks=1,
        num_segments=5, num_classes=3
    ).to(DEVICE)
    model_val.load_state_dict(torch.load(ckpt_path, map_location=DEVICE))
    model_val.eval()

    def evaluate(model, metas):
        metas_copy = []
        for d in metas:
            new_d = d.copy()
            if isinstance(new_d['label'], str):
                new_d['label'] = label_map[new_d['label']]
            metas_copy.append(new_d)

        ds     = EEGMultiDataset(EEGDataset(DATA_DIR, metas_copy), metas_copy)
        loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

        all_preds, all_labels = [], []
        with torch.no_grad():
            for X, y in loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                all_preds.append(logits.argmax(1).cpu().numpy())
                all_labels.append(y.cpu().numpy())

        preds = np.concatenate(all_preds)
        labs  = np.concatenate(all_labels)
        metrics = {
            "acc":       accuracy_score(labs, preds),
            "precision": precision_score(labs, preds, average='macro', zero_division=0),
            "recall":    recall_score(labs, preds, average='macro', zero_division=0),
            "f1":        f1_score(labs, preds, average='macro', zero_division=0),
        }
        counts = count_labels(metas_copy)
        return metrics, counts

    # evaluate model trained with validation
    print("\n=== Test performance (model with validation) ===")
    m_val_within, c_val_within = evaluate(model_val, test_within_meta)
    print(f" test_within: total={len(test_within_meta)}  A={c_val_within[0]}, C={c_val_within[1]}, F={c_val_within[2]}")
    print(f"    Acc={m_val_within['acc']:.4f}  Prec={m_val_within['precision']:.4f}  "
          f"Rec={m_val_within['recall']:.4f}  F1={m_val_within['f1']:.4f}")

    m_val_cross, c_val_cross = evaluate(model_val, test_cross_meta)
    print(f" test_cross:  total={len(test_cross_meta)}  A={c_val_cross[0]}, C={c_val_cross[1]}, F={c_val_cross[2]}")
    print(f"    Acc={m_val_cross['acc']:.4f}  Prec={m_val_cross['precision']:.4f}  "
          f"Rec={m_val_cross['recall']:.4f}  F1={m_val_cross['f1']:.4f}")

    # ─── Full train on all training data, no validation ─────────────
    print("\n=== Full-train performance (no validation) ===")
    full_loader = DataLoader(dataset_train, batch_size=BATCH_SIZE,
                             shuffle=True, num_workers=NUM_WORKERS)

    model_full = EEGformer(
        in_channels=19, input_length=input_len,
        kernel_size=10, num_filters=120,
        num_heads=3, num_blocks=1,
        num_segments=5, num_classes=3
    ).to(DEVICE)

    optimizer = torch.optim.AdamW(model_full.parameters(),
                                  lr=BEST_LR, weight_decay=BEST_WD)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=BEST_LR, epochs=MAX_EPOCHS,
        steps_per_epoch=len(full_loader), pct_start=PCT_START,
        anneal_strategy='cos', cycle_momentum=False
    )
    criterion = nn.CrossEntropyLoss()

    # train for MAX_EPOCHS on full data
    for epoch in range(1, MAX_EPOCHS+1):
        model_full.train()
        for X, y in full_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model_full(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()
            scheduler.step()

    model_full.eval()
    m_full_within, c_full_within = evaluate(model_full, test_within_meta)
    print(f" test_within: total={len(test_within_meta)}  A={c_full_within[0]}, C={c_full_within[1]}, F={c_full_within[2]}")
    print(f"    Acc={m_full_within['acc']:.4f}  Prec={m_full_within['precision']:.4f}  "
          f"Rec={m_full_within['recall']:.4f}  F1={m_full_within['f1']:.4f}")

    m_full_cross, c_full_cross = evaluate(model_full, test_cross_meta)
    print(f" test_cross:  total={len(test_cross_meta)}  A={c_full_cross[0]}, C={c_full_cross[1]}, F={c_full_cross[2]}")
    print(f"    Acc={m_full_cross['acc']:.4f}  Prec={m_full_cross['precision']:.4f}  "
          f"Rec={m_full_cross['recall']:.4f}  F1={m_full_cross['f1']:.4f}")

Attempting to create new mne-python configuration file:
/root/.mne/mne-python.json
Now using CUDA device 0
Enabling CUDA with 39.14 GiB available memory


[I 2025-05-10 21:10:08,122] A new study created in memory with name: eeg_multiobj_AD_CN_FTD_fixed


Before Balancing  A=1388, C=1102, F=729
After  Balancing  A=915, C=915, F=729  Total=2559

--- Trial 0 hyperparams (fixed) --> lr=2.90e-05, wd=2.05e-05


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


[Trial 0] TRAIN n=2047 A=732, C=732, F=583 | VAL n=512 A=183, C=183, F=146
Epoch 001 | train_loss=1.1655 acc=0.3283 | val_loss=1.0987 acc=0.3242 | prec=0.5474 rec=0.3347 f1=0.2640 | time=492.3s
Epoch 002 | train_loss=1.1677 acc=0.3205 | val_loss=1.0976 acc=0.3379 | prec=0.3361 rec=0.3239 f1=0.2838 | time=15.4s
Epoch 003 | train_loss=1.1707 acc=0.3327 | val_loss=1.0980 acc=0.3613 | prec=0.3829 rec=0.3767 f1=0.3485 | time=15.4s
Epoch 004 | train_loss=1.1717 acc=0.3258 | val_loss=1.0986 acc=0.3086 | prec=0.3722 rec=0.3432 f1=0.2414 | time=15.1s
Epoch 005 | train_loss=1.1814 acc=0.3009 | val_loss=1.1003 acc=0.2871 | prec=0.2615 rec=0.3342 f1=0.1578 | time=14.7s
Epoch 006 | train_loss=1.1509 acc=0.3468 | val_loss=1.0978 acc=0.3438 | prec=0.2870 rec=0.3234 f1=0.2092 | time=14.8s
Epoch 007 | train_loss=1.1518 acc=0.3385 | val_loss=1.1008 acc=0.2773 | prec=0.1676 rec=0.3224 f1=0.1575 | time=14.5s
Epoch 008 | train_loss=1.1391 acc=0.3454 | val_loss=1.1003 acc=0.2832 | prec=0.1864 rec=0.2877 f1=

epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1_score,▃▃▁▂▂▃▃▁▁▄▅▅▆▅▆▇▆▇▇▇▇▇▇▇██▇██▇██████████
precision,▃▂▂▁▂▄▃▃▃▄▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇██▇████████████
recall,▂▁▁▁▁▁▁▂▁▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇████▇██▇████████
train_acc,▁▁▁▁▂▁▁▁▁▂▂▂▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇█▇▇███████
train_loss,█████▇▇▇▇▇▅▄▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
val_acc,▂▂▁▂▁▂▂▂▂▃▂▅▅▅▅▆▆▆▆▇▇▇▇▇▇█▇█▇███████████
val_loss,█████████▆▆▅▅▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▃▂▁▁▂▃▁▂▂▁▁
epoch,83
f1_score,0.70227
precision,0.70492


[I 2025-05-10 21:38:45,203] Trial 0 finished with values: [0.6993866618722677, 0.708984375] and parameters: {}.



Fixed-hparam run → lr=2.90e-05, wd=2.05e-05 → (val_loss=0.6994, val_acc=0.7090)

=== Test performance (model with validation) ===
 test_within: total=344  A=146, C=126, F=72
    Acc=0.7703  Prec=0.7527  Rec=0.7703  F1=0.7557
 test_cross:  total=873  A=319, C=307, F=247
    Acc=0.5178  Prec=0.5082  Rec=0.5086  F1=0.5012

=== Full-train performance (no validation) ===
 test_within: total=344  A=146, C=126, F=72
    Acc=0.7820  Prec=0.7672  Rec=0.7475  F1=0.7542
 test_cross:  total=873  A=319, C=307, F=247
    Acc=0.5441  Prec=0.5787  Rec=0.5216  F1=0.5033


#### Classification Performance of Leave-One-Subject-Out (LOSO) validation for the AD-CN problem

In [ ]:
import os
import json
import copy
import random
import time
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import GroupKFold, GroupShuffleSplit
from sklearn.metrics import confusion_matrix

from eeg_dataset import EEGDataset
from model_optimized_6 import EEGformer

# ─── 재현성 설정 ─────────────────────────────────────────────────────
SEED = 0
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark     = False

def worker_init_fn(worker_id):
    np.random.seed(SEED + worker_id)
    random.seed(SEED + worker_id)

# ─── 설정 ─────────────────────────────────────────────────────────
DATA_DIR    = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE  = 'labels.json'
DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE   = 32
MAX_EPOCHS   = 150   # 최대 epoch
LR           = 2.2e-05
WD           = 2.3e-06
PCT_START    = 0.2
ES_PATIENCE  = 15    # validation_loss 개선 없을 때 조기종료
VAL_SIZE     = 0.2   # train_meta 중 validation 비율

# ─── Dataset 래퍼 클래스 ──────────────────────────────────────────
class BinaryEEGDataset(Dataset):
    def __init__(self, raw_ds, meta):
        self.raw_ds = raw_ds
        self.meta   = meta
    def __len__(self):
        return len(self.raw_ds)
    def __getitem__(self, idx):
        x, _ = self.raw_ds[idx]
        y    = self.meta[idx]['label']
        return x, y

# ─── subject_id 추출 함수 ─────────────────────────────────────────
def extract_subject_id(m):
    fn = os.path.basename(m['file_name'])
    return fn.split('_')[0]

# ─── 메타데이터 로드 및 필터링 ─────────────────────────────────────
all_meta = json.load(open(os.path.join(DATA_DIR, LABEL_FILE), 'r'))
all_meta = [m for m in all_meta if m['label'] in ('A','C')]
groups   = [extract_subject_id(m) for m in all_meta]
subjects = sorted(set(groups))

# ─── LOSO 검증 셋업 ──────────────────────────────────────────────
gkf = GroupKFold(n_splits=len(subjects))
total_tp = total_tn = total_fp = total_fn = 0

# ─── Fold별 학습/평가 루프 ───────────────────────────────────────
for fold, (train_idx, test_idx) in enumerate(gkf.split(all_meta, groups=groups), 1):
    print(f"\n>>> Fold {fold}/{len(subjects)}  (leave out {subjects[fold-1]})")

    # 메타 분할 및 라벨 매핑
    train_meta = [copy.deepcopy(all_meta[i]) for i in train_idx]
    test_meta  = [copy.deepcopy(all_meta[i]) for i in test_idx]
    for m in train_meta: m['label'] = 0 if m['label']=='A' else 1
    for m in test_meta:  m['label'] = 0 if m['label']=='A' else 1

    # train_meta 안에서 validation split by subject
    groups_train = [extract_subject_id(m) for m in train_meta]
    gss = GroupShuffleSplit(n_splits=1, test_size=VAL_SIZE, random_state=SEED)
    tr_idx2, val_idx = next(gss.split(train_meta, groups=groups_train))
    val_meta   = [train_meta[i] for i in val_idx]
    train_meta = [train_meta[i] for i in tr_idx2]

    # DataLoader 준비
    raw_train = EEGDataset(DATA_DIR, train_meta)
    raw_val   = EEGDataset(DATA_DIR, val_meta)
    raw_test  = EEGDataset(DATA_DIR, test_meta)
    train_ds  = BinaryEEGDataset(raw_train, train_meta)
    val_ds    = BinaryEEGDataset(raw_val,   val_meta)
    test_ds   = BinaryEEGDataset(raw_test,  test_meta)

    train_loader = DataLoader(
        train_ds, batch_size=BATCH_SIZE, shuffle=True,
        num_workers=4, worker_init_fn=worker_init_fn
    )
    val_loader = DataLoader(
        val_ds, batch_size=BATCH_SIZE, shuffle=False,
        num_workers=4, worker_init_fn=worker_init_fn
    )
    test_loader = DataLoader(
        test_ds, batch_size=BATCH_SIZE, shuffle=False,
        num_workers=4, worker_init_fn=worker_init_fn
    )

    # 모델/옵티마이저/스케줄러/손실 함수
    input_len = train_ds[0][0].shape[-1]
    model = EEGformer(
        in_channels=19, input_length=input_len,
        kernel_size=10, num_filters=120,
        num_heads=3, num_blocks=1,
        num_segments=5, num_classes=2
    ).to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=LR, epochs=MAX_EPOCHS,
        steps_per_epoch=len(train_loader), pct_start=PCT_START,
        anneal_strategy='cos', cycle_momentum=False
    )
    criterion = nn.CrossEntropyLoss()

    # Early stopping 및 체크포인트 변수
    best_val_loss = float('inf')
    best_state    = None
    es_count      = 0

    # ─── 학습 + 검증 루프 ──────────────────────────────────────────
    for epoch in range(1, MAX_EPOCHS+1):
        t0 = time.time()
        # Train
        model.train()
        train_loss_sum = 0.0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()
            scheduler.step()
            # dynamic weight decay
            cur_lr = optimizer.param_groups[0]['lr']
            for g in optimizer.param_groups:
                g['weight_decay'] = WD * (cur_lr / LR)
            train_loss_sum += loss.item()
        avg_train_loss = train_loss_sum / len(train_loader)

        # Validate
        model.eval()
        val_loss_sum = val_correct = val_total = 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                val_loss_sum += criterion(logits, y).item()
                preds = logits.argmax(1)
                val_correct += (preds == y).sum().item()
                val_total += y.size(0)
        avg_val_loss = val_loss_sum / len(val_loader)
        avg_val_acc  = val_correct / val_total if val_total>0 else 0.0
        elapsed = time.time() - t0

        print(f"  Fold {fold} Epoch {epoch}/{MAX_EPOCHS}  "
              f"Train Loss={avg_train_loss:.4f}  "
              f"Val Loss={avg_val_loss:.4f}  "
              f"Val Acc={avg_val_acc:.4f}  "
              f"Time={elapsed:.1f}s")

        # Early stopping & checkpoint on val_loss
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_state    = copy.deepcopy(model.state_dict())
            es_count      = 0
        else:
            es_count += 1
            if es_count >= ES_PATIENCE:
                print(f"  → Early stopping at epoch {epoch} (no val_loss improvement)")
                break

    # 최적의 validation 시점 모델 로드
    if best_state is not None:
        model.load_state_dict(best_state)

    # ─── 최종 테스트 & 혼동행렬 누적 ─────────────────────────────────
    all_preds, all_labels = [], []
    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            preds = model(X).argmax(1)
            all_preds.append(preds.cpu().numpy())
            all_labels.append(y.cpu().numpy())
    preds = np.concatenate(all_preds)
    labs  = np.concatenate(all_labels)
    tn, fp, fn, tp = confusion_matrix(labs, preds, labels=[0,1]).ravel()
    total_tp += tp
    total_tn += tn
    total_fp += fp
    total_fn += fn

    acc = (tp + tn) / (tp + tn + fp + fn)
    print(f"  Fold {fold} Final ACC = {acc:.4f}   (TP={tp}  TN={tn}  FP={fp}  FN={fn})")

    # 메모리 해제
    del model, optimizer, scheduler
    torch.cuda.empty_cache()

# ─── 전체 LOSO 지표 계산 ─────────────────────────────────────────
ACC  = (total_tp + total_tn) / (total_tp + total_tn + total_fp + total_fn)
SENS = total_tp / (total_tp + total_fn)
SPEC = total_tn / (total_tn + total_fp)
F1   = 2 * total_tp / (2*total_tp + total_fp + total_fn)

print("\n=== LOSO 평가 결과 ===")
print(f"Accuracy   = {ACC:.4%}")
print(f"Sensitivity= {SENS:.4%}")
print(f"Specificity= {SPEC:.4%}")
print(f"F1-score   = {F1:.4%}")

Now using CUDA device 0
Enabling CUDA with 39.14 GiB available memory

>>> Fold 1/65  (leave out sub-001)
  Fold 1 Epoch 1/150  Train Loss=0.7415  Val Loss=0.6998  Val Acc=0.4133  Time=19.5s
  Fold 1 Epoch 2/150  Train Loss=0.7539  Val Loss=0.7006  Val Acc=0.4103  Time=18.6s
  Fold 1 Epoch 3/150  Train Loss=0.7284  Val Loss=0.6975  Val Acc=0.4042  Time=18.8s
  Fold 1 Epoch 4/150  Train Loss=0.7352  Val Loss=0.6868  Val Acc=0.5882  Time=18.7s
  Fold 1 Epoch 5/150  Train Loss=0.7336  Val Loss=0.6915  Val Acc=0.5747  Time=18.8s
  Fold 1 Epoch 6/150  Train Loss=0.7357  Val Loss=0.6831  Val Acc=0.5897  Time=18.6s
  Fold 1 Epoch 7/150  Train Loss=0.7300  Val Loss=0.6843  Val Acc=0.5897  Time=18.7s
  Fold 1 Epoch 8/150  Train Loss=0.7197  Val Loss=0.6828  Val Acc=0.5897  Time=18.7s
  Fold 1 Epoch 9/150  Train Loss=0.7256  Val Loss=0.6858  Val Acc=0.5897  Time=18.8s
  Fold 1 Epoch 10/150  Train Loss=0.7307  Val Loss=0.6821  Val Acc=0.5897  Time=18.6s
  Fold 1 Epoch 11/150  Train Loss=0.7114  V